In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

import scipy
import pylab
import math
import random

import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.neighbors import NearestNeighbors
from pandas.api.types import is_numeric_dtype
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import haversine_distances

%matplotlib inline
sns.set()

In [76]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 50

In [173]:
#загрузка 310
df_310 = pd.read_excel('Квартиры_ММО_310.xlsx', sheet_name = 0)
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО
1,2,2682,GUAR_OBJECT_4302_130_1,50:08:0050435:778,231.5,2024-11-01,75211000.0,2023-11-29,NaN,"Россия, Московская область, городской округ Ис...",55.791025,37.013437,МО
2,3,2682,GUAR_OBJECT_4335_76_1,50:23:0000000:117210,99.3,2024-11-01,12787000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579169,38.216326,МО
3,4,2682,GUAR_OBJECT_4335_77_1,50:23:0110103:665,69.9,2024-11-01,9144000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579444,38.218060,МО
4,5,2682,GUAR_OBJECT_4338_83_1,50:23:0020216:277,96.7,2024-11-01,7450928.0,2023-03-17,NaN,"Россия, Московская область, Раменский городско...",55.584843,38.262401,МО
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4485,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК
4486,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК
4487,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК
4488,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК


### Полигоны спутников

In [19]:
polygons_sputnik = {'Одинцово': [(55.693741, 37.296200), (55.693038, 37.294612), (55.688480, 37.289419), (55.686126, 37.281033), (55.682955, 37.281158),
                            (55.682921, 37.275364), (55.685600, 37.274613), (55.686839, 37.271849), (55.689760, 37.264940), (55.690561, 37.258292),
                            (55.689605, 37.251555), (55.682744, 37.242955), (55.679752, 37.234286), (55.674252, 37.225806), (55.666311, 37.224771),
                            (55.667539, 37.241637), (55.663813, 37.243946), (55.661969, 37.238024), (55.658573, 37.238539), (55.659058, 37.247851),
                            (55.657753, 37.249057), (55.656239, 37.254134), (55.656239, 37.255293), (55.652330, 37.257284), (55.650122, 37.260438),
                            (55.653104, 37.276583), (55.655589, 37.289749), (55.657025, 37.305147), (55.662944, 37.308838), (55.665613, 37.307079),
                            (55.671238, 37.312941), (55.672802, 37.305645), (55.674512, 37.306246), (55.680540, 37.305255), (55.683006, 37.316160),
                            (55.687116, 37.311160), (55.690623, 37.310525), (55.692454, 37.304190), (55.694114, 37.302173),
                            (55.694163, 37.298955)],
            'Трехгорка': [(55.708141, 37.330163), (55.707729, 37.317417), (55.698230, 37.316962), (55.689503, 37.322069), (55.695430, 37.334793),
                           (55.697436, 37.324301), (55.701569, 37.326511), (55.706816, 37.329948)],
            'ВНИИССОК': [(55.660311, 37.195576), (55.654927, 37.191235), (55.654448, 37.193144), (55.653033, 37.192807), (55.653382, 37.200551),
                          (55.652817, 37.209053), (55.654042, 37.215756), (55.655122, 37.217816), (55.653790, 37.225423), (55.654182, 37.238851),
                          (55.659171, 37.238374), (55.662053, 37.237705), (55.660988, 37.230366), (55.661167, 37.222556), (55.660954, 37.212475),
                          (55.660216, 37.199939)],
              'Ромашково': [(55.718884, 37.322175), (55.722808, 37.327746), (55.724455, 37.332939), (55.724964, 37.339590), (55.726640, 37.345204),
                             (55.728505, 37.348079), (55.734414, 37.335805), (55.737900, 37.324956), (55.725749, 37.311215), (55.721365, 37.313833),
                             (55.720759, 37.316493)],
              'Немчиновка': [(55.718845, 37.382681), (55.716193, 37.375383), (55.714258, 37.369898), (55.709162, 37.355833), (55.706267, 37.350932),
                              (55.708944, 37.349104), (55.713141, 37.348070), (55.719752, 37.333243), (55.723097, 37.337835), (55.729750, 37.351993),
                              (55.729537, 37.359829), (55.730099, 37.370803), (55.729372, 37.377073), (55.725168, 37.379570),
                              (55.721133, 37.381716)],
            'Новоивановское': [(55.718551, 37.382836), (55.713384, 37.385836), (55.709275, 37.389089), (55.706191, 37.392361), (55.704885, 37.389189),
                                (55.706218, 37.387451), (55.704854, 37.382533), (55.704161, 37.377396), (55.702888, 37.369950), (55.701056, 37.367067),
                                (55.697760, 37.359664), (55.697849, 37.347232), (55.696928, 37.340494), (55.697716, 37.339571), (55.702340, 37.345815),
                                (55.707991, 37.353703), (55.710864, 37.360930), (55.714935, 37.372466), (55.717200, 37.378920)],
              'Заречье': [(55.698823, 37.402271), (55.693079, 37.410424), (55.689724, 37.413635), (55.683653, 37.416141), (55.680913, 37.417750),
                           (55.677326, 37.408339), (55.671471, 37.397970), (55.671859, 37.385954), (55.680235, 37.378300), (55.691296, 37.380352),
                           (55.693309, 37.391842), (55.696368, 37.394700), (55.697410, 37.399442)],
              'Красногорск': [(55.830639, 37.379684), (55.828803, 37.394104), (55.814810, 37.390138), (55.807589, 37.387924), (55.804302, 37.386121),
                               (55.795687, 37.378311), (55.799216, 37.371358), (55.793221, 37.358656), (55.796812, 37.349526), (55.797148, 37.352137),
                               (55.801337, 37.359092), (55.803764, 37.357827), (55.806489, 37.347878), (55.803982, 37.341659), (55.801177, 37.336386),
                               (55.800137, 37.334561), (55.800802, 37.327825), (55.803548, 37.314006), (55.803983, 37.304308), (55.810779, 37.312592),
                               (55.814646, 37.303421), (55.814646, 37.303421), (55.826816, 37.296658), (55.828545, 37.278307), (55.825453, 37.270239),
                               (55.816321, 37.266634), (55.810569, 37.261038), (55.810666, 37.250910), (55.811502, 37.241716), (55.823796, 37.208140),
                               (55.833666, 37.215469), (55.845880, 37.225151), (55.848884, 37.239039), (55.849057, 37.257578), (55.843589, 37.261209),
                               (55.838448, 37.264131), (55.836359, 37.269474), (55.841522, 37.292520), (55.839831, 37.313205), (55.839863, 37.324290),
                               (55.836747, 37.330384), (55.838969, 37.335662), (55.838945, 37.341241), (55.836578, 37.345404), (55.834163, 37.342314),
                               (55.833047, 37.344903), (55.832480, 37.347800), (55.831154, 37.347109), (55.828992, 37.348469), (55.826311, 37.344736),
                               (55.820409, 37.350581), (55.817968, 37.354186), (55.820640, 37.361112), (55.825915, 37.364668), (55.828258, 37.368702),
                               (55.829808, 37.375182)],
            'Архангельское': [(55.796945, 37.302202), (55.795373, 37.301837), (55.794237, 37.302062), (55.792600, 37.302206), (55.791198, 37.302251),
                               (55.791007, 37.305820), (55.790354, 37.309092), (55.787179, 37.308721), (55.787068, 37.312469), (55.779721, 37.311605),
                               (55.784234, 37.293421), (55.782034, 37.290089), (55.781185, 37.276262), (55.783325, 37.277704), (55.788894, 37.281403),
                               (55.793404, 37.284416), (55.793404, 37.284416), (55.795529, 37.295964), (55.796329, 37.299393)],
            'Новый': [(55.812406, 37.296491), (55.808854, 37.299409), (55.804890, 37.299238), (55.803246, 37.289367), (55.809690, 37.273111),
                       (55.810990, 37.273677), (55.810715, 37.284659), (55.812022, 37.292770)],
        'Глухово_Ильинское-Усово_Бузланово': [(55.786659, 37.258631), (55.781363, 37.262580), (55.764812, 37.257438), (55.764009, 37.246658),
                                               (55.764633, 37.225501), (55.763655, 37.205082), (55.766786, 37.190756), (55.775127, 37.196267),
                                               (55.777225, 37.205071), (55.774895, 37.209990), (55.775761, 37.223371), (55.777096, 37.230040),
                                               (55.779261, 37.235297), (55.782920, 37.242039), (55.782079, 37.252386), (55.784865, 37.256561)],
           'Путилково': [(55.871132, 37.411525), (55.868092, 37.406891), (55.864033, 37.401861), (55.860919, 37.398728), (55.854340, 37.395089),
                          (55.855426, 37.387235), (55.854547, 37.381528), (55.855397, 37.374867), (55.859384, 37.379914), (55.867011, 37.378283),
                          (55.872358, 37.383424), (55.878029, 37.382725), (55.877625, 37.387860), (55.877228, 37.394145), (55.875917, 37.404919),
                          (55.873048, 37.409081)],
              'Отрадное': [(55.874338, 37.339796), (55.869435, 37.336929), (55.867384, 37.33293), (55.867384, 37.332938), (55.862929, 37.324183),
                            (55.864739, 37.320707), (55.864015, 37.316673), (55.870164, 37.310073), (55.870164, 37.310073), (55.871064, 37.306365),
                            (55.868443, 37.302056), (55.870807, 37.296344), (55.875187, 37.293868), (55.878027, 37.297769), (55.878514, 37.306159),
                            (55.879028, 37.311872), (55.882226, 37.316910), (55.876753, 37.326163), (55.879645, 37.330609), (55.879645, 37.330609),
                            (55.877136, 37.335055)], 
               'Химки': [(55.877759, 37.430556), (55.877663, 37.428496), (55.877349, 37.425814), (55.878079, 37.423486), (55.877729, 37.420814),
                          (55.878386, 37.417091), (55.881462, 37.409946), (55.884640, 37.409925), (55.886329, 37.404807), (55.888186, 37.403605),
                          (55.891460, 37.404099), (55.894076, 37.403037), (55.895671, 37.402168), (55.899198, 37.400344), (55.901458, 37.399078),
                          (55.901221, 37.396910), (55.901914, 37.392576), (55.903240, 37.390033), (55.905012, 37.389658), (55.909291, 37.389851),
                          (55.912581, 37.389915), (55.915389, 37.395258), (55.916823, 37.398412), (55.921607, 37.393327), (55.926270, 37.396803),
                          (55.930226, 37.408257), (55.932406, 37.427376), (55.935887, 37.431818), (55.941188, 37.442468), (55.940393, 37.448477),
                          (55.928132, 37.449249), (55.918416, 37.455026), (55.913885, 37.455283), (55.911474, 37.460733), (55.909257, 37.472020),
                          (55.909257, 37.472020), (55.913065, 37.483479), (55.911732, 37.486042), (55.906911, 37.485806), (55.903934, 37.485184),
                          (55.903078, 37.491235), (55.905826, 37.493080), (55.905971, 37.500655), (55.901125, 37.506298), (55.897195, 37.507349),
                          (55.892047, 37.496213), (55.889515, 37.490527), (55.887079, 37.482373), (55.883799, 37.469412), (55.882990, 37.464069),
                          (55.882629, 37.452761), (55.881607, 37.445091), (55.879399, 37.437001), (55.877783, 37.431830)],
          'Планерная_Новогорск_парк': [(55.929899, 37.356660), (55.924370, 37.369664), (55.921755, 37.375758), (55.921213, 37.377775),
                                        (55.921213, 37.377775), (55.917779, 37.370179), (55.915610, 37.366960), (55.911886, 37.367368),
                                        (55.908174, 37.370071), (55.903371, 37.370264), (55.899923, 37.359471), (55.898934, 37.334323),
                                        (55.903756, 37.330246), (55.904287, 37.325740), (55.908240, 37.322392), (55.918773, 37.326512),
                                        (55.924291, 37.343378), (55.927351, 37.350845)],
               'Клязьма': [(55.959961, 37.486629), (55.951342, 37.480793), (55.944503, 37.479248), (55.939541, 37.482595), (55.936651, 37.475986),
                            (55.940842, 37.468433), (55.943732, 37.456846), (55.945080, 37.445087), (55.954135, 37.429252), (55.964703, 37.427192),
                            (55.966316, 37.436590), (55.970094, 37.444229), (55.973560, 37.451310), (55.966460, 37.466631), (55.964270, 37.471995),
                            (55.964559, 37.480707), (55.962224, 37.485556)], 
           'Долгопрудный': [(55.907420, 37.537080), (55.923194, 37.528841), (55.932010, 37.524582), (55.941609, 37.519496), (55.943259, 37.528787),
                             (55.943993, 37.534238), (55.946366, 37.533165), (55.946161, 37.531319), (55.947377, 37.528895), (55.950652, 37.531405),
                             (55.953481, 37.532371), (55.955301, 37.531477), (55.954240, 37.524796), (55.956912, 37.523359), (55.958694, 37.527757),
                             (55.960102, 37.532800), (55.962656, 37.533031), (55.965834, 37.533911), (55.968638, 37.532666), (55.974353, 37.525328),
                             (55.976856, 37.524105), (55.979623, 37.525285), (55.978511, 37.511626), (55.977272, 37.502507), (55.978247, 37.491778),
                             (55.982481, 37.484032), (55.976478, 37.470127), (55.974239, 37.473474), (55.972037, 37.473796), (55.968668, 37.478195),
                             (55.961316, 37.486583), (55.958355, 37.489459), (55.952240, 37.481519), (55.943956, 37.479631), (55.930515, 37.490274),
                             (55.917474, 37.491403), (55.912413, 37.486038), (55.907035, 37.485288), (55.906483, 37.500801), (55.910774, 37.502818),
                             (55.911907, 37.511015), (55.914486, 37.517538), (55.920752, 37.515221), (55.921607, 37.519233), (55.918354, 37.520564),
                             (55.912221, 37.525882)], 
      'Мытищи': [(55.882346, 37.725823), (55.887954, 37.737217), (55.892331, 37.744427), (55.902741, 37.759288), (55.906563, 37.764395),
                  (55.910263, 37.777076), (55.914059, 37.788342), (55.916999, 37.793191), (55.925265, 37.799521), (55.925265, 37.795315),
                  (55.930337, 37.801238), (55.932125, 37.802257), (55.931379, 37.804628), (55.936918, 37.811246), (55.941651, 37.812963),
                  (55.949659, 37.801054), (55.958287, 37.788366), (55.959118, 37.785490), (55.959479, 37.779868), (55.950414, 37.771307),
                  (55.950582, 37.765899), (55.944526, 37.765626), (55.944671, 37.757987), (55.935602, 37.741507), (55.933672, 37.719415),
                  (55.921721, 37.702549), (55.920131, 37.685212), (55.918347, 37.681263), (55.910948, 37.670578), (55.898573, 37.671900),
                  (55.895226, 37.673230), (55.894696, 37.692671), (55.893490, 37.701383), (55.890499, 37.709751), (55.885603, 37.719321)], 
      'Королев': [(55.914265, 37.787771), (55.917229, 37.793436), (55.925206, 37.799530), (55.925230, 37.795303), (55.930345, 37.801301),
                   (55.932068, 37.802288), (55.931297, 37.804606), (55.937103, 37.811397), (55.942895, 37.813564), (55.943473, 37.824958),
                   (55.951143, 37.827576), (55.955429, 37.833477), (55.952588, 37.839271), (55.952504, 37.850150), (55.954346, 37.866651),
                   (55.950758, 37.888516), (55.938976, 37.895682), (55.936820, 37.897227), (55.934947, 37.896068), (55.925709, 37.905083),
                   (55.918634, 37.913807), (55.907003, 37.912204), (55.904650, 37.891106), (55.895536, 37.881708), (55.902312, 37.842912),
                   (55.906338, 37.838835), (55.906410, 37.817807), (55.910797, 37.817850), (55.910701, 37.810039), (55.904313, 37.809739),
                   (55.899900, 37.794246), (55.900985, 37.792057), (55.906458, 37.788753), (55.910098, 37.788066), (55.914268, 37.789611)],
              'Нагорное': [(55.898195, 37.638865), (55.900172, 37.644658), (55.903764, 37.642727), (55.905452, 37.635603), (55.910249, 37.631269),
                            (55.913166, 37.627063), (55.914250, 37.622385), (55.913045, 37.616463), (55.907405, 37.614145), (55.907477, 37.609554),
                            (55.904801, 37.606893), (55.901884, 37.617836), (55.899545, 37.629338), (55.898605, 37.634831)],
              'Бородино': [(55.918545, 37.661326), (55.919593, 37.662571), (55.919690, 37.665167), (55.921063, 37.668536), (55.922316, 37.667967),
                            (55.923166, 37.666047), (55.925497, 37.665747), (55.926268, 37.666079), (55.928605, 37.653795), (55.927491, 37.651466),
                            (55.926238, 37.652840), (55.925214, 37.654760), (55.922815, 37.655919), (55.920067, 37.659245)], 
              'Измайлово_рабпос': [(55.556162, 37.650426), (55.557812, 37.643444), (55.560624, 37.640340), (55.566700, 37.636438),
                                    (55.567454, 37.637731), (55.565218, 37.645847), (55.566555, 37.646819), (55.565950, 37.652272),
                                    (55.563101, 37.661900), (55.560391, 37.660484)],
          'Булатниково_мкр': [(55.560166, 37.622227), (55.560643, 37.621737), (55.562011, 37.626182), (55.560694, 37.629084), (55.560041, 37.627506),
                               (55.559447, 37.624297), (55.560521, 37.623506)],
             'Живописный_ЖК': [(55.555898, 37.632123), (55.557881, 37.635900), (55.557234, 37.638417), (55.555606, 37.641857), (55.554410, 37.639885),
                                (55.552388, 37.632828)], 
              'Южная_Битца_ЖК': [(55.555018, 37.589133), (55.556592, 37.589396), (55.560776, 37.595801), (55.563127, 37.602996), (55.556785, 37.605188),
                                  (55.555864, 37.602868), (55.554723, 37.593448)],
              'Бутово_Парк_ЖК': [(55.541674, 37.584199), (55.550177, 37.587029), (55.550099, 37.588212), (55.548128, 37.592226), (55.544309, 37.596896),
                                  (55.542605, 37.590460)], 
              'Лопатино': [(55.529073, 37.645182), (55.527434, 37.639779), (55.528155, 37.633400), (55.534549, 37.632317), (55.538571, 37.635296),
                                (55.540460, 37.646372), (55.534837, 37.652769), (55.532755, 37.650888)], 
       'Восточное_Бутово_ЖК': [(55.529256, 37.611289), (55.538599, 37.610173), (55.539273, 37.618193), (55.538401, 37.622326), (55.522056, 37.620918),
                                (55.519703, 37.617422), (55.519558, 37.614205), (55.522105, 37.610086)],
              'Дрожжино': [(55.516461, 37.587538), (55.521711, 37.578610), (55.538044, 37.583271), (55.525989, 37.601591), (55.518397, 37.599590)],
               'Видное': [(55.539170, 37.731748), (55.538195, 37.731711), (55.538218, 37.725054), (55.537018, 37.718577), (55.544091, 37.698546),
                            (55.547127, 37.690953), (55.550118, 37.684425), (55.552579, 37.682440), (55.553764, 37.690606), (55.559059, 37.692248),
                            (55.559130, 37.696078), (55.558580, 37.697888), (55.561382, 37.699538), (55.567372, 37.717768), (55.556931, 37.739297),
                            (55.551600, 37.745556), (55.549236, 37.742070), (55.551293, 37.736832), (55.553022, 37.726820), (55.550136, 37.718100),
                            (55.548754, 37.719887), (55.547785, 37.722193), (55.545175, 37.723267), (55.545132, 37.727353)],
              'Расторгуево': [(55.535881, 37.680175), (55.533474, 37.697605), (55.545030, 37.691305), (55.552545, 37.679952), (55.553642, 37.668612),
                               (55.548537, 37.663027), (55.546134, 37.649178), (55.543612, 37.644266), (55.537581, 37.649494), (55.534788, 37.652756),
                               (55.532921, 37.651167), (55.527683, 37.662841), (55.535030, 37.665480), (55.535165, 37.670146), (55.537936, 37.679042),
                               (55.537088, 37.678986)], 
          'Первый_Южный_ЖК': [(55.571271, 37.691030), (55.562965, 37.685999), (55.563590, 37.682727), (55.563658, 37.680169), (55.563366, 37.678035),
                               (55.565522, 37.678044), (55.566541, 37.675968), (55.565733, 37.675518), (55.565782, 37.674451), (55.568418, 37.674793),
                               (55.569489, 37.677149), (55.569067, 37.682022), (55.572001, 37.690177)], 
          'Новый_мкр': [(55.550677, 37.660620), (55.550335, 37.662793), (55.549113, 37.663519), (55.549635, 37.660100)], 
              'Усадьба_Суханово_ЖК': [(55.531336, 37.667245), (55.534075, 37.666659), (55.534443, 37.669872), (55.534078, 37.671170),
                                       (55.532373, 37.669807)], 
              'Сапроново': [(55.528367, 37.726886), (55.523901, 37.718034), (55.525225, 37.714258), (55.528158, 37.713903), (55.529385, 37.716291),
                             (55.530543, 37.715458), (55.531058, 37.716533), (55.531726, 37.716491), (55.531075, 37.709694), (55.533961, 37.707590),
                             (55.537884, 37.722879), (55.537995, 37.731655)],
          'Совхоз_им_Ленина': [(55.585829, 37.715597), (55.590133, 37.726782), (55.588947, 37.732379), (55.582969, 37.745159), (55.581055, 37.742372),
                                (55.581355, 37.728143), (55.582643, 37.722049)],
             'Развилка': [(55.587871, 37.763772), (55.583564, 37.755882), (55.585103, 37.752461), (55.590702, 37.747383), (55.593372, 37.749431),
                           (55.595085, 37.742850), (55.599344, 37.747701), (55.597988, 37.760978), (55.598426, 37.769152), (55.600264, 37.768643),
                           (55.600656, 37.773648), (55.596655, 37.774902), (55.595827, 37.763741), (55.594390, 37.760784), (55.590960, 37.761554),
                           (55.590958, 37.762331), (55.590642, 37.762143), (55.590341, 37.764109), (55.588662, 37.763544)],
              'Мисайлово': [(55.550585, 37.809183), (55.553659, 37.798104), (55.560366, 37.802839), (55.559970, 37.807741), (55.561902, 37.809329),
                             (55.562565, 37.821648), (55.557302, 37.824294), (55.557381, 37.840562), (55.550330, 37.840395), (55.554674, 37.826540),
                             (55.549503, 37.826233), (55.549464, 37.821361)],
              'Молоково': [(55.561673, 37.861774), (55.561488, 37.840986), (55.563796, 37.841131), (55.565784, 37.840382), (55.565399, 37.860307),
                            (55.564169, 37.862650), (55.564903, 37.863934), (55.564240, 37.865201), (55.562489, 37.862187)], 
              'Дзержинский': [(55.631437, 37.871060), (55.628565, 37.871502), (55.625111, 37.869078), (55.621395, 37.870210), (55.624572, 37.864185),
                               (55.625003, 37.855772), (55.623510, 37.850308), (55.621746, 37.849241), (55.621129, 37.846388), (55.622421, 37.842560),
                               (55.623254, 37.842352), (55.623638, 37.840793), (55.624445, 37.838949), (55.624445, 37.836252), (55.626700, 37.836390),
                               (55.627156, 37.839041), (55.626780, 37.844645), (55.630919, 37.843150), (55.632831, 37.840775), (55.634122, 37.839919),
                               (55.641017, 37.840964), (55.639556, 37.844808), (55.637749, 37.848677), (55.639303, 37.854245), (55.634951, 37.875223),
                               (55.632460, 37.874046)], 
              'Лыткарино': [(55.573338, 37.912757), (55.569011, 37.908645), (55.573973, 37.897477), (55.579972, 37.893109), (55.584087, 37.890958),
                             (55.589122, 37.885473), (55.591405, 37.884847), (55.594595, 37.884609), (55.596896, 37.876066), (55.598501, 37.875664),
                             (55.599492, 37.881262), (55.597551, 37.883848), (55.598783, 37.890570), (55.597440, 37.891470), (55.592305, 37.890114),
                             (55.589684, 37.908260), (55.586263, 37.915926), (55.585105, 37.915728), (55.582771, 37.913306), (55.578183, 37.911994),
                             (55.575381, 37.914206), (55.574983, 37.912515)],
         'Томилино_Парк_ЖК': [(55.624941, 37.953335), (55.621915, 37.958318), (55.618296, 37.951834), (55.616273, 37.941307), (55.619917, 37.939083),
                               (55.624078, 37.937765), (55.625570, 37.938184), (55.627270, 37.941327), (55.627323, 37.944460),
                               (55.626613, 37.948702)], 
        'Октябрьский': [(55.606400, 37.986145), (55.603511, 37.970444), (55.604763, 37.965728), (55.607171, 37.964523), (55.607915, 37.968314),
                                (55.611004, 37.966298), (55.611760, 37.970413), (55.613954, 37.970463), (55.615069, 37.968682), (55.615808, 37.970789),
                                (55.615888, 37.973341), (55.614960, 37.976197), (55.613319, 37.976498), (55.609967, 37.980265), (55.610490, 37.983924),
                                (55.608762, 37.986233)], 
              'Островцы': [(55.605077, 37.980284), (55.606155, 37.986382), (55.604404, 37.991341), (55.602781, 37.991479), (55.595021, 37.987944),
                            (55.596888, 37.977710), (55.599736, 37.973710), (55.603350, 37.981405)], 
              'Жилино-1': [(55.637348, 37.948637), (55.638011, 37.940109), (55.639665, 37.940130), (55.641976, 37.945907),
                            (55.639328, 37.950998)],
              'Люберцы': [(55.656287, 37.898824), (55.675961, 37.857024), (55.677118, 37.858722), (55.687223, 37.864423), (55.687223, 37.864423),
                           (55.690747, 37.859839), (55.691550, 37.860392), (55.693707, 37.864381), (55.698817, 37.856388), (55.700121, 37.858495),
                           (55.700699, 37.857337), (55.701199, 37.860152), (55.695380, 37.871585), (55.700895, 37.881074), (55.701192, 37.880539),
                           (55.705299, 37.887215), (55.707209, 37.905369), (55.704823, 37.911414), (55.697318, 37.914370), (55.699515, 37.918899),
                           (55.696865, 37.924711), (55.698205, 37.926705), (55.697956, 37.939954), (55.696622, 37.939938), (55.696835, 37.936210),
                           (55.695139, 37.928370), (55.688548, 37.922087), (55.688082, 37.918754), (55.683052, 37.913180), (55.680499, 37.919027),
                           (55.668617, 37.923496), (55.660729, 37.939650)],
              'Котельники': [(55.673346, 37.850400), (55.677209, 37.853701), (55.662049, 37.886351), (55.653462, 37.875781), (55.653035, 37.872992),
                              (55.652299, 37.873134), (55.651883, 37.868907), (55.654724, 37.864758), (55.653169, 37.861386), (55.647114, 37.865400),
                              (55.642959, 37.860612), (55.639297, 37.854251), (55.637742, 37.848680), (55.640672, 37.841860), (55.659464, 37.841629),
                              (55.666352, 37.853261), (55.674143, 37.854541)], 
              'Томилино': [(55.649342, 37.932059), (55.658357, 37.917524), (55.661300, 37.944304), (55.662755, 37.943769), (55.663028, 37.950260),
                            (55.666395, 37.950395), (55.669744, 37.944668), (55.671725, 37.944945), (55.671497, 37.957724), (55.665547, 37.952844),
                            (55.662219, 37.952926), (55.662433, 37.955528), (55.665668, 37.956654), (55.665807, 37.959761), (55.666654, 37.960542),
                            (55.666358, 37.962299), (55.662247, 37.961315), (55.661563, 37.965511), (55.652233, 37.970033), (55.656166, 37.951792),
                            (55.654197, 37.952471), (55.653733, 37.949305), (55.653094, 37.949476), (55.651595, 37.946597)], 
              'Красково': [(55.658986, 37.977506), (55.657414, 37.977019), (55.657511, 37.974942), (55.660625, 37.972932), (55.663590, 37.976219),
                            (55.662428, 37.983153), (55.660955, 37.986555), (55.666247, 37.994017), (55.665798, 38.002606), (55.659531, 38.001957),
                            (55.656536, 37.996302), (55.659826, 37.990708), (55.657256, 37.984993)], 
              'Коренёво': [(55.670243, 38.009167), (55.668932, 38.007304), (55.671777, 38.000926), (55.673068, 38.000525), (55.674620, 38.001629),
                            (55.675435, 37.996891), (55.676628, 37.998557), (55.677878, 37.996207), (55.678281, 37.996914), (55.677073, 37.999126),
                            (55.679732, 38.003850), (55.678895, 38.005304), (55.677468, 38.002470), (55.673859, 38.007843), (55.672201, 38.007276),
                            (55.671368, 38.006179)], 
              'Малаховка': [(55.633007, 37.983130), (55.634406, 37.976175), (55.638688, 37.978329), (55.641405, 37.977903), (55.654272, 37.992058),
                             (55.662667, 38.008118), (55.659468, 38.024667), (55.658094, 38.038064), (55.642995, 38.047627), (55.634930, 38.024530),
                             (55.629071, 38.025557), (55.629154, 38.018099), (55.624309, 38.017750), (55.624880, 38.009774), (55.620727, 38.000655),
                             (55.622983, 37.995935), (55.627103, 38.000609), (55.630555, 37.982256)], 
      'Малаховский_квартал_Заречье': [(55.633229, 37.978988), (55.632727, 37.983145), (55.629552, 37.981460), (55.630417, 37.976849)], 
              'Зенино': [(55.694335, 37.955747), (55.696184, 37.951130), (55.704533, 37.959391), (55.705962, 37.958074), (55.708584, 37.947674),
                          (55.711368, 37.950111), (55.709147, 37.964130), (55.702283, 37.967228), (55.693798, 37.959669)],
              'Марусино': [(55.692948, 37.969573), (55.691630, 37.963451), (55.693314, 37.963836), (55.693863, 37.965405), (55.695114, 37.964307),
                            (55.695772, 37.967558), (55.695142, 37.968972), (55.697347, 37.973380), (55.695268, 37.974232), (55.694636, 37.978018),
                            (55.692891, 37.977956), (55.692866, 37.974497), (55.691463, 37.974648), (55.690486, 37.970162)],
              'Мотяково': [(55.696072, 38.002274), (55.696837, 38.002693), (55.699550, 38.029041), (55.698534, 38.027869), (55.697491, 38.025503),
                            (55.695024, 38.017868), (55.695375, 38.012844), (55.694077, 38.012433), (55.694186, 38.008174),
                            (55.695970, 38.005881)],
              'Павлино_мкр': [(55.732174, 37.970995), (55.732904, 37.967174), (55.729886, 37.961097), (55.726856, 37.957707), (55.724947, 37.965734),
                           (55.721311, 37.967413), (55.714742, 37.974079), (55.711521, 37.972482), (55.711540, 37.974010), (55.723552, 37.979837),
                           (55.723027, 37.984751), (55.725164, 37.986233)], 
              'Реутов': [(55.771823, 37.861402), (55.771445, 37.865755), (55.768675, 37.864837), (55.768673, 37.867390), (55.767727, 37.867867),
                          (55.767087, 37.864782), (55.761525, 37.868397), (55.759629, 37.868078), (55.760200, 37.862548), (55.752280, 37.862130),
                          (55.757291, 37.888230), (55.753100, 37.890440), (55.751677, 37.890587), (55.751419, 37.887991), (55.749703, 37.884826),
                          (55.745266, 37.861386), (55.744513, 37.846831), (55.752797, 37.844606), (55.753105, 37.846865), (55.761311, 37.846723),
                          (55.765212, 37.851603), (55.768161, 37.846113), (55.774409, 37.846076), (55.775132, 37.846734), (55.775958, 37.848549),
                          (55.770349, 37.856373)],
          'Салтыковка_мкр': [(55.762217, 37.937599), (55.743381, 37.946038), (55.742311, 37.942739), (55.744073, 37.927736), (55.749013, 37.923474),
                              (55.765899, 37.930914), (55.766402, 37.934510), (55.764138, 37.935457), (55.764531, 37.937967)], 
              'Кучино_мкр': [(55.755119, 37.960922), (55.744224, 37.964909), (55.742483, 37.964704), (55.742410, 37.959386), (55.747256, 37.960589),
                              (55.748569, 37.951224), (55.746193, 37.948756), (55.751015, 37.942041), (55.755415, 37.947755), (55.755742, 37.951490),
                              (55.757838, 37.950082), (55.758437, 37.952459), (55.755135, 37.954999), (55.755020, 37.959273), (55.756265, 37.959948),
                              (55.758813, 37.964372), (55.761286, 37.963460), (55.760642, 37.958906), (55.763538, 37.957950), (55.765580, 37.963643),
                              (55.764328, 37.969612), (55.756805, 37.968643)], 
             'Керамик_мкр': [(55.747918, 37.978133), (55.750919, 37.978392), (55.750797, 37.971272), (55.748754, 37.965401), (55.744248, 37.965034),
                              (55.743468, 37.974660), (55.745093, 37.977305)],
              'Ольгино_мкр': [(55.751541, 37.986190), (55.749317, 37.985890), (55.748845, 37.983381), (55.744971, 37.982880), (55.738802, 37.981310),
                               (55.738554, 37.987691), (55.736490, 37.988352), (55.736067, 37.995258), (55.743113, 37.995733), (55.745273, 37.993984),
                               (55.747115, 37.993879), (55.749310, 37.994928), (55.751648, 37.994859)],
      'Железнодорожный_мкр': [(55.755081, 37.998377), (55.751645, 37.994856), (55.749312, 37.994917), (55.747116, 37.993867), (55.745272, 37.993974),
                               (55.743113, 37.995718), (55.736074, 37.995247), (55.736495, 37.988353), (55.732825, 37.987762), (55.732721, 38.010641),
                               (55.743411, 38.034876), (55.743421, 38.040939), (55.748623, 38.033564), (55.748682, 38.030414), (55.749898, 38.030409),
                               (55.749863, 38.027712), (55.751315, 38.027659), (55.754901, 38.021995)], 
              'Саввино_мкр': [(55.726386, 38.021036), (55.732719, 38.010629), (55.743418, 38.034873), (55.743437, 38.040091), (55.737635, 38.040959),
                               (55.735203, 38.040563), (55.732566, 38.034065), (55.730768, 38.038103), (55.726523, 38.033303)],
              'Балашиха-1': [(55.786139, 37.976693), (55.786886, 37.941210), (55.790237, 37.931625), (55.793153, 37.931537), (55.794503, 37.926658),
                            (55.804018, 37.927349), (55.810858, 37.982602), (55.805787, 38.005060), (55.796709, 38.014943), (55.793717, 38.015660),
                            (55.790709, 38.013082)], 
              'Измайловский_Лес_ЖК': [(55.783296, 37.846512), (55.787835, 37.845801), (55.787879, 37.857334), (55.783818, 37.857360),
                                       (55.783328, 37.853042), (55.783501, 37.848941)], 
      'Новое_Измайлово_мкр': [(55.785624, 37.880115), (55.783851, 37.881669), (55.782412, 37.880586), (55.782560, 37.876724), (55.782928, 37.874936),
                               (55.784366, 37.873976), (55.785817, 37.880165), (55.789671, 37.877792), (55.791125, 37.884093), (55.792711, 37.882777),
                               (55.794247, 37.889069), (55.793877, 37.895606), (55.789967, 37.894445)],
          'Дзержинского_мкр': [(55.775182, 37.900328), (55.773330, 37.891288), (55.774979, 37.886719), (55.777129, 37.889206), (55.777939, 37.886417),
                                (55.781206, 37.886623), (55.780831, 37.896273), (55.779492, 37.904754), (55.778948, 37.904645)],
            'Балашиха-2': [(55.826206, 37.916663), (55.826865, 37.898267), (55.829730, 37.916268), (55.838030, 37.917997), (55.840977, 37.930704),
                            (55.843595, 37.978273), (55.836325, 37.982928), (55.819041, 37.975205), (55.812460, 37.965912), (55.805975, 37.941312),
                            (55.808138, 37.938795), (55.811285, 37.941793), (55.809746, 37.947407), (55.820439, 37.949531),
                            (55.823614, 37.921157)], 
              'Балашиха-2': [(55.826206, 37.916663), (55.826865, 37.898267), (55.829730, 37.916268), (55.838030, 37.917997), (55.840977, 37.930704),
                            (55.843595, 37.978273), (55.836325, 37.982928), (55.819041, 37.975205), (55.812460, 37.965912), (55.805975, 37.941312),
                            (55.808138, 37.938795), (55.811285, 37.941793), (55.809746, 37.947407), (55.820439, 37.949531),
                            (55.823614, 37.921157)],
              '1_Мая_мкр': [(55.810323, 37.840335), (55.810513, 37.844175), (55.805597, 37.847566), (55.801840, 37.847558),
                             (55.802622, 37.840593)],
             'Янтарный_мкр': [(55.820785, 37.878461), (55.820137, 37.873946), (55.822537, 37.872423), (55.825349, 37.891065), (55.820706, 37.891568),
                               (55.819723, 37.880405), (55.820074, 37.879206)] 
             }

In [20]:
def point_inside_polygon(x, y, poly, include_edges = True):
    '''
    Проверьте, находится ли точка (x,y) внутри многоугольника poly.

    poly - это многоугольник с N вершинами, определенный как 
    [(x1,y1),...,(xN,yN)] или [(x1,y1),...,(xN,yN),(x1,y1)]
    (функция отлично работает в обоих случаях)

    Геометрическая идея: точка находится внутри многоугольника, если горизонтальный луч
    справа от точки пересекает многоугольник четное число раз. 
    Отлично работает для невыпуклых многоугольников.
    '''
    n = len(poly)
    inside = False

    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if p1y == p2y:
            if y == p1y:
                if min(p1x, p2x) <= x <= max(p1x, p2x):
                    # point is on horisontal edge
                    inside = include_edges
                    break
                elif x < min(p1x, p2x):  # point is to the left from current edge
                    inside = not inside
        else:  # p1y!= p2y
            if min(p1y, p2y) <= y <= max(p1y, p2y):
                xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x

                if x == xinters:  # point is right on the edge
                    inside = include_edges
                    break

                if x < xinters:  # point is to the left from current edge
                    inside = not inside

        p1x, p1y = p2x, p2y

    return inside

In [21]:
from tqdm import tqdm

k = len(df_310['lat']) #Широта

districts_sputnik = []

for i in tqdm(range(0, k, 1), 'Обработано'):
    for j, c, p in zip(range(1, len(polygons_sputnik.keys()) + 1, 1), polygons_sputnik.keys(), polygons_sputnik.values()): #enumerate(polygons.items())
        poly = p
        x = df_310['lat'][i] 
        y = df_310['lon'][i]
        res = point_inside_polygon(x, y, poly, include_edges = True)

        if res == True:
            district = c
            districts_sputnik.append(district)
            break
        
        if (j == len(polygons_sputnik.keys()) and res == False):
            district = 999
            districts_sputnik.append(district)

Обработано: 100%|██████████| 4490/4490 [00:09<00:00, 496.76it/s]


In [22]:
df_310['poly_sputnik'] = pd.DataFrame(districts_sputnik, columns = ['poly_sputnik'])
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье
1,2,2682,GUAR_OBJECT_4302_130_1,50:08:0050435:778,231.5,2024-11-01,75211000.0,2023-11-29,NaN,"Россия, Московская область, городской округ Ис...",55.791025,37.013437,МО,999
2,3,2682,GUAR_OBJECT_4335_76_1,50:23:0000000:117210,99.3,2024-11-01,12787000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579169,38.216326,МО,999
3,4,2682,GUAR_OBJECT_4335_77_1,50:23:0110103:665,69.9,2024-11-01,9144000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579444,38.218060,МО,999
4,5,2682,GUAR_OBJECT_4338_83_1,50:23:0020216:277,96.7,2024-11-01,7450928.0,2023-03-17,NaN,"Россия, Московская область, Раменский городско...",55.584843,38.262401,МО,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4485,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999
4486,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999
4487,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999
4488,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999


### Полигоны округов

In [23]:
polygons_o = {'ЦАО': [(55.758523, 37.514315), (55.761734, 37.522611), (55.759960, 37.524068), (55.763399, 37.532165), (55.765027, 37.534036),
                       (55.770157, 37.537550), (55.772432, 37.541764), (55.773657, 37.554441), (55.775326, 37.569902), (55.775645, 37.576512),
                       (55.777899, 37.582124), (55.779388, 37.584201), (55.780861, 37.585400), (55.792161, 37.588633), (55.792482, 37.599288),
                       (55.785132, 37.601742), (55.785132, 37.601742), (55.782573, 37.612369), (55.783400, 37.612945), (55.783527, 37.615062),
                       (55.786415, 37.616795), (55.793141, 37.616306), (55.791883, 37.634613), (55.797154, 37.634947), (55.795143, 37.639548),
                       (55.789850, 37.647921), (55.782217, 37.688370), (55.782061, 37.700490), (55.781211, 37.705170), (55.779233, 37.709754),
                       (55.775328, 37.713666), (55.770690, 37.704579), (55.770021, 37.702536), (55.766758, 37.697212), (55.769494, 37.692178),
                       (55.767541, 37.688675), (55.763991, 37.685772), (55.760540, 37.684498), (55.759044, 37.680459), (55.759281, 37.672890),
                       (55.757957, 37.670735), (55.753436, 37.673056), (55.752558, 37.672680), (55.750754, 37.670524), (55.748408, 37.682138),
                       (55.743065, 37.693905), (55.740435, 37.697294), (55.736339, 37.699712), (55.735657, 37.698468), (55.729322, 37.693609),
                       (55.733110, 37.685318), (55.730155, 37.685798), (55.730083, 37.683751), (55.729508, 37.683849), (55.729206, 37.683287),
                       (55.729075, 37.681277), (55.728423, 37.681395), (55.728374, 37.680508), (55.727792, 37.680314), (55.727304, 37.677570),
                       (55.725788, 37.670583), (55.730922, 37.667208), (55.731810, 37.665615), (55.730698, 37.660637), (55.726749, 37.663078),
                       (55.726628, 37.656593), (55.728916, 37.654270), (55.727984, 37.652153), (55.729472, 37.645933), (55.727979, 37.644865),
                       (55.722389, 37.643704), (55.722188, 37.641936), (55.723008, 37.639573), (55.723510, 37.636443), (55.722037, 37.635822),
                       (55.722529, 37.633060), (55.721418, 37.627570), (55.720090, 37.615553), (55.721034, 37.614376), (55.721407, 37.610078),
                       (55.719511, 37.609335), (55.721250, 37.600272), (55.719833, 37.598121), (55.720137, 37.597463), (55.719980, 37.597229),
                       (55.719980, 37.597229), (55.713878, 37.588035), (55.711319, 37.585319), (55.711071, 37.581107), (55.714030, 37.577165),
                       (55.711390, 37.568351), (55.710345, 37.559297), (55.713279, 37.546688), (55.720110, 37.541023), (55.726753, 37.544217),
                       (55.736854, 37.566028), (55.745439, 37.574154), (55.752063, 37.572229), (55.753614, 37.562693), (55.747179, 37.545255),
                       (55.744626, 37.533485), (55.745377, 37.525371), (55.747050, 37.522622)],  
            'САО': [(55.759724, 37.524292), (55.777965, 37.506042), (55.794056, 37.502747), (55.812463, 37.487279), (55.818550, 37.486363),
                      (55.824375, 37.492228), (55.826951, 37.490167), (55.826536, 37.478295), (55.829427, 37.475128), (55.870335, 37.453904),
                      (55.879182, 37.437275), (55.882736, 37.448714), (55.883004, 37.462293), (55.887688, 37.484037), (55.904380, 37.523660),
                      (55.907337, 37.536429), (55.901704, 37.541571), (55.892747, 37.565527), (55.890616, 37.567023), (55.877006, 37.568608),
                      (55.862249, 37.574690), (55.854418, 37.572736), (55.844214, 37.575242), (55.840613, 37.578258), (55.838860, 37.578772),
                      (55.836495, 37.577962), (55.832365, 37.573771), (55.822417, 37.574003), (55.812086, 37.579099), (55.808700, 37.584782),
                      (55.808244, 37.580820), (55.805278, 37.582443), (55.794878, 37.584447), (55.792349, 37.588713), (55.781017, 37.585611),
                      (55.779084, 37.584079), (55.776131, 37.578906), (55.773209, 37.546381), (55.772011, 37.541030), (55.770236, 37.537566),
                      (55.765036, 37.534081), (55.763440, 37.532233)], 
            'САО_Молжаниновский': [(55.916185, 37.397041), (55.921158, 37.377736), (55.929950, 37.356645), (55.924246, 37.343406), (55.931074, 37.339444),
                       (55.933899, 37.331319), (55.935379, 37.335057), (55.934820, 37.336403), (55.935012, 37.336793), (55.935742, 37.335891),
                       (55.939045, 37.343416), (55.936728, 37.349570), (55.937897, 37.352106), (55.938854, 37.353738), (55.940385, 37.350159),
                       (55.940869, 37.351041), (55.946590, 37.341405), (55.946228, 37.340538), (55.948548, 37.338020), (55.948860, 37.338431),
                       (55.951205, 37.336126), (55.953706, 37.335015), (55.954912, 37.339623), (55.956101, 37.342321), (55.951941, 37.348857),
                       (55.952505, 37.350383), (55.951154, 37.353488), (55.951708, 37.354866), (55.952974, 37.355447), (55.953926, 37.356305),
                       (55.955885, 37.358913), (55.955970, 37.373551), (55.955562, 37.376309), (55.954255, 37.375055), (55.953606, 37.375323),
                       (55.950910, 37.381494), (55.951430, 37.383637), (55.952572, 37.382796), (55.953210, 37.384573), (55.952907, 37.386107),
                       (55.953341, 37.387610), (55.952984, 37.389372), (55.952388, 37.389246), (55.952019, 37.390053), (55.952244, 37.392842),
                       (55.951412, 37.393016), (55.950753, 37.391861), (55.949837, 37.391752), (55.949017, 37.392207), (55.948701, 37.393061),
                       (55.948677, 37.394201), (55.951387, 37.407052), (55.951518, 37.407646), (55.952859, 37.408631), (55.952925, 37.407661),
                       (55.953246, 37.407453), (55.953916, 37.408183), (55.954188, 37.409672), (55.954003, 37.410508), (55.953287, 37.410757),
                       (55.953148, 37.411302), (55.953435, 37.412094), (55.953924, 37.412168), (55.953704, 37.413317), (55.954179, 37.413121),
                       (55.954558, 37.413925), (55.954365, 37.416169), (55.953319, 37.416879), (55.953169, 37.419608), (55.952438, 37.420366),
                       (55.951594, 37.419820), (55.951326, 37.422358), (55.949262, 37.420769), (55.947248, 37.420893), (55.946592, 37.418919),
                       (55.945751, 37.415418), (55.945240, 37.410392), (55.939200, 37.407432), (55.925511, 37.410176), (55.924033, 37.408263),
                       (55.926238, 37.396824), (55.922833, 37.394367), (55.920553, 37.394132), (55.916797, 37.398429)],
              'СВАО': [(55.792178, 37.588355), (55.794816, 37.584535), (55.805172, 37.582567), (55.808273, 37.580756), (55.808705, 37.585004),
                        (55.811938, 37.579418), (55.811938, 37.579418), (55.822272, 37.574123), (55.832103, 37.573817), (55.836332, 37.578285),
                        (55.840728, 37.578691), (55.844096, 37.575562), (55.852659, 37.572764), (55.859603, 37.574631), (55.861956, 37.574714),
                        (55.877604, 37.568310), (55.890316, 37.567130), (55.892157, 37.565917), (55.893566, 37.563712), (55.900980, 37.543048),
                        (55.904147, 37.538385), (55.907063, 37.537043), (55.911393, 37.578074), (55.909933, 37.588738), (55.901170, 37.622182),
                        (55.897321, 37.644383), (55.895299, 37.673098), (55.894602, 37.695442), (55.892145, 37.707172), (55.878818, 37.733904),
                        (55.876999, 37.730999), (55.874953, 37.731556), (55.872981, 37.724273), (55.873358, 37.719328), (55.869629, 37.715035),
                        (55.864970, 37.706901), (55.863187, 37.708973), (55.860857, 37.709319), (55.860558, 37.718039), (55.852972, 37.721656),
                        (55.850567, 37.720343), (55.846895, 37.714062), (55.848015, 37.690356), (55.828445, 37.671636), (55.822524, 37.669889),
                        (55.797841, 37.651243), (55.795868, 37.653624), (55.795309, 37.651477), (55.796270, 37.650992), (55.796248, 37.650382),
                        (55.792777, 37.650126), (55.789009, 37.651607), (55.797320, 37.635252), (55.791886, 37.634640), (55.793028, 37.616298),
                        (55.786115, 37.616658), (55.783933, 37.615375), (55.783319, 37.612968), (55.782522, 37.612482), (55.785315, 37.604461),
                        (55.785125, 37.601776), (55.792451, 37.599196)], 
            'СВАО_Северный': [(55.907353, 37.537140), (55.941834, 37.519274), (55.944275, 37.537639), (55.943806, 37.541855), (55.943968, 37.543139),
                         (55.952848, 37.537686), (55.955870, 37.548325), (55.954739, 37.548992), (55.954614, 37.549371), (55.952543, 37.550576),
                         (55.951973, 37.550207), (55.951329, 37.550663), (55.951316, 37.551961), (55.952311, 37.551998), (55.952347, 37.551745),
                         (55.953481, 37.551846), (55.953516, 37.552566), (55.953887, 37.552489), (55.954018, 37.553281), (55.953984, 37.555101),
                         (55.951625, 37.555058), (55.957568, 37.573927), (55.955734, 37.578656), (55.954160, 37.580779), (55.949692, 37.579459),
                         (55.951274, 37.562649), (55.941421, 37.564320), (55.941457, 37.569750), (55.936693, 37.570139), (55.933598, 37.565776),
                         (55.932593, 37.560714), (55.928344, 37.561089), (55.920446, 37.559192), (55.919531, 37.558548),
                         (55.909348, 37.555843)], 
            'ВАО': [(55.789063, 37.651531), (55.792778, 37.650129), (55.796253, 37.650386), (55.796272, 37.651000), (55.795316, 37.651502),
                     (55.795880, 37.653605), (55.797753, 37.650808), (55.799699, 37.651525), (55.824402, 37.670506), (55.848032, 37.690316),
                     (55.846899, 37.714055), (55.850600, 37.720356), (55.852972, 37.721644), (55.860553, 37.718034), (55.860851, 37.709345),
                     (55.863186, 37.708971), (55.864772, 37.706462), (55.869515, 37.714837), (55.872804, 37.718513), (55.872987, 37.724271),
                     (55.874953, 37.731554), (55.876983, 37.730940), (55.878581, 37.733761), (55.829796, 37.828793), (55.822851, 37.836862),
                     (55.813904, 37.839055), (55.776973, 37.842889), (55.715676, 37.838780), (55.709783, 37.836214), (55.729929, 37.776126),
                     (55.736957, 37.761758), (55.738931, 37.750283), (55.745785, 37.737581), (55.749357, 37.724073), (55.752007, 37.719507),
                     (55.756084, 37.717614), (55.762305, 37.720714), (55.766398, 37.720770), (55.775362, 37.713729), (55.780346, 37.707925),
                     (55.782096, 37.699343), (55.782205, 37.688403)],
              'ВАО_Восточный': [(55.806540, 37.856438), (55.813583, 37.845325), (55.816704, 37.852718), (55.817922, 37.857303), (55.818401, 37.857283),
                          (55.818822, 37.858110), (55.819600, 37.857542), (55.819994, 37.859089), (55.819511, 37.859428), (55.819390, 37.860044),
                          (55.819967, 37.862288), (55.820038, 37.862259), (55.821529, 37.868708), (55.821946, 37.868556), (55.822620, 37.872149),
                          (55.820082, 37.873856), (55.820686, 37.878452), (55.819565, 37.880481), (55.820480, 37.884295), (55.816553, 37.884196),
                          (55.814250, 37.879808), (55.812752, 37.879548), (55.811540, 37.878181), (55.808696, 37.873744), (55.807715, 37.870197),
                          (55.806324, 37.871437), (55.804035, 37.866408), (55.804075, 37.865599), (55.808942, 37.861721), (55.808720, 37.860056),
                          (55.807950, 37.860526)],
             'ВАО_Новокосино': [(55.731014, 37.840422), (55.743982, 37.841861), (55.745240, 37.861403), (55.749213, 37.882678),
                                 (55.748232, 37.888252), (55.741488, 37.890250), (55.742530, 37.885815), (55.742655, 37.881789),
                                 (55.742463, 37.880790), (55.738364, 37.873005), (55.736272, 37.866533), (55.732425, 37.862138),
                                 (55.730431, 37.861256), (55.731039, 37.847530)],
             'ВАО_Косино-Ухтомский': [(55.695283, 37.871671), (55.703331, 37.854682), (55.709834, 37.836049), (55.713518, 37.837939),
                                       (55.731017, 37.840423), (55.731154, 37.847471), (55.730543, 37.861185), (55.732535, 37.862063),
                                       (55.732378, 37.869604), (55.730120, 37.873055), (55.728127, 37.872078), (55.726610, 37.870210),
                                       (55.723458, 37.872322), (55.723876, 37.885954), (55.725546, 37.903467), (55.730630, 37.918332),
                                       (55.731271, 37.923590), (55.729922, 37.930583), (55.726866, 37.937527), (55.721004, 37.945421),
                                       (55.716181, 37.936443), (55.715919, 37.926529), (55.712205, 37.926096), (55.707949, 37.921781),
                                       (55.706961, 37.899397), (55.705237, 37.887005), (55.703259, 37.883883), (55.702475, 37.882330),
                                       (55.701242, 37.880629), (55.700849, 37.881414), (55.700110, 37.879262)],
             'ВАО_Жулебино': [(55.687088, 37.830548), (55.696601, 37.829664), (55.709844, 37.836049), (55.705347, 37.849350),
                               (55.701291, 37.859167), (55.701008, 37.858748), (55.701287, 37.858183), (55.700725, 37.857299),
                               (55.700163, 37.858401), (55.698954, 37.856609), (55.693713, 37.864243), (55.691173, 37.859912),
                               (55.688258, 37.864758), (55.687382, 37.864601), (55.677151, 37.858996), (55.675558, 37.856587),
                               (55.681320, 37.844402)], 
            'ВАО_Некрасовка': [(55.676187, 37.920612), (55.680492, 37.918964), (55.681489, 37.913792), (55.683044, 37.914301),
                                (55.683355, 37.913304), (55.686262, 37.917239), (55.688041, 37.918272), (55.688449, 37.921222),
                                (55.689357, 37.923449), (55.694030, 37.927587), (55.694319, 37.927541), (55.695148, 37.928374),
                                (55.696936, 37.935910), (55.696565, 37.939845), (55.697952, 37.939965), (55.698206, 37.926694),
                                (55.696893, 37.924534), (55.699531, 37.918889), (55.697100, 37.915330), (55.697813, 37.913275),
                                (55.702192, 37.913522), (55.707182, 37.914610), (55.707937, 37.921727), (55.707937, 37.921727),
                                (55.715916, 37.926521), (55.716163, 37.936253), (55.720899, 37.945479), (55.720157, 37.955582),
                                (55.717759, 37.958228), (55.717759, 37.958228), (55.714822, 37.960760), (55.714088, 37.959908),
                                (55.710567, 37.966744), (55.709291, 37.963901), (55.711641, 37.949790), (55.708682, 37.947340),
                                (55.705971, 37.958079), (55.704532, 37.959404), (55.696199, 37.950901), (55.694550, 37.955623),
                                (55.692751, 37.953936), (55.692768, 37.956347), (55.693875, 37.957525), (55.692783, 37.960175),
                                (55.689633, 37.958009), (55.694171, 37.943346), (55.687192, 37.936859), (55.686899, 37.928628),
                                (55.676065, 37.929937)],
            'ЮВАО': [(55.709783, 37.836214), (55.729929, 37.776126), (55.736957, 37.761758), (55.738931, 37.750283), (55.745785, 37.737581),
                      (55.749357, 37.724073), (55.752007, 37.719507), (55.756084, 37.717614), (55.762305, 37.720714), (55.766398, 37.720770),
                      (55.775362, 37.713729), (55.770763, 37.704517), (55.770084, 37.702607), (55.766761, 37.697177), (55.769504, 37.692168),
                      (55.767546, 37.688670), (55.763992, 37.685755), (55.762058, 37.685720), (55.760558, 37.684502), (55.759559, 37.682788),
                      (55.759029, 37.680492), (55.759314, 37.672947), (55.758861, 37.671529), (55.757713, 37.670700), (55.756475, 37.670964),
                      (55.753877, 37.672961), (55.752557, 37.672677), (55.750704, 37.670440), (55.749522, 37.676338), (55.748378, 37.682124),
                      (55.743088, 37.693809), (55.740689, 37.697002), (55.736441, 37.698816), (55.729480, 37.693795), (55.729450, 37.692443),
                      (55.733148, 37.685355), (55.730161, 37.685789), (55.730086, 37.683697), (55.729513, 37.683835), (55.729198, 37.683287),
                      (55.729089, 37.681262), (55.728431, 37.681393), (55.728374, 37.680422), (55.727733, 37.680468), (55.728290, 37.678881),
                      (55.727344, 37.677585), (55.725825, 37.670566), (55.730949, 37.667360), (55.731702, 37.666191), (55.730679, 37.660638),
                      (55.726767, 37.663155), (55.717785, 37.663718), (55.705342, 37.667708), (55.700548, 37.667240), (55.691677, 37.711038),
                      (55.679946, 37.711933), (55.667815, 37.674593), (55.654922, 37.671227), (55.650900, 37.675680), (55.644058, 37.691877),
                      (55.639633, 37.712849), (55.640263, 37.747663), (55.649046, 37.784125), (55.632722, 37.784973), (55.623880, 37.794086),
                      (55.649268, 37.833037), (55.656970, 37.839124), (55.662173, 37.839633), (55.691320, 37.829417), (55.697834, 37.829956)],
            'ЮАО': [(55.726767, 37.663155), (55.717785, 37.663718), (55.705342, 37.667708), (55.700548, 37.667240), (55.691677, 37.711038),
                     (55.679946, 37.711933), (55.667815, 37.674593), (55.654922, 37.671227), (55.650900, 37.675680), (55.644058, 37.691877),
                     (55.639633, 37.712849), (55.640263, 37.747663), (55.649046, 37.784125), (55.632722, 37.784973), (55.623880, 37.794086),
                     (55.602219, 37.754026), (55.591790, 37.729425), (55.575855, 37.687957), (55.572215, 37.673664), (55.572056, 37.664213),
                     (55.575002, 37.614112), (55.576035, 37.596601), (55.577220, 37.588593), (55.581035, 37.572675), (55.607952, 37.569657),
                     (55.634429, 37.577364), (55.642726, 37.579075), (55.645380, 37.580238), (55.642618, 37.589290), (55.642234, 37.591648),
                     (55.641379, 37.605616), (55.656530, 37.605359), (55.663496, 37.613577), (55.664350, 37.605116), (55.672410, 37.604203),
                     (55.680930, 37.614969), (55.684544, 37.613577), (55.683976, 37.609314), (55.684707, 37.606246), (55.686572, 37.604939),
                     (55.690070, 37.605542), (55.691699, 37.601922), (55.695089, 37.597636), (55.706316, 37.587312), (55.706871, 37.588273),
                     (55.711008, 37.581165), (55.711320, 37.585318), (55.713495, 37.587102), (55.720119, 37.597439), (55.719869, 37.598011),
                     (55.721262, 37.600296), (55.719562, 37.609356), (55.721409, 37.610072), (55.721045, 37.614382), (55.720090, 37.615521),
                     (55.721342, 37.627279), (55.722525, 37.633374), (55.722025, 37.635812), (55.723516, 37.636444), (55.723011, 37.639572),
                     (55.722257, 37.641821), (55.722389, 37.643697), (55.727979, 37.644861), (55.729469, 37.645959), (55.727987, 37.652144),
                     (55.728915, 37.654249), (55.726627, 37.656627)], 
            'ЮЗАО': [(55.581035, 37.572675), (55.607952, 37.569657), (55.634429, 37.577364), (55.642726, 37.579075), (55.645380, 37.580238),
                      (55.642618, 37.589290), (55.642234, 37.591648), (55.641379, 37.605616), (55.656530, 37.605359), (55.663496, 37.613577),
                      (55.664350, 37.605116), (55.672410, 37.604203), (55.680930, 37.614969), (55.684544, 37.613577), (55.683976, 37.609314),
                      (55.684707, 37.606246), (55.686572, 37.604939), (55.690070, 37.605542), (55.691699, 37.601922), (55.695089, 37.597636),
                      (55.706316, 37.587312), (55.706871, 37.588273), (55.711008, 37.581165), (55.714109, 37.577097), (55.711254, 37.567972),
                      (55.710338, 37.559263), (55.693398, 37.536264), (55.679706, 37.511645), (55.673648, 37.524143), (55.658259, 37.501129),
                      (55.638936, 37.459564), (55.611150, 37.491980), (55.597963, 37.507356), (55.594491, 37.516601)], 
            'ЮЗАО_Северное Бутово': [(55.549999, 37.577104), (55.553679, 37.573592), (55.554208, 37.567775), (55.553751, 37.560001),
                                      (55.555829, 37.559518), (55.557410, 37.560485), (55.560485, 37.558646), (55.559322, 37.554237),
                                      (55.572415, 37.537233), (55.573195, 37.536855), (55.577310, 37.536889), (55.579580, 37.538690),
                                      (55.587015, 37.547773), (55.576214, 37.591374), (55.568712, 37.597371), (55.560847, 37.594121)], 
            'ЮЗАО_Южное Бутово': [(55.549999, 37.577104), (55.553679, 37.573592), (55.554208, 37.567775), (55.553751, 37.560001),
                                   (55.555829, 37.559518), (55.557410, 37.560485), (55.560485, 37.558646), (55.559322, 37.554237),
                                   (55.552686, 37.548876), (55.552071, 37.544933), (55.555275, 37.529266), (55.548548, 37.514568),
                                   (55.545016, 37.511143), (55.542946, 37.507935), (55.540022, 37.495911), (55.540188, 37.487794),
                                   (55.537327, 37.475549), (55.534674, 37.472796), (55.528162, 37.472867), (55.526404, 37.476523),
                                   (55.530008, 37.489500), (55.526423, 37.504530), (55.526980, 37.506836), (55.525521, 37.519378),
                                   (55.522516, 37.520167), (55.521159, 37.527565), (55.518140, 37.535316), (55.523193, 37.534900),
                                   (55.527215, 37.538720), (55.537681, 37.569633), (55.520463, 37.565392), (55.520341, 37.566914),
                                   (55.517116, 37.566219), (55.516713, 37.567710), (55.517204, 37.576814), (55.549750, 37.586649),
                                   (55.556343, 37.587823)], 
            'ЗАО': [(55.710338, 37.559263), (55.693398, 37.536264), (55.679706, 37.511645), (55.673648, 37.524143), (55.658259, 37.501129),
                     (55.638936, 37.459564), (55.662340, 37.432416), (55.682889, 37.416810), (55.691666, 37.412221), (55.701885, 37.399034),
                     (55.708840, 37.389739), (55.713558, 37.385919), (55.732184, 37.376408), (55.747156, 37.369005), (55.771060, 37.369502),
                     (55.771949, 37.382186), (55.770984, 37.387202), (55.771131, 37.398078), (55.772350, 37.402825), (55.773883, 37.404194),
                     (55.775769, 37.408899), (55.774158, 37.433357), (55.777370, 37.448874), (55.766818, 37.463729), (55.762796, 37.464644),
                     (55.755299, 37.456269), (55.749235, 37.436660), (55.746533, 37.437218), (55.741353, 37.448112), (55.740588, 37.455352),
                     (55.743314, 37.465636), (55.749004, 37.474384), (55.755088, 37.476552), (55.762052, 37.476029), (55.767418, 37.478666),
                     (55.768326, 37.486673), (55.765383, 37.498813), (55.759595, 37.512150), (55.753837, 37.517938), (55.746851, 37.522947),
                     (55.744523, 37.532652), (55.747177, 37.545264), (55.753600, 37.562019), (55.752060, 37.572213), (55.745443, 37.574164),
                     (55.734939, 37.563440), (55.727893, 37.547205), (55.722235, 37.541192), (55.713930, 37.545493)], 
            'ЗАО_Солнцево': [(55.625796, 37.417189), (55.627246, 37.393760), (55.627623, 37.392356), (55.627974, 37.392721),
                              (55.628517, 37.387745), (55.631685, 37.388641), (55.632945, 37.384814), (55.632488, 37.379825),
                              (55.639923, 37.374198), (55.644697, 37.372964), (55.651114, 37.376665), (55.650340, 37.372058),
                              (55.654222, 37.366219), (55.659071, 37.370028), (55.659120, 37.376979), (55.659489, 37.378007),
                              (55.664463, 37.380133), (55.664542, 37.386298), (55.661678, 37.389402), (55.660898, 37.393976),
                              (55.662719, 37.408288), (55.667814, 37.428018), (55.656312, 37.438803), (55.638794, 37.459533),
                              (55.633473, 37.446602)],
            'ЗАО_Ново-Переделкино': [(55.625796, 37.417189), (55.627246, 37.393760), (55.627623, 37.392356), (55.627974, 37.392721),
                                      (55.628517, 37.387745), (55.631685, 37.388641), (55.632945, 37.384814), (55.632488, 37.379825),
                                      (55.639923, 37.374198), (55.644697, 37.372964), (55.651114, 37.376665), (55.650340, 37.372058),
                                      (55.654222, 37.366219), (55.659071, 37.370028), (55.664587, 37.369861), (55.666302, 37.363989),
                                      (55.666073, 37.347575), (55.657751, 37.340961), (55.654082, 37.343075), (55.651915, 37.336112),
                                      (55.641343, 37.330670), (55.635425, 37.337907), (55.630516, 37.356149), (55.628292, 37.363266),
                                      (55.624036, 37.369864), (55.614774, 37.376206)],
           'ЗАО_ИЦ_Сколково': [(55.698751, 37.402361), (55.694798, 37.393540), (55.696726, 37.390180), (55.691738, 37.357933),
                                            (55.685604, 37.355185), (55.682187, 37.361187), (55.676354, 37.354320), (55.678534, 37.348962),
                                            (55.676264, 37.346124), (55.681795, 37.330143), (55.682778, 37.326057), (55.693540, 37.337691),
                                            (55.692553, 37.338658), (55.690038, 37.337933), (55.687330, 37.339009), (55.688193, 37.339876),
                                            (55.688709, 37.342430), (55.689212, 37.343816), (55.690556, 37.343855), (55.693901, 37.342440),
                                            (55.698609, 37.345063), (55.697663, 37.347305), (55.700133, 37.351723), (55.702261, 37.351483),
                                            (55.704780, 37.356112), (55.701764, 37.353118), (55.700451, 37.353326), (55.700614, 37.361408),
                                            (55.700942, 37.365574), (55.703757, 37.365931), (55.704319, 37.368047), (55.707114, 37.366007),
                                            (55.707415, 37.368181), (55.707374, 37.369073), (55.708485, 37.369311), (55.708493, 37.368396),
                                            (55.708054, 37.365101), (55.708953, 37.364239), (55.710832, 37.370321), (55.711737, 37.375414),
                                            (55.708767, 37.377575), (55.696870, 37.378245), (55.698686, 37.382081), (55.697044, 37.385486),
                                            (55.698435, 37.388289), (55.697060, 37.389706), (55.698496, 37.392311), (55.700863, 37.388906),
                                            (55.702150, 37.390368), (55.702524, 37.390298), (55.703271, 37.388115), (55.703552, 37.388441),
                                            (55.704032, 37.388080), (55.706094, 37.393345)],
        'ЗАО_Рублёво-Архангельское': [(55.788213, 37.370545), (55.768619, 37.344509), (55.771037, 37.309236), (55.775425, 37.310778),
                                                 (55.774622, 37.315681), (55.776743, 37.323069), (55.781129, 37.328259), (55.783388, 37.322280),
                                                 (55.786406, 37.312795), (55.779691, 37.313229), (55.780157, 37.311704), (55.784186, 37.311423),
                                                 (55.784441, 37.310515), (55.787788, 37.312905), (55.788286, 37.315260), (55.792881, 37.316328),
                                                 (55.793682, 37.314239), (55.797053, 37.323027), (55.796298, 37.324371), (55.796441, 37.324874),
                                                 (55.796834, 37.324672), (55.797127, 37.323949), (55.797785, 37.321478), (55.797842, 37.320903),
                                                 (55.797655, 37.320131), (55.797276, 37.319176), (55.796621, 37.319250), (55.796454, 37.318467),
                                                 (55.795788, 37.316700), (55.796245, 37.316496), (55.796382, 37.316217), (55.796344, 37.315781),
                                                 (55.796136, 37.315127), (55.796623, 37.315438), (55.796890, 37.315218), (55.796298, 37.313715),
                                                 (55.796080, 37.313575), (55.795976, 37.313644), (55.795668, 37.312604), (55.796869, 37.311793),
                                                 (55.796799, 37.311340), (55.796402, 37.311556), (55.795957, 37.309307), (55.797386, 37.308985),
                                                 (55.798158, 37.314178), (55.799263, 37.314628), (55.799384, 37.316188), (55.800529, 37.319429),
                                                 (55.800374, 37.320282), (55.800763, 37.321731), (55.799633, 37.333342), (55.806861, 37.347994),
                                                 (55.803346, 37.358323), (55.801262, 37.359539), (55.798765, 37.353737), (55.797158, 37.352155),
                                                 (55.796527, 37.348848), (55.792597, 37.357782), (55.789559, 37.368845),
                                                 (55.788752, 37.370035)], 
            'СЗАО': [(55.771060, 37.369502), (55.771949, 37.382186), (55.770984, 37.387202), (55.771131, 37.398078), (55.772350, 37.402825),
                      (55.773883, 37.404194), (55.775769, 37.408899), (55.774158, 37.433357), (55.777370, 37.448874), (55.766818, 37.463729),
                      (55.762796, 37.464644), (55.755299, 37.456269), (55.749235, 37.436660), (55.746533, 37.437218), (55.741353, 37.448112),
                      (55.740588, 37.455352), (55.743314, 37.465636), (55.749004, 37.474384), (55.755088, 37.476552), (55.762052, 37.476029),
                      (55.767418, 37.478666), (55.768326, 37.486673), (55.765383, 37.498813), (55.759595, 37.512150), (55.758524, 37.514335),
                      (55.761125, 37.520987), (55.762467, 37.517492), (55.762964, 37.517970), (55.767263, 37.514272), (55.769630, 37.513369),
                      (55.775112, 37.508863), (55.777980, 37.506566), (55.794012, 37.503002), (55.811362, 37.488286), (55.813280, 37.487325),
                      (55.816999, 37.486628), (55.820220, 37.487018), (55.820220, 37.487018), (55.827052, 37.490260), (55.826555, 37.488237),
                      (55.826648, 37.478004), (55.828566, 37.475865), (55.833363, 37.476788), (55.868884, 37.452957), (55.876352, 37.439381),
                      (55.879398, 37.437310), (55.870890, 37.412018), (55.864516, 37.402231), (55.860639, 37.398702), (55.848070, 37.391971),
                      (55.842910, 37.393402), (55.839930, 37.395311), (55.835460, 37.396140), (55.808831, 37.388351), (55.801710, 37.384262),
                      (55.789728, 37.372638), (55.784623, 37.370111)],
            'СЗАО_Митино': [(55.829754, 37.394383), (55.830678, 37.379889), (55.827986, 37.367453), (55.822193, 37.361537), (55.822193, 37.361537),
                             (55.822255, 37.349155), (55.824434, 37.351659), (55.824646, 37.350845), (55.826386, 37.349711), (55.826614, 37.349116),
                             (55.826082, 37.344787), (55.829074, 37.348534), (55.831033, 37.348181), (55.832210, 37.348713), (55.833089, 37.343485),
                             (55.834010, 37.342094), (55.836537, 37.345069), (55.839208, 37.340522), (55.840264, 37.340671), (55.840571, 37.347262),
                             (55.843881, 37.349253), (55.843881, 37.349253), (55.845486, 37.333146), (55.852702, 37.340641), (55.855277, 37.339505),
                             (55.860581, 37.342423), (55.861348, 37.345224), (55.862276, 37.346630), (55.863814, 37.349703), (55.864310, 37.357810),
                             (55.867411, 37.361564), (55.867345, 37.379307), (55.859477, 37.380047), (55.855173, 37.375006), (55.853446, 37.379774),
                             (55.855862, 37.386977), (55.854180, 37.394529), (55.846745, 37.391154), (55.836264, 37.395528),
                             (55.829758, 37.394386)], 
             'СЗАО_Куркино': [(55.870650, 37.411247), (55.875953, 37.403772), (55.877911, 37.383291), (55.883896, 37.371138), (55.892530, 37.377279),
                               (55.894856, 37.374675), (55.895033, 37.371222), (55.901358, 37.362690), (55.903120, 37.370536), (55.905525, 37.368465),
                               (55.905368, 37.367905), (55.906701, 37.366291), (55.906701, 37.366291), (55.911650, 37.387056), (55.912411, 37.390165),
                               (55.904266, 37.389814), (55.902327, 37.391612), (55.900730, 37.399253), (55.891700, 37.404359), (55.888014, 37.404103),
                               (55.886812, 37.404759), (55.886206, 37.405559), (55.884758, 37.409810), (55.880870, 37.409884), (55.878353, 37.417029),
                               (55.878311, 37.421203), (55.877932, 37.421207), (55.877590, 37.431595), (55.873491, 37.418155)],
              'Зеленоград': [(55.947951, 37.217683), (55.948223, 37.204872), (55.948619, 37.202356), (55.948967, 37.201743), (55.951797, 37.202907),
                              (55.953242, 37.202841), (55.954738, 37.195904), (55.953850, 37.195252), (55.954578, 37.193884), (55.955066, 37.194319),
                              (55.956138, 37.188825), (55.955469, 37.187530), (55.953117, 37.186401), (55.954252, 37.183328), (55.955533, 37.183324),
                              (55.958273, 37.182599), (55.957549, 37.174816), (55.958153, 37.172201), (55.960067, 37.173232), (55.960719, 37.168709),
                              (55.958856, 37.169279), (55.958541, 37.165848), (55.959029, 37.163024), (55.960022, 37.161923), (55.961708, 37.159950),
                              (55.961988, 37.158150), (55.961888, 37.151321), (55.963729, 37.146633), (55.965587, 37.148524), (55.966141, 37.145661),
                              (55.969881, 37.146329), (55.970210, 37.149138), (55.977105, 37.148902), (55.989540, 37.144465), (55.993153, 37.150625),
                              (55.993105, 37.152588), (55.999290, 37.151161), (56.011409, 37.137053), (56.018009, 37.157968), (56.014628, 37.159642),
                              (56.014231, 37.164867), (56.015058, 37.177610), (56.020153, 37.184889), (56.020027, 37.187325), (56.021289, 37.188961),
                              (55.993171, 37.260781), (55.989908, 37.257271), (55.988971, 37.254304), (55.984198, 37.257284), (55.982310, 37.268008),
                              (55.981471, 37.266184), (55.979971, 37.264958), (55.973087, 37.262043), (55.969908, 37.261035), (55.969896, 37.261540),
                              (55.968121, 37.261533), (55.968291, 37.267903), (55.967960, 37.268226), (55.967015, 37.263159), (55.967251, 37.261058),
                              (55.967811, 37.260149), (55.967611, 37.257605), (55.967926, 37.252366), (55.970431, 37.253327), (55.979257, 37.240766),
                              (55.978596, 37.232801), (55.980479, 37.225196), (55.981498, 37.224164), (55.981663, 37.222016), (55.981174, 37.219939),
                              (55.977263, 37.218519), (55.976162, 37.216863), (55.976056, 37.215363), (55.976717, 37.212548), (55.970185, 37.213150),
                              (55.967162, 37.211631), (55.964218, 37.209235), (55.963357, 37.206069), (55.955874, 37.212599), (55.952990, 37.211427),
                              (55.951157, 37.218851), (55.950259, 37.219907)],
            'Новомосковский_административный_округ': [(55.638691, 37.459594), (55.633091, 37.446006), (55.621573, 37.401756), (55.614527, 37.374292),
                                                       (55.623227, 37.371063), (55.629809, 37.360561), (55.632681, 37.343008), (55.632681, 37.343008),
                                                       (55.647319, 37.330852), (55.652699, 37.336618), (55.654506, 37.342781), (55.656984, 37.341269),
                                                       (55.662799, 37.339313), (55.664264, 37.319490), (55.661401, 37.318918), (55.661030, 37.324135),
                                                       (55.651069, 37.322809), (55.651174, 37.311568), (55.646570, 37.311112), (55.651488, 37.271477),
                                                       (55.650060, 37.262134), (55.647881, 37.252660), (55.643618, 37.240326), (55.637446, 37.235170),
                                                       (55.632868, 37.241648), (55.628860, 37.235187), (55.627356, 37.238081), (55.624831, 37.235808),
                                                       (55.619204, 37.227126), (55.622765, 37.214573), (55.622885, 37.206503), (55.618724, 37.185452),
                                                       (55.612886, 37.182305), (55.605282, 37.172046), (55.605530, 37.164036), (55.604274, 37.151905),
                                                       (55.609855, 37.145546), (55.605467, 37.132382), (55.599216, 37.132359), (55.600781, 37.119563),
                                                       (55.605847, 37.116722), (55.600431, 37.095498), (55.590406, 37.087177), (55.575362, 37.099515),
                                                       (55.564103, 37.115983), (55.549640, 37.124626), (55.547346, 37.114922), (55.538740, 37.113206),
                                                       (55.530408, 37.118001), (55.519126, 37.119951), (55.518822, 37.116967), (55.512758, 37.118701),
                                                       (55.513193, 37.138162), (55.500041, 37.136125), (55.496697, 37.123245), (55.487473, 37.126892),
                                                       (55.481868, 37.138854), (55.479942, 37.149183), (55.483939, 37.185171), (55.492918, 37.201800),
                                                       (55.497073, 37.244037), (55.506589, 37.271231), (55.517894, 37.273575), (55.526171, 37.286026),
                                                       (55.530276, 37.299797), (55.539281, 37.298332), (55.543717, 37.310989), (55.539082, 37.313626),
                                                       (55.538816, 37.333636), (55.547056, 37.343597), (55.550494, 37.353529), (55.547870, 37.365131),
                                                       (55.555659, 37.367914), (55.561537, 37.376733), (55.556871, 37.388979), (55.549647, 37.407964),
                                                       (55.544780, 37.411890), (55.537898, 37.395509), (55.533777, 37.390762), (55.528676, 37.389415),
                                                       (55.526500, 37.396556), (55.519836, 37.398900), (55.512289, 37.416625), (55.514018, 37.407865),
                                                       (55.509629, 37.404378), (55.501631, 37.404964), (55.500599, 37.408334), (55.506652, 37.418207),
                                                       (55.505671, 37.432592), (55.501610, 37.433037), (55.494026, 37.454395), (55.488071, 37.444053),
                                                       (55.480069, 37.449854), (55.475060, 37.447540), (55.467921, 37.458351), (55.458399, 37.484631),
                                                       (55.455969, 37.492307), (55.450008, 37.498488), (55.446911, 37.485685), (55.436451, 37.510295),
                                                       (55.436251, 37.535316), (55.437563, 37.538430), (55.444975, 37.534776), (55.446793, 37.531259),
                                                       (55.453688, 37.531999), (55.453478, 37.536896), (55.456987, 37.548052), (55.464643, 37.547126),
                                                       (55.465392, 37.545843), (55.472673, 37.544785), (55.473365, 37.560821), (55.479359, 37.563090),
                                                       (55.487380, 37.565467), (55.487647, 37.568151), (55.486698, 37.578650), (55.483744, 37.578321),
                                                       (55.483704, 37.579851), (55.486792, 37.587807), (55.493555, 37.583429), (55.488770, 37.595152),
                                                       (55.488356, 37.606546), (55.492806, 37.607911), (55.502334, 37.605651), (55.509883, 37.608029),
                                                       (55.513169, 37.604874), (55.514759, 37.598048), (55.517938, 37.585665), (55.521771, 37.578462),
                                                       (55.517230, 37.577049), (55.516936, 37.566786), (55.520368, 37.566856), (55.528194, 37.559347),
                                                       (55.530050, 37.549319), (55.525697, 37.536583), (55.522185, 37.534300), (55.517991, 37.535100),
                                                       (55.522532, 37.520199), (55.525537, 37.519399), (55.527646, 37.500354), (55.530103, 37.486159),
                                                       (55.526324, 37.480109), (55.528514, 37.471988), (55.534029, 37.470270), (55.535564, 37.475707),
                                                       (55.537326, 37.475613), (55.540213, 37.487833), (55.539892, 37.494942), (55.542936, 37.507936),
                                                       (55.548545, 37.514673), (55.555204, 37.529385), (55.552122, 37.544946), (55.554938, 37.552196),
                                                       (55.561461, 37.554809), (55.563009, 37.550360), (55.567624, 37.553397), (55.572480, 37.537201),
                                                       (55.577387, 37.536965), (55.579614, 37.538637), (55.580348, 37.551137), (55.584962, 37.545534),
                                                       (55.586695, 37.547229), (55.593521, 37.518730), (55.596364, 37.510387), (55.607999, 37.495297),
                                                       (55.622836, 37.477708), (55.633068, 37.466086)],
              'Троицкий_административный_округ': [(55.544628, 37.411967), (55.553643, 37.392255), (55.561511, 37.376808), (55.553176, 37.367376),
                                                   (55.548600, 37.368282), (55.550421, 37.353659), (55.545867, 37.341797), (55.541196, 37.338048),
                                                   (55.538440, 37.326006), (55.539211, 37.298326), (55.530078, 37.299686), (55.521078, 37.274124),
                                                   (55.506541, 37.271199), (55.502613, 37.251181), (55.497002, 37.247103), (55.492858, 37.201899),
                                                   (55.483619, 37.184929), (55.480532, 37.152265), (55.482122, 37.136077), (55.473443, 37.139208),
                                                   (55.470286, 37.093041), (55.464647, 37.091805), (55.439071, 37.150117), (55.449131, 37.112738),
                                                   (55.441331, 37.089269), (55.453399, 37.080611), (55.454729, 37.058950), (55.462768, 37.060558),
                                                   (55.464219, 37.036806), (55.456799, 37.019962), (55.445317, 37.019158), (55.458439, 36.993986),
                                                   (55.460097, 36.974003), (55.493631, 36.977882), (55.496219, 36.936010), (55.515601, 36.942003),
                                                   (55.515424, 36.919902), (55.505964, 36.919340), (55.508175, 36.816063), (55.465492, 36.803764),
                                                   (55.441653, 36.803202), (55.431892, 36.817187), (55.422341, 36.824023), (55.418701, 36.841973),
                                                   (55.414014, 36.834262), (55.401066, 36.861858), (55.392119, 36.848216), (55.384940, 36.862700),
                                                   (55.380461, 36.856114), (55.357212, 36.904127), (55.353950, 36.915719), (55.341224, 36.935970),
                                                   (55.329330, 36.944818), (55.315490, 36.986455), (55.296948, 36.983758), (55.288902, 37.021610),
                                                   (55.282286, 37.022036), (55.268321, 36.983474), (55.264566, 36.952720), (55.264566, 36.952720),
                                                   (55.241512, 36.937673), (55.238457, 36.949833), (55.218391, 36.947610), (55.212034, 36.964170),
                                                   (55.205648, 36.965779), (55.180740, 36.982536), (55.180660, 36.996683), (55.183088, 37.018266),
                                                   (55.174390, 37.018308), (55.156639, 37.019420), (55.142627, 37.039273), (55.144397, 37.064647),
                                                   (55.149209, 37.111068), (55.161875, 37.157283), (55.165483, 37.171370), (55.165483, 37.171370),
                                                   (55.195135, 37.174212), (55.215865, 37.169558), (55.231569, 37.163585), (55.234936, 37.193572),
                                                   (55.244538, 37.200862), (55.254750, 37.211572), (55.247762, 37.234721), (55.239408, 37.245348),
                                                   (55.240608, 37.269980), (55.258796, 37.271504), (55.248609, 37.291110), (55.240161, 37.306804),
                                                   (55.224577, 37.300749), (55.220669, 37.316895), (55.228321, 37.316772), (55.228744, 37.333289),
                                                   (55.237972, 37.334401), (55.238702, 37.351989), (55.250680, 37.405372), (55.291317, 37.417935),
                                                   (55.300423, 37.404136), (55.308012, 37.394909), (55.316517, 37.382923), (55.309093, 37.410109),
                                                   (55.313980, 37.428685), (55.328870, 37.420818), (55.335754, 37.405289), (55.343103, 37.422713),
                                                   (55.351247, 37.419418), (55.352843, 37.442937), (55.364412, 37.427697), (55.374475, 37.438283),
                                                   (55.374569, 37.444502), (55.381441, 37.448209), (55.367683, 37.461967), (55.371272, 37.467610),
                                                   (55.390081, 37.470205), (55.404193, 37.459990), (55.406940, 37.467942), (55.412798, 37.467201),
                                                   (55.412962, 37.452867), (55.414720, 37.436143), (55.426809, 37.433343), (55.432392, 37.426436),
                                                   (55.434429, 37.386687), (55.444534, 37.399919), (55.446470, 37.388193), (55.445410, 37.373420),
                                                   (55.453603, 37.387870), (55.463180, 37.394146), (55.463037, 37.411142), (55.469454, 37.429215),
                                                   (55.487864, 37.444024), (55.494054, 37.454459), (55.502186, 37.439232), (55.500467, 37.437552),
                                                   (55.502225, 37.431829), (55.505743, 37.432538), (55.505689, 37.429274), (55.506750, 37.417918),
                                                   (55.510062, 37.405308), (55.512210, 37.416782), (55.518517, 37.407319), (55.519845, 37.398519),
                                                   (55.526596, 37.396461), (55.526530, 37.394536), (55.528651, 37.389497), (55.544628, 37.411967),
                                                   (55.539611, 37.399411)]
           }

In [24]:
def point_inside_polygon(x, y, poly, include_edges = True):
    '''
    Проверьте, находится ли точка (x,y) внутри многоугольника poly.

    poly - это многоугольник с N вершинами, определенный как 
    [(x1,y1),...,(xN,yN)] или [(x1,y1),...,(xN,yN),(x1,y1)]
    (функция отлично работает в обоих случаях)

    Геометрическая идея: точка находится внутри многоугольника, если горизонтальный луч
    справа от точки пересекает многоугольник четное число раз. 
    Отлично работает для невыпуклых многоугольников.
    '''
    n = len(poly)
    inside = False

    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if p1y == p2y:
            if y == p1y:
                if min(p1x, p2x) <= x <= max(p1x, p2x):
                    # point is on horisontal edge
                    inside = include_edges
                    break
                elif x < min(p1x, p2x):  # point is to the left from current edge
                    inside = not inside
        else:  # p1y!= p2y
            if min(p1y, p2y) <= y <= max(p1y, p2y):
                xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x

                if x == xinters:  # point is right on the edge
                    inside = include_edges
                    break

                if x < xinters:  # point is to the left from current edge
                    inside = not inside

        p1x, p1y = p2x, p2y

    return inside

In [25]:
from tqdm import tqdm

k = len(df_310['lat']) #Широта

districts_okrug = []

for i in tqdm(range(0, k, 1), 'Обработано'):
    for j, c, p in zip(range(1, len(polygons_o.keys()) + 1, 1), polygons_o.keys(), polygons_o.values()): #enumerate(polygons.items())
        poly = p
        x = df_310['lat'][i] 
        y = df_310['lon'][i]
        res = point_inside_polygon(x, y, poly, include_edges = True)

        if res == True:
            district = c
            districts_okrug.append(district)
            break
        
        if (j == len(polygons_o.keys()) and res == False):
            district = 999
            districts_okrug.append(district)

Обработано: 100%|██████████| 4490/4490 [00:05<00:00, 840.02it/s] 


In [26]:
 len(districts_okrug)

4490

In [27]:
df_310['poly_okrug'] = pd.DataFrame(districts_okrug, columns = ['poly_okrug'])
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,999
1,2,2682,GUAR_OBJECT_4302_130_1,50:08:0050435:778,231.5,2024-11-01,75211000.0,2023-11-29,NaN,"Россия, Московская область, городской округ Ис...",55.791025,37.013437,МО,999,999
2,3,2682,GUAR_OBJECT_4335_76_1,50:23:0000000:117210,99.3,2024-11-01,12787000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579169,38.216326,МО,999,999
3,4,2682,GUAR_OBJECT_4335_77_1,50:23:0110103:665,69.9,2024-11-01,9144000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579444,38.218060,МО,999,999
4,5,2682,GUAR_OBJECT_4338_83_1,50:23:0020216:277,96.7,2024-11-01,7450928.0,2023-03-17,NaN,"Россия, Московская область, Раменский городско...",55.584843,38.262401,МО,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4485,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ
4486,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ
4487,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ
4488,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ


In [28]:
df_310[(df_310['poly_sputnik'] == 999) & (df_310['poly_okrug'] == 999)]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug
1,2,2682,GUAR_OBJECT_4302_130_1,50:08:0050435:778,231.5,2024-11-01,75211000.0,2023-11-29,NaN,"Россия, Московская область, городской округ Ис...",55.791025,37.013437,МО,999,999
2,3,2682,GUAR_OBJECT_4335_76_1,50:23:0000000:117210,99.3,2024-11-01,12787000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579169,38.216326,МО,999,999
3,4,2682,GUAR_OBJECT_4335_77_1,50:23:0110103:665,69.9,2024-11-01,9144000.0,2023-03-17,NaN,"Россия, Московская область, Раменское, Северно...",55.579444,38.218060,МО,999,999
4,5,2682,GUAR_OBJECT_4338_83_1,50:23:0020216:277,96.7,2024-11-01,7450928.0,2023-03-17,NaN,"Россия, Московская область, Раменский городско...",55.584843,38.262401,МО,999,999
5,6,2682,GUAR_OBJECT_4338_84_1,50:23:0020216:276,96.3,2024-11-01,7420108.0,2023-03-17,NaN,"Россия, Московская область, Раменский городско...",55.585347,38.261655,МО,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,1933,2440,3630000014681,77:03:0002015:4321,47.0,2024-11-01,12300000.0,2024-06-19,"РОССИЙСКАЯ ФЕДЕРАЦИЯ,143909,Москва г,Гольяново...","Россия, Московская область, Балашиха, квартал ...",55.835554,37.831928,МСК,999,999
1979,1980,990,3610000000436,77:05:0005004:2686,31.5,2024-11-01,9638000.0,2024-02-19,"РОССИЯ, , Москва г, , , , Каширское ш , 26, 3, 61","Россия, Московская область, Ленинский городско...",55.566898,37.765219,МСК,999,999
4132,4133,1927,117309275641,77:07:0014002:3930,36.0,2024-11-01,13169698.0,2023-12-06,"640002, обл Курганская, г Курган, ул Коли Мяго...","Россия, Курган, улица Коли Мяготина, 78",55.437041,65.325865,МСК,999,999
4137,4138,1000,225087336225627097569057571626745713089,77:09:0005007:5128,107.0,2024-11-01,45888991.0,2023-09-01,"г. Москва, пр-д Березовая Реща, д. 12, кв. 284...","Россия, Московская область, Раменский городско...",55.355029,38.193491,МСК,999,999


Объекты с 999 и в спутнике и в округе удаляем - это не Москва и не город-спутник 

In [29]:
df_310 = df_310[~((df_310['poly_sputnik'] == 999) & (df_310['poly_okrug'] == 999))]
df_310.reset_index(drop=True , inplace=True)
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,999
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,999
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,999
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,999
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ


### Полигоны районов

In [31]:
polygons_r = {'НАО_Внуково': [(55.614846, 37.375300), (55.623878, 37.370606), (55.629702, 37.361083), (55.632420, 37.343480), (55.634146, 37.341468),
                           (55.635550, 37.338484), (55.641998, 37.330135), (55.652275, 37.336741), (55.654380, 37.342740), (55.656970, 37.341356),
                           (55.659575, 37.331902), (55.661599, 37.336967), (55.663068, 37.337051), (55.664271, 37.319528), (55.661484, 37.319038),
                           (55.661026, 37.324153), (55.651071, 37.322735), (55.651243, 37.311559), (55.646374, 37.311222), (55.649543, 37.282961),
                           (55.651452, 37.271447), (55.649295, 37.257266), (55.643834, 37.240891), (55.637341, 37.235286), (55.634037, 37.243221),
                           (55.628975, 37.235151), (55.627447, 37.237751), (55.624906, 37.235860), (55.623148, 37.225764), (55.621238, 37.229478),
                           (55.619365, 37.227655), (55.621524, 37.224819), (55.620493, 37.222388), (55.622769, 37.214692), (55.620132, 37.211417),
                           (55.622807, 37.207078), (55.618830, 37.185497), (55.612905, 37.186811), (55.612892, 37.182200), (55.607344, 37.177455),
                           (55.605327, 37.171908), (55.605503, 37.164201), (55.605680, 37.147360), (55.609791, 37.146068), (55.608454, 37.140187),
                           (55.605541, 37.132792), (55.600383, 37.133972), (55.598844, 37.128247), (55.600686, 37.119782), (55.605718, 37.116664),
                           (55.602151, 37.101258), (55.600121, 37.095756), (55.596211, 37.091768), (55.592515, 37.094865), (55.590484, 37.087335),
                           (55.575317, 37.099516), (55.564115, 37.116048), (55.549867, 37.124740), (55.561446, 37.166406), (55.569740, 37.194833),
                           (55.579941, 37.239386), (55.592190, 37.289816), (55.601948, 37.327226), (55.608991, 37.354346)], 
            'НАО_Филимонковский': [(55.614718, 37.376190), (55.608859, 37.393620), (55.601821, 37.407183), (55.598688, 37.401234), (55.589391, 37.400580),
                                (55.576722, 37.375357), (55.567116, 37.368159), (55.561520, 37.376666), (55.555485, 37.367802), (55.548470, 37.368337),
                                (55.550460, 37.353585), (55.539735, 37.333359), (55.538385, 37.313669), (55.543715, 37.311408), (55.539229, 37.298202),
                                (55.529917, 37.299630), (55.521009, 37.273931), (55.511018, 37.272623), (55.505278, 37.266691), (55.497893, 37.246963),
                                (55.492793, 37.201781), (55.483511, 37.184625), (55.479985, 37.148727), (55.487592, 37.127013), (55.496217, 37.123905),
                                (55.500168, 37.136207), (55.513212, 37.138133), (55.512790, 37.118695), (55.518876, 37.117163), (55.519174, 37.120052),
                                (55.530051, 37.115981), (55.532410, 37.120446), (55.538071, 37.122592), (55.538766, 37.113135), (55.547156, 37.114887),
                                (55.550454, 37.128139), (55.568424, 37.190315), (55.578249, 37.231848), (55.597656, 37.310786),
                                (55.608688, 37.352897)],
            'НАО_Коммунарка': [(55.614670, 37.376347), (55.611259, 37.381632), (55.607426, 37.396743), (55.601540, 37.406921), (55.598424, 37.401264),
                            (55.589267, 37.400583), (55.585668, 37.394623), (55.576528, 37.374270), (55.567102, 37.368254), (55.561492, 37.376855),
                            (55.557794, 37.388322), (55.555680, 37.393972), (55.553615, 37.392107), (55.551565, 37.400553), (55.549645, 37.407963),
                            (55.548658, 37.407160), (55.544692, 37.411969), (55.537482, 37.394808), (55.534177, 37.391044), (55.528719, 37.389360),
                            (55.526539, 37.396477), (55.520291, 37.397045), (55.512194, 37.416657), (55.511405, 37.416731), (55.514032, 37.407967),
                            (55.509723, 37.404473), (55.508488, 37.409833), (55.501700, 37.404977), (55.501202, 37.417912), (55.505667, 37.432420),
                            (55.502251, 37.432603), (55.500454, 37.437671), (55.498190, 37.448813), (55.498190, 37.448813), (55.502195, 37.458022),
                            (55.507292, 37.443176), (55.510046, 37.449863), (55.511406, 37.447465), (55.516410, 37.453427), (55.522719, 37.465795),
                            (55.529175, 37.471206), (55.534019, 37.470291), (55.535545, 37.475782), (55.537322, 37.475587), (55.540152, 37.487734),
                            (55.540055, 37.495494), (55.542999, 37.507885), (55.548561, 37.514543), (55.555225, 37.529387), (55.552950, 37.550761),
                            (55.561435, 37.554826), (55.567674, 37.553426), (55.572516, 37.536769), (55.579573, 37.538748), (55.578370, 37.546135),
                            (55.580394, 37.550964), (55.584893, 37.545487), (55.586824, 37.547367), (55.596510, 37.510537), (55.604878, 37.499001),
                            (55.623737, 37.477414), (55.638747, 37.459367), (55.634285, 37.449894), (55.632100, 37.442763), (55.625865, 37.418385),
                            (55.619450, 37.393651)],
              'НАО_Щербинка': [(55.509778, 37.608996), (55.512863, 37.604636), (55.514694, 37.598067), (55.512702, 37.591951), (55.521987, 37.578052),
                            (55.517240, 37.577088), (55.516938, 37.566769), (55.520318, 37.566946), (55.520577, 37.564715), (55.527122, 37.561368),
                            (55.529778, 37.552195), (55.528972, 37.542586), (55.524428, 37.535309), (55.518185, 37.535208), (55.522543, 37.520172),
                            (55.525492, 37.519463), (55.527117, 37.507191), (55.529979, 37.489443), (55.528411, 37.483206), (55.526470, 37.478819),
                            (55.529116, 37.471010), (55.527617, 37.466160), (55.523472, 37.466295), (55.522574, 37.465607), (55.518817, 37.458537),
                            (55.516510, 37.452768), (55.512393, 37.449358), (55.511327, 37.447348), (55.509816, 37.450480), (55.508451, 37.442948),
                            (55.506936, 37.442967), (55.505454, 37.450272), (55.501509, 37.456792), (55.498895, 37.454395), (55.500162, 37.449131),
                            (55.494049, 37.454521), (55.490838, 37.446024), (55.487921, 37.443969), (55.482109, 37.444781), (55.482246, 37.446429),
                            (55.475064, 37.447546), (55.472499, 37.452403), (55.469881, 37.461971), (55.467812, 37.458649), (55.465713, 37.458108),
                            (55.458131, 37.484584), (55.455848, 37.492140), (55.450014, 37.498215), (55.447476, 37.485200), (55.442752, 37.490996),
                            (55.441310, 37.499666), (55.437255, 37.504972), (55.437255, 37.504972), (55.436752, 37.538442), (55.444971, 37.534713),
                            (55.446774, 37.531522), (55.453668, 37.532444), (55.453450, 37.536981), (55.457022, 37.547978), (55.464625, 37.546997),
                            (55.464920, 37.545690), (55.472674, 37.545075), (55.473394, 37.560820), (55.487209, 37.565426), (55.486732, 37.578676),
                            (55.483735, 37.577682), (55.483672, 37.579934), (55.486832, 37.587310), (55.493609, 37.583437), (55.488744, 37.595186),
                            (55.488356, 37.606625), (55.489697, 37.611527), (55.502451, 37.608943)], 
            'ТАО_Троицк': [(55.544700, 37.411916), (55.549719, 37.407861), (55.553653, 37.392225), (55.561565, 37.376862), (55.555510, 37.367932),
                        (55.548481, 37.368634), (55.550559, 37.353583), (55.546403, 37.342509), (55.541318, 37.337167), (55.538421, 37.327496),
                        (55.538996, 37.313576), (55.543750, 37.311509), (55.543750, 37.311509), (55.539264, 37.298343), (55.530131, 37.299630),
                        (55.520979, 37.274099), (55.517793, 37.273553), (55.512469, 37.271782), (55.506118, 37.269989), (55.504304, 37.254196),
                        (55.497110, 37.246710), (55.481590, 37.255054), (55.479774, 37.259382), (55.458310, 37.263087), (55.443085, 37.270496),
                        (55.441999, 37.288588), (55.455917, 37.294164), (55.450709, 37.302626), (55.456670, 37.312101), (55.457246, 37.317950),
                        (55.453302, 37.337135), (55.447872, 37.371371), (55.452349, 37.386578), (55.463208, 37.394263), (55.467040, 37.400423),
                        (55.467040, 37.400423), (55.469477, 37.429122), (55.482079, 37.444797), (55.494035, 37.454390), (55.498241, 37.448736),
                        (55.500432, 37.437545), (55.501317, 37.417658), (55.501804, 37.404907), (55.512449, 37.416722), (55.519750, 37.398629),
                        (55.526563, 37.39625), (55.528686, 37.389349), (55.537554, 37.394769)], 
        'ТАО_Краснопахорский': [(55.455964, 37.294537), (55.448997, 37.290644), (55.442099, 37.288697), (55.443159, 37.270640), (55.458317, 37.263103),
                             (55.475267, 37.260279), (55.484887, 37.258747), (55.496942, 37.246968), (55.492780, 37.201453), (55.483492, 37.184846),
                             (55.479940, 37.149099), (55.482219, 37.136180), (55.473398, 37.139079), (55.470340, 37.092985), (55.464717, 37.091825),
                             (55.438927, 37.150223), (55.439962, 37.118335), (55.425006, 37.082994), (55.419937, 37.073389), (55.407015, 37.076214),
                             (55.398585, 37.053176), (55.387438, 37.059077), (55.377431, 37.103897), (55.367314, 37.142628), (55.367850, 37.185314),
                             (55.382947, 37.205116), (55.386390, 37.223545), (55.369051, 37.285809), (55.357987, 37.318998), (55.352195, 37.334284),
                             (55.358069, 37.339981), (55.356673, 37.350148), (55.358028, 37.353861), (55.350695, 37.377260), (55.342230, 37.396873),
                             (55.332408, 37.396657), (55.327593, 37.387283), (55.322571, 37.409776), (55.314852, 37.427281), (55.328820, 37.420698),
                             (55.336378, 37.405048), (55.343089, 37.422775), (55.351211, 37.419474), (55.352860, 37.442801), (55.364721, 37.427735),
                             (55.381108, 37.447319), (55.367684, 37.462220), (55.371367, 37.467495), (55.388234, 37.463889), (55.389992, 37.470477),
                             (55.404780, 37.461974), (55.406781, 37.468620), (55.406781, 37.468620), (55.415499, 37.450212), (55.413988, 37.436796),
                             (55.426753, 37.433448), (55.432349, 37.426429), (55.434446, 37.386757), (55.444436, 37.399961), (55.446346, 37.387911),
                             (55.447917, 37.371285), (55.455450, 37.328224), (55.457316, 37.318124), (55.456653, 37.312115), (55.450735, 37.302779),
                             (55.453586, 37.297411), (55.455433, 37.294216)], 
              'ТАО_Бекасово': [(55.449199, 37.112602), (55.441374, 37.089303), (55.453426, 37.079529), (55.454711, 37.058883), (55.462714, 37.060563),
                            (55.464129, 37.036788), (55.455948, 37.032064), (55.456851, 37.019972), (55.445220, 37.019173), (55.446203, 37.006341),
                            (55.450581, 37.005513), (55.458369, 36.994003), (55.459305, 36.989587), (55.455539, 36.973555), (55.493661, 36.977876),
                            (55.496281, 36.935170), (55.515455, 36.941963), (55.517366, 36.939067), (55.515538, 36.919803), (55.505897, 36.919389),
                            (55.508335, 36.815934), (55.465961, 36.815219), (55.465695, 36.803757), (55.439993, 36.803055), (55.421752, 36.825523),
                            (55.418772, 36.841422), (55.413998, 36.834487), (55.401054, 36.861805), (55.392133, 36.848281), (55.384817, 36.862746),
                            (55.380584, 36.856297), (55.354565, 36.909683), (55.354029, 36.916132), (55.340129, 36.938311), (55.326592, 36.950610),
                            (55.315611, 36.986348), (55.333741, 36.994767), (55.332133, 36.998574), (55.340965, 37.011561), (55.352423, 36.970225),
                            (55.367705, 36.976406), (55.373494, 36.981377), (55.377218, 36.998216), (55.390144, 37.012905), (55.395854, 37.006277),
                            (55.402761, 37.036058), (55.398912, 37.064406), (55.406991, 37.076229), (55.420595, 37.073363), (55.432133, 37.097457),
                            (55.441401, 37.112549), (55.445143, 37.107937)],
             'ТАО_Вороново': [(55.358003, 37.353877), (55.357862, 37.339904), (55.358791, 37.319830), (55.358428, 37.316709), (55.373511, 37.268560),
                           (55.386456, 37.223340), (55.382268, 37.212856), (55.383016, 37.202761), (55.367922, 37.183528), (55.369136, 37.161146),
                           (55.368272, 37.144045), (55.378144, 37.096897), (55.386410, 37.068551), (55.387344, 37.059222), (55.400097, 37.052342),
                           (55.402323, 37.027092), (55.396266, 37.006708), (55.387174, 37.010985), (55.376628, 36.996369), (55.369267, 36.976849),
                           (55.352897, 36.970579), (55.340895, 37.011275), (55.332987, 36.990826), (55.309434, 36.987652), (55.296967, 36.983889),
                           (55.288273, 37.026010), (55.282995, 37.026056), (55.276421, 36.986700), (55.268241, 36.983254), (55.264746, 36.953149),
                           (55.242090, 36.941043), (55.238515, 36.950201), (55.224833, 36.949340), (55.218405, 36.947436), (55.209325, 36.961158),
                           (55.203839, 36.966916), (55.180584, 36.982241), (55.180722, 37.006501), (55.175609, 37.018115), (55.156234, 37.020555),
                           (55.142731, 37.039319), (55.149186, 37.110997), (55.155778, 37.141957), (55.164729, 37.161955), (55.194913, 37.174497),
                           (55.215814, 37.169478), (55.231508, 37.163603), (55.234709, 37.196383), (55.249649, 37.205936), (55.253300, 37.219028),
                           (55.243348, 37.248098), (55.240482, 37.269917), (55.251710, 37.276721), (55.248629, 37.291152), (55.231735, 37.305797),
                           (55.224527, 37.300745), (55.220716, 37.316930), (55.238097, 37.334866), (55.238627, 37.352116), (55.250564, 37.405346),
                           (55.278761, 37.410718), (55.290469, 37.414910), (55.300709, 37.404293), (55.308942, 37.393160), (55.316566, 37.382748),
                           (55.309099, 37.410031), (55.314586, 37.427556), (55.322423, 37.409791), (55.328242, 37.389620), (55.332532, 37.396939),
                           (55.344224, 37.391682), (55.346327, 37.384305), (55.350615, 37.388050), (55.351457, 37.373412)],
              'САО_Молжаниновский': [(55.916185, 37.397041), (55.921158, 37.377736), (55.929950, 37.356645), (55.924246, 37.343406), (55.931074, 37.339444),
                       (55.933899, 37.331319), (55.935379, 37.335057), (55.934820, 37.336403), (55.935012, 37.336793), (55.935742, 37.335891),
                       (55.939045, 37.343416), (55.936728, 37.349570), (55.937897, 37.352106), (55.938854, 37.353738), (55.940385, 37.350159),
                       (55.940869, 37.351041), (55.946590, 37.341405), (55.946228, 37.340538), (55.948548, 37.338020), (55.948860, 37.338431),
                       (55.951205, 37.336126), (55.953706, 37.335015), (55.954912, 37.339623), (55.956101, 37.342321), (55.951941, 37.348857),
                       (55.952505, 37.350383), (55.951154, 37.353488), (55.951708, 37.354866), (55.952974, 37.355447), (55.953926, 37.356305),
                       (55.955885, 37.358913), (55.955970, 37.373551), (55.955562, 37.376309), (55.954255, 37.375055), (55.953606, 37.375323),
                       (55.950910, 37.381494), (55.951430, 37.383637), (55.952572, 37.382796), (55.953210, 37.384573), (55.952907, 37.386107),
                       (55.953341, 37.387610), (55.952984, 37.389372), (55.952388, 37.389246), (55.952019, 37.390053), (55.952244, 37.392842),
                       (55.951412, 37.393016), (55.950753, 37.391861), (55.949837, 37.391752), (55.949017, 37.392207), (55.948701, 37.393061),
                       (55.948677, 37.394201), (55.951387, 37.407052), (55.951518, 37.407646), (55.952859, 37.408631), (55.952925, 37.407661),
                       (55.953246, 37.407453), (55.953916, 37.408183), (55.954188, 37.409672), (55.954003, 37.410508), (55.953287, 37.410757),
                       (55.953148, 37.411302), (55.953435, 37.412094), (55.953924, 37.412168), (55.953704, 37.413317), (55.954179, 37.413121),
                       (55.954558, 37.413925), (55.954365, 37.416169), (55.953319, 37.416879), (55.953169, 37.419608), (55.952438, 37.420366),
                       (55.951594, 37.419820), (55.951326, 37.422358), (55.949262, 37.420769), (55.947248, 37.420893), (55.946592, 37.418919),
                       (55.945751, 37.415418), (55.945240, 37.410392), (55.939200, 37.407432), (55.925511, 37.410176), (55.924033, 37.408263),
                       (55.926238, 37.396824), (55.922833, 37.394367), (55.920553, 37.394132), (55.916797, 37.398429)], 
              'СВАО_Северный': [(55.907353, 37.537140), (55.941834, 37.519274), (55.944275, 37.537639), (55.943806, 37.541855), (55.943968, 37.543139),
                         (55.952848, 37.537686), (55.955870, 37.548325), (55.954739, 37.548992), (55.954614, 37.549371), (55.952543, 37.550576),
                         (55.951973, 37.550207), (55.951329, 37.550663), (55.951316, 37.551961), (55.952311, 37.551998), (55.952347, 37.551745),
                         (55.953481, 37.551846), (55.953516, 37.552566), (55.953887, 37.552489), (55.954018, 37.553281), (55.953984, 37.555101),
                         (55.951625, 37.555058), (55.957568, 37.573927), (55.955734, 37.578656), (55.954160, 37.580779), (55.949692, 37.579459),
                         (55.951274, 37.562649), (55.941421, 37.564320), (55.941457, 37.569750), (55.936693, 37.570139), (55.933598, 37.565776),
                         (55.932593, 37.560714), (55.928344, 37.561089), (55.920446, 37.559192), (55.919531, 37.558548),
                         (55.909348, 37.555843)], 
              'ВАО_Восточный': [(55.806540, 37.856438), (55.813583, 37.845325), (55.816704, 37.852718), (55.817922, 37.857303), (55.818401, 37.857283),
                          (55.818822, 37.858110), (55.819600, 37.857542), (55.819994, 37.859089), (55.819511, 37.859428), (55.819390, 37.860044),
                          (55.819967, 37.862288), (55.820038, 37.862259), (55.821529, 37.868708), (55.821946, 37.868556), (55.822620, 37.872149),
                          (55.820082, 37.873856), (55.820686, 37.878452), (55.819565, 37.880481), (55.820480, 37.884295), (55.816553, 37.884196),
                          (55.814250, 37.879808), (55.812752, 37.879548), (55.811540, 37.878181), (55.808696, 37.873744), (55.807715, 37.870197),
                          (55.806324, 37.871437), (55.804035, 37.866408), (55.804075, 37.865599), (55.808942, 37.861721), (55.808720, 37.860056),
                          (55.807950, 37.860526)],
             'ВАО_Новокосино': [(55.731014, 37.840422), (55.743982, 37.841861), (55.745240, 37.861403), (55.749213, 37.882678),
                                 (55.748232, 37.888252), (55.741488, 37.890250), (55.742530, 37.885815), (55.742655, 37.881789),
                                 (55.742463, 37.880790), (55.738364, 37.873005), (55.736272, 37.866533), (55.732425, 37.862138),
                                 (55.730431, 37.861256), (55.731039, 37.847530)], 
             'ВАО_Косино-Ухтомский': [(55.695283, 37.871671), (55.703331, 37.854682), (55.709834, 37.836049), (55.713518, 37.837939),
                                       (55.731017, 37.840423), (55.731154, 37.847471), (55.730543, 37.861185), (55.732535, 37.862063),
                                       (55.732378, 37.869604), (55.730120, 37.873055), (55.728127, 37.872078), (55.726610, 37.870210),
                                       (55.723458, 37.872322), (55.723876, 37.885954), (55.725546, 37.903467), (55.730630, 37.918332),
                                       (55.731271, 37.923590), (55.729922, 37.930583), (55.726866, 37.937527), (55.721004, 37.945421),
                                       (55.716181, 37.936443), (55.715919, 37.926529), (55.712205, 37.926096), (55.707949, 37.921781),
                                       (55.706961, 37.899397), (55.705237, 37.887005), (55.703259, 37.883883), (55.702475, 37.882330),
                                       (55.701242, 37.880629), (55.700849, 37.881414), (55.700110, 37.879262)],
             'ЮВАО_Жулебино': [(55.687088, 37.830548), (55.696601, 37.829664), (55.709844, 37.836049), (55.705347, 37.849350),
                               (55.701291, 37.859167), (55.701008, 37.858748), (55.701287, 37.858183), (55.700725, 37.857299),
                               (55.700163, 37.858401), (55.698954, 37.856609), (55.693713, 37.864243), (55.691173, 37.859912),
                               (55.688258, 37.864758), (55.687382, 37.864601), (55.677151, 37.858996), (55.675558, 37.856587),
                               (55.681320, 37.844402)], 
            'ВАО_Некрасовка': [(55.676187, 37.920612), (55.680492, 37.918964), (55.681489, 37.913792), (55.683044, 37.914301),
                                (55.683355, 37.913304), (55.686262, 37.917239), (55.688041, 37.918272), (55.688449, 37.921222),
                                (55.689357, 37.923449), (55.694030, 37.927587), (55.694319, 37.927541), (55.695148, 37.928374),
                                (55.696936, 37.935910), (55.696565, 37.939845), (55.697952, 37.939965), (55.698206, 37.926694),
                                (55.696893, 37.924534), (55.699531, 37.918889), (55.697100, 37.915330), (55.697813, 37.913275),
                                (55.702192, 37.913522), (55.707182, 37.914610), (55.707937, 37.921727), (55.707937, 37.921727),
                                (55.715916, 37.926521), (55.716163, 37.936253), (55.720899, 37.945479), (55.720157, 37.955582),
                                (55.717759, 37.958228), (55.717759, 37.958228), (55.714822, 37.960760), (55.714088, 37.959908),
                                (55.710567, 37.966744), (55.709291, 37.963901), (55.711641, 37.949790), (55.708682, 37.947340),
                                (55.705971, 37.958079), (55.704532, 37.959404), (55.696199, 37.950901), (55.694550, 37.955623),
                                (55.692751, 37.953936), (55.692768, 37.956347), (55.693875, 37.957525), (55.692783, 37.960175),
                                (55.689633, 37.958009), (55.694171, 37.943346), (55.687192, 37.936859), (55.686899, 37.928628),
                                (55.676065, 37.929937)], 
              'ЮЗАО_Северное Бутово': [(55.549999, 37.577104), (55.553679, 37.573592), (55.554208, 37.567775), (55.553751, 37.560001),
                                      (55.555829, 37.559518), (55.557410, 37.560485), (55.560485, 37.558646), (55.559322, 37.554237),
                                      (55.572415, 37.537233), (55.573195, 37.536855), (55.577310, 37.536889), (55.579580, 37.538690),
                                      (55.587015, 37.547773), (55.576214, 37.591374), (55.568712, 37.597371), (55.560847, 37.594121)], 
            'ЮЗАО_Южное Бутово': [(55.549999, 37.577104), (55.553679, 37.573592), (55.554208, 37.567775), (55.553751, 37.560001),
                                   (55.555829, 37.559518), (55.557410, 37.560485), (55.560485, 37.558646), (55.559322, 37.554237),
                                   (55.552686, 37.548876), (55.552071, 37.544933), (55.555275, 37.529266), (55.548548, 37.514568),
                                   (55.545016, 37.511143), (55.542946, 37.507935), (55.540022, 37.495911), (55.540188, 37.487794),
                                   (55.537327, 37.475549), (55.534674, 37.472796), (55.528162, 37.472867), (55.526404, 37.476523),
                                   (55.530008, 37.489500), (55.526423, 37.504530), (55.526980, 37.506836), (55.525521, 37.519378),
                                   (55.522516, 37.520167), (55.521159, 37.527565), (55.518140, 37.535316), (55.523193, 37.534900),
                                   (55.527215, 37.538720), (55.537681, 37.569633), (55.520463, 37.565392), (55.520341, 37.566914),
                                   (55.517116, 37.566219), (55.516713, 37.567710), (55.517204, 37.576814), (55.549750, 37.586649),
                                   (55.556343, 37.587823)], 
              'ЗАО_Солнцево': [(55.625796, 37.417189), (55.627246, 37.393760), (55.627623, 37.392356), (55.627974, 37.392721),
                              (55.628517, 37.387745), (55.631685, 37.388641), (55.632945, 37.384814), (55.632488, 37.379825),
                              (55.639923, 37.374198), (55.644697, 37.372964), (55.651114, 37.376665), (55.650340, 37.372058),
                              (55.654222, 37.366219), (55.659071, 37.370028), (55.659120, 37.376979), (55.659489, 37.378007),
                              (55.664463, 37.380133), (55.664542, 37.386298), (55.661678, 37.389402), (55.660898, 37.393976),
                              (55.662719, 37.408288), (55.667814, 37.428018), (55.656312, 37.438803), (55.638794, 37.459533),
                              (55.633473, 37.446602)],
            'ЗАО_Ново-Переделкино': [(55.625796, 37.417189), (55.627246, 37.393760), (55.627623, 37.392356), (55.627974, 37.392721),
                                      (55.628517, 37.387745), (55.631685, 37.388641), (55.632945, 37.384814), (55.632488, 37.379825),
                                      (55.639923, 37.374198), (55.644697, 37.372964), (55.651114, 37.376665), (55.650340, 37.372058),
                                      (55.654222, 37.366219), (55.659071, 37.370028), (55.664587, 37.369861), (55.666302, 37.363989),
                                      (55.666073, 37.347575), (55.657751, 37.340961), (55.654082, 37.343075), (55.651915, 37.336112),
                                      (55.641343, 37.330670), (55.635425, 37.337907), (55.630516, 37.356149), (55.628292, 37.363266),
                                      (55.624036, 37.369864), (55.614774, 37.376206)], 
           'ЗАО_ИЦ_Сколково': [(55.698751, 37.402361), (55.694798, 37.393540), (55.696726, 37.390180), (55.691738, 37.357933),
                                            (55.685604, 37.355185), (55.682187, 37.361187), (55.676354, 37.354320), (55.678534, 37.348962),
                                            (55.676264, 37.346124), (55.681795, 37.330143), (55.682778, 37.326057), (55.693540, 37.337691),
                                            (55.692553, 37.338658), (55.690038, 37.337933), (55.687330, 37.339009), (55.688193, 37.339876),
                                            (55.688709, 37.342430), (55.689212, 37.343816), (55.690556, 37.343855), (55.693901, 37.342440),
                                            (55.698609, 37.345063), (55.697663, 37.347305), (55.700133, 37.351723), (55.702261, 37.351483),
                                            (55.704780, 37.356112), (55.701764, 37.353118), (55.700451, 37.353326), (55.700614, 37.361408),
                                            (55.700942, 37.365574), (55.703757, 37.365931), (55.704319, 37.368047), (55.707114, 37.366007),
                                            (55.707415, 37.368181), (55.707374, 37.369073), (55.708485, 37.369311), (55.708493, 37.368396),
                                            (55.708054, 37.365101), (55.708953, 37.364239), (55.710832, 37.370321), (55.711737, 37.375414),
                                            (55.708767, 37.377575), (55.696870, 37.378245), (55.698686, 37.382081), (55.697044, 37.385486),
                                            (55.698435, 37.388289), (55.697060, 37.389706), (55.698496, 37.392311), (55.700863, 37.388906),
                                            (55.702150, 37.390368), (55.702524, 37.390298), (55.703271, 37.388115), (55.703552, 37.388441),
                                            (55.704032, 37.388080), (55.706094, 37.393345)],
        'ЗАО_Рублёво-Архангельское': [(55.788213, 37.370545), (55.768619, 37.344509), (55.771037, 37.309236), (55.775425, 37.310778),
                                                 (55.774622, 37.315681), (55.776743, 37.323069), (55.781129, 37.328259), (55.783388, 37.322280),
                                                 (55.786406, 37.312795), (55.779691, 37.313229), (55.780157, 37.311704), (55.784186, 37.311423),
                                                 (55.784441, 37.310515), (55.787788, 37.312905), (55.788286, 37.315260), (55.792881, 37.316328),
                                                 (55.793682, 37.314239), (55.797053, 37.323027), (55.796298, 37.324371), (55.796441, 37.324874),
                                                 (55.796834, 37.324672), (55.797127, 37.323949), (55.797785, 37.321478), (55.797842, 37.320903),
                                                 (55.797655, 37.320131), (55.797276, 37.319176), (55.796621, 37.319250), (55.796454, 37.318467),
                                                 (55.795788, 37.316700), (55.796245, 37.316496), (55.796382, 37.316217), (55.796344, 37.315781),
                                                 (55.796136, 37.315127), (55.796623, 37.315438), (55.796890, 37.315218), (55.796298, 37.313715),
                                                 (55.796080, 37.313575), (55.795976, 37.313644), (55.795668, 37.312604), (55.796869, 37.311793),
                                                 (55.796799, 37.311340), (55.796402, 37.311556), (55.795957, 37.309307), (55.797386, 37.308985),
                                                 (55.798158, 37.314178), (55.799263, 37.314628), (55.799384, 37.316188), (55.800529, 37.319429),
                                                 (55.800374, 37.320282), (55.800763, 37.321731), (55.799633, 37.333342), (55.806861, 37.347994),
                                                 (55.803346, 37.358323), (55.801262, 37.359539), (55.798765, 37.353737), (55.797158, 37.352155),
                                                 (55.796527, 37.348848), (55.792597, 37.357782), (55.789559, 37.368845),
                                                 (55.788752, 37.370035)], 
              'СЗАО_Митино': [(55.829754, 37.394383), (55.830678, 37.379889), (55.827986, 37.367453), (55.822193, 37.361537), (55.822193, 37.361537),
                               (55.822255, 37.349155), (55.824434, 37.351659), (55.824646, 37.350845), (55.826386, 37.349711), (55.826614, 37.349116),
                               (55.826082, 37.344787), (55.829074, 37.348534), (55.831033, 37.348181), (55.832210, 37.348713), (55.833089, 37.343485),
                               (55.834010, 37.342094), (55.836537, 37.345069), (55.839208, 37.340522), (55.840264, 37.340671), (55.840571, 37.347262),
                               (55.843881, 37.349253), (55.843881, 37.349253), (55.845486, 37.333146), (55.852702, 37.340641), (55.855277, 37.339505),
                               (55.860581, 37.342423), (55.861348, 37.345224), (55.862276, 37.346630), (55.863814, 37.349703), (55.864310, 37.357810),
                               (55.867411, 37.361564), (55.867345, 37.379307), (55.859477, 37.380047), (55.855173, 37.375006), (55.853446, 37.379774),
                               (55.855862, 37.386977), (55.854180, 37.394529), (55.846745, 37.391154), (55.836264, 37.395528),
                               (55.829758, 37.394386)], 
             'СЗАО_Куркино': [(55.870650, 37.411247), (55.875953, 37.403772), (55.877911, 37.383291), (55.883896, 37.371138), (55.892530, 37.377279),
                               (55.894856, 37.374675), (55.895033, 37.371222), (55.901358, 37.362690), (55.903120, 37.370536), (55.905525, 37.368465),
                               (55.905368, 37.367905), (55.906701, 37.366291), (55.906701, 37.366291), (55.911650, 37.387056), (55.912411, 37.390165),
                               (55.904266, 37.389814), (55.902327, 37.391612), (55.900730, 37.399253), (55.891700, 37.404359), (55.888014, 37.404103),
                               (55.886812, 37.404759), (55.886206, 37.405559), (55.884758, 37.409810), (55.880870, 37.409884), (55.878353, 37.417029),
                               (55.878311, 37.421203), (55.877932, 37.421207), (55.877590, 37.431595), (55.873491, 37.418155)],
              'ЮАО_Донской': [(55.721224, 37.600230), (55.719819, 37.5981590), (55.720125, 37.597457), (55.717688, 37.593449), (55.715142, 37.590096),
                               (55.713347, 37.587547), (55.713090, 37.588240), (55.711137, 37.585495), (55.711007, 37.583995), (55.711227, 37.582796),
                               (55.710931, 37.582113), (55.709576, 37.583822), (55.707016, 37.588256), (55.706586, 37.587674), (55.705081, 37.589107),
                               (55.702039, 37.591975), (55.698256, 37.594751), (55.694929, 37.597548), (55.690675, 37.603466), (55.689259, 37.609479),
                               (55.689766, 37.616890), (55.691824, 37.628838), (55.698929, 37.627669), (55.705318, 37.628309), (55.705874, 37.621507),
                               (55.705131, 37.616449), (55.704372, 37.611251), (55.707622, 37.608155), (55.708271, 37.607253), (55.709483, 37.606427),
                               (55.711360, 37.606195), (55.716746, 37.608273), (55.716680, 37.613617), (55.719887, 37.613639), (55.720119, 37.615503),
                               (55.721051, 37.614393), (55.721410, 37.610058), (55.719501, 37.609313), (55.720783, 37.602880)], 
            'ЮАО_Даниловский': [(55.716717, 37.608320), (55.711448, 37.606226), (55.709536, 37.606426), (55.708240, 37.607364), (55.707614, 37.608066),
                                 (55.704998, 37.610684), (55.704378, 37.611562), (55.705122, 37.616570), (55.705947, 37.622671), (55.705320, 37.628302),
                                 (55.694412, 37.628002), (55.686878, 37.631465), (55.686878, 37.631465), (55.684898, 37.634857), (55.684725, 37.644468),
                                 (55.685349, 37.654902), (55.686239, 37.665164), (55.698477, 37.666957), (55.705312, 37.667686), (55.709815, 37.666345),
                                 (55.717923, 37.663656), (55.722951, 37.663592), (55.725281, 37.663634), (55.726767, 37.663002), (55.726589, 37.656552),
                                 (55.728963, 37.654462), (55.728042, 37.651929), (55.729477, 37.645953), (55.727991, 37.644900), (55.725246, 37.644273),
                                 (55.722393, 37.643747), (55.722199, 37.641851), (55.723002, 37.639745), (55.723522, 37.636428), (55.722048, 37.635722),
                                 (55.722568, 37.632826), (55.721358, 37.627550), (55.720942, 37.622363), (55.720312, 37.617714), (55.719896, 37.613633),
                                 (55.716689, 37.613586)], 
              'ЮАО_Нагорный': [(55.690014, 37.605517), (55.689415, 37.604975), (55.688180, 37.605350), (55.686460, 37.604918), (55.684810, 37.605986),
                                (55.684030, 37.608167), (55.684047, 37.610284), (55.684773, 37.614142), (55.685058, 37.616865), (55.684536, 37.616902),
                                (55.684097, 37.613747), (55.683061, 37.614102), (55.681100, 37.614903), (55.678406, 37.611393), (55.677269, 37.611310),
                                (55.676308, 37.608906), (55.673529, 37.605976), (55.672510, 37.604265), (55.671507, 37.605004), (55.668998, 37.605835),
                                (55.666089, 37.611617), (55.665451, 37.610644), (55.665444, 37.607513), (55.665040, 37.607501), (55.664984, 37.605492),
                                (55.664350, 37.605320), (55.663484, 37.613505), (55.660565, 37.610044), (55.656566, 37.605260), (55.651297, 37.605419),
                                (55.646278, 37.605539), (55.641384, 37.605590), (55.640685, 37.618091), (55.640716, 37.619716), (55.642297, 37.619709),
                                (55.642487, 37.623258), (55.643142, 37.623252), (55.643239, 37.624090), (55.648132, 37.623008), (55.653001, 37.622504),
                                (55.663012, 37.622504), (55.666526, 37.622686), (55.671758, 37.626645), (55.675625, 37.626108), (55.681925, 37.623781),
                                (55.685621, 37.622992), (55.690717, 37.621809), (55.689361, 37.614957), (55.689361, 37.608400)], 
              'ЮАО_Нагатино-Садовники': [(55.690713, 37.621856), (55.681929, 37.623751), (55.673546, 37.626850), (55.670407, 37.625773),
                                          (55.666653, 37.622716), (55.656078, 37.622467), (55.654151, 37.627087), (55.652876, 37.629747),
                                          (55.652571, 37.631161), (55.652647, 37.633753), (55.658313, 37.641767), (55.654254, 37.650662),
                                          (55.650257, 37.660090), (55.650831, 37.660440), (55.653788, 37.654885), (55.655498, 37.653785),
                                          (55.659648, 37.654848), (55.667930, 37.661242), (55.677128, 37.663643), (55.686240, 37.665091),
                                          (55.684197, 37.639278), (55.685687, 37.632947), (55.691668, 37.628796), (55.691328, 37.625297)],
              'ЮАО_Нагатинский_Затон': [(55.700860, 37.667354), (55.686056, 37.665113), (55.672677, 37.662734), (55.667923, 37.661483),
                                         (55.659314, 37.654569), (55.652845, 37.654772), (55.650619, 37.660902), (55.653328, 37.664353),
                                         (55.649025, 37.675202), (55.645341, 37.680054), (55.647613, 37.683331), (55.653299, 37.672263),
                                         (55.666866, 37.673911), (55.669830, 37.679022), (55.679485, 37.708720), (55.689427, 37.712384),
                                         (55.694466, 37.700897), (55.698538, 37.675315)], 
              'ЮАО_Москворечье-Сабурово': [(55.656813, 37.622594), (55.649621, 37.622541), (55.639508, 37.625404), (55.640081, 37.635013),
                                            (55.641263, 37.641863), (55.638562, 37.641233), (55.637540, 37.655085), (55.638368, 37.654973),
                                            (55.637199, 37.673736), (55.636177, 37.688183), (55.628284, 37.690084), (55.631636, 37.727049),
                                            (55.636526, 37.728300), (55.640928, 37.728231), (55.639605, 37.712735), (55.643790, 37.692864),
                                            (55.647499, 37.683486), (55.645414, 37.680780), (55.648918, 37.674976), (55.649694, 37.673799),
                                            (55.653336, 37.664586), (55.650479, 37.660353), (55.655162, 37.648535), (55.658309, 37.641775),
                                            (55.652780, 37.633786), (55.652549, 37.631123), (55.654498, 37.626378)], 
              'ЮАО_Братеево': [(55.640897, 37.728380), (55.634037, 37.728165), (55.631531, 37.730140), (55.630685, 37.751269), (55.626053, 37.754846),
                                (55.621218, 37.764434), (55.619046, 37.771558), (55.618764, 37.780238), (55.617714, 37.782489), (55.623354, 37.792851),
                                (55.633140, 37.785051), (55.641681, 37.784020), (55.648408, 37.784914), (55.648485, 37.778107), (55.640747, 37.749037),
                                (55.640024, 37.738985)], 
             'ЮАО_Зябликово': [(55.630927, 37.751186), (55.630670, 37.729140), (55.626688, 37.727424), (55.626532, 37.728456), (55.624033, 37.729542),
                                (55.623222, 37.729229), (55.623001, 37.727874), (55.619619, 37.727806), (55.615646, 37.728921), (55.612833, 37.730500),
                                (55.610267, 37.731855), (55.610886, 37.736672), (55.610632, 37.744712), (55.610676, 37.746904), (55.611425, 37.753127),
                                (55.613477, 37.762021), (55.616175, 37.772987), (55.619067, 37.775491), (55.619223, 37.770436), (55.622253, 37.761949),
                                (55.625636, 37.755451), (55.628213, 37.752716)],
             'ЮАО_Орехово-Борисово_Южное': [(55.612912, 37.730429), (55.608623, 37.703315), (55.604969, 37.693360), (55.599198, 37.702506),
                                             (55.593977, 37.718963), (55.589670, 37.724192), (55.598714, 37.745926), (55.608730, 37.766013),
                                             (55.618045, 37.782127), (55.619173, 37.774765), (55.616348, 37.773116), (55.613625, 37.762164),
                                             (55.610951, 37.746725), (55.610641, 37.746692), (55.610784, 37.739992), (55.610747, 37.735042),
                                             (55.610239, 37.731774)], 
              'ЮАО_Орехово-Борисово_Северное': [(55.631373, 37.729503), (55.630934, 37.708224), (55.628566, 37.689933), (55.636611, 37.686809),
                                                 (55.630676, 37.684813), (55.620085, 37.671781), (55.619415, 37.672703), (55.621926, 37.675787),
                                                 (55.621735, 37.679455), (55.617912, 37.677894), (55.612464, 37.678842), (55.609887, 37.680787),
                                                 (55.605390, 37.692162), (55.604977, 37.693475), (55.606529, 37.696924), (55.606242, 37.698805),
                                                 (55.606817, 37.700611), (55.607627, 37.699974), (55.608851, 37.704067), (55.611459, 37.720474),
                                                 (55.613034, 37.730395), (55.617900, 37.728092), (55.622456, 37.727858), (55.623163, 37.729111),
                                                 (55.624632, 37.729130), (55.625994, 37.728679), (55.626467, 37.727490), (55.628752, 37.728235),
                                                 (55.629714, 37.729540)], 
              'ЮАО_Бирюлёво_Восточное': [(55.621721, 37.679164), (55.621893, 37.675466), (55.619514, 37.672427), (55.620154, 37.671099),
                                          (55.615180, 37.659783), (55.609853, 37.642151), (55.601955, 37.649107), (55.586558, 37.658250),
                                          (55.578344, 37.660810), (55.572147, 37.664069), (55.572345, 37.674950), (55.575839, 37.687856),
                                          (55.584501, 37.711492), (55.589643, 37.724235), (55.594662, 37.717374), (55.599032, 37.703047),
                                          (55.604861, 37.693432), (55.610085, 37.680549), (55.615669, 37.677685), (55.619276, 37.678568)], 
              'ЮАО_Бирюлёво_Западное': [(55.609847, 37.641703), (55.600544, 37.629772), (55.595585, 37.621645), (55.590550, 37.615961),
                                         (55.583856, 37.614520), (55.574914, 37.613386), (55.573364, 37.641990), (55.572119, 37.664164),
                                         (55.578761, 37.660412), (55.590062, 37.656522), (55.601925, 37.649001), (55.606495, 37.644777)], 
              'ЮАО_Чертаново_Южное': [(55.610163, 37.609115), (55.603407, 37.605783), (55.604367, 37.600263), (55.604170, 37.598916),
                                       (55.605134, 37.589946), (55.606257, 37.590163), (55.604790, 37.582241), (55.599383, 37.576794),
                                       (55.597959, 37.575841), (55.589199, 37.575767), (55.580945, 37.572723), (55.576701, 37.591031),
                                       (55.575911, 37.598345), (55.575027, 37.613453), (55.588763, 37.615360), (55.595109, 37.620310),
                                       (55.599402, 37.628951), (55.604168, 37.633356), (55.607368, 37.624090), (55.609005, 37.615986)],
              'ЮАО_Чертаново_Центральное': [(55.622239, 37.614824), (55.620709, 37.614052), (55.621530, 37.608378), (55.623516, 37.595426),
                                             (55.624196, 37.591178), (55.623589, 37.588217), (55.625216, 37.585770), (55.625386, 37.582852),
                                             (55.620004, 37.574612), (55.618387, 37.580955), (55.617051, 37.579324), (55.616735, 37.580698),
                                             (55.615181, 37.579539), (55.613301, 37.574707), (55.608759, 37.574750), (55.608394, 37.569875),
                                             (55.606242, 37.570218), (55.606048, 37.576398), (55.604298, 37.577556), (55.604687, 37.582148),
                                             (55.606325, 37.585940), (55.606343, 37.590233), (55.604981, 37.589973), (55.604822, 37.595274),
                                             (55.604086, 37.599990), (55.604159, 37.601324), (55.603386, 37.605844), (55.610202, 37.609129),
                                             (55.608090, 37.621262), (55.605638, 37.629213), (55.604066, 37.633749), (55.610169, 37.641197),
                                             (55.613149, 37.636700), (55.618318, 37.632863), (55.618703, 37.630618), (55.619349, 37.625373),
                                             (55.620997, 37.624542)],
              'ЮАО_Чертаново_Северное': [(55.622194, 37.614933), (55.620708, 37.614098), (55.621529, 37.608306), (55.623523, 37.595450),
                                          (55.624151, 37.591059), (55.623554, 37.588259), (55.625236, 37.586312), (55.625563, 37.583063),
                                          (55.631024, 37.585723), (55.632735, 37.577255), (55.634408, 37.577466), (55.638512, 37.581578),
                                          (55.642760, 37.579215), (55.644840, 37.579803), (55.645329, 37.580370), (55.642668, 37.589107),
                                          (55.642165, 37.592664), (55.641371, 37.605637), (55.640679, 37.618369), (55.640709, 37.619713),
                                          (55.642374, 37.619702), (55.642550, 37.623332), (55.643134, 37.623291), (55.643219, 37.624024),
                                          (55.636923, 37.625828), (55.630241, 37.627535), (55.625297, 37.628933), (55.618389, 37.632818),
                                          (55.618392, 37.630778), (55.618675, 37.630545), (55.619313, 37.625388), (55.621065, 37.624726),
                                          (55.621064, 37.623560)], 
              'ЮАО_Царицыно': [(55.636356, 37.686813), (55.637492, 37.669652), (55.638503, 37.654823), (55.637587, 37.654971), (55.638594, 37.641259),
                                (55.641272, 37.641766), (55.640034, 37.634719), (55.640049, 37.625187), (55.633139, 37.626789), (55.626102, 37.628611),
                                (55.619854, 37.632009), (55.613196, 37.636562), (55.609951, 37.641631), (55.615131, 37.659583), (55.617682, 37.668030),
                                (55.623469, 37.675974), (55.630360, 37.684550), (55.633141, 37.685805)],
              'ЮЗАО_Академический': [(55.706461, 37.587541), (55.701650, 37.579081), (55.695765, 37.568739), (55.691686, 37.561386),
                                      (55.684081, 37.552293), (55.677974, 37.562885), (55.672720, 37.578065), (55.674933, 37.580311),
                                      (55.676902, 37.579542), (55.679097, 37.586170), (55.682289, 37.589374), (55.684444, 37.591923),
                                      (55.686034, 37.593418), (55.690151, 37.597993), (55.690889, 37.599727), (55.691729, 37.601945),
                                      (55.696569, 37.595970), (55.701042, 37.592679), (55.704454, 37.589600)], 
              'ЮЗАО_Гагаринский': [(55.714218, 37.577616), (55.711421, 37.569255), (55.710221, 37.559092), (55.705450, 37.552634),
                                    (55.698983, 37.543800), (55.693284, 37.536155), (55.689923, 37.530036), (55.684530, 37.539635),
                                    (55.680821, 37.547483), (55.684083, 37.552203), (55.688436, 37.557592), (55.691764, 37.561708),
                                    (55.695398, 37.568074), (55.700543, 37.577179), (55.706975, 37.588335), (55.711098, 37.581500)],
              'ЮЗАО_Котловка': [(55.691700, 37.601904), (55.690277, 37.598223), (55.686011, 37.593453), (55.684043, 37.591430), (55.682206, 37.589325),
                                 (55.680062, 37.586729), (55.679102, 37.586072), (55.676891, 37.579548), (55.674894, 37.580296), (55.672722, 37.577993),
                                 (55.669919, 37.585373), (55.665998, 37.581197), (55.665244, 37.587673), (55.665221, 37.595147), (55.666288, 37.596779),
                                 (55.664860, 37.601076), (55.664311, 37.605323), (55.664931, 37.605701), (55.665023, 37.607479), (55.665446, 37.607469),
                                 (55.665451, 37.610715), (55.666114, 37.611544), (55.669119, 37.605795), (55.672621, 37.604759), (55.673536, 37.605960),
                                 (55.675471, 37.608456), (55.676665, 37.609444), (55.677394, 37.611449), (55.678414, 37.611377), (55.681105, 37.614852),
                                 (55.683046, 37.614184), (55.684126, 37.613807), (55.684533, 37.616951), (55.685023, 37.616687), (55.684508, 37.612828),
                                 (55.683972, 37.609273), (55.684853, 37.606024), (55.686141, 37.604977), (55.689245, 37.605068),
                                 (55.690115, 37.605615)], 
              'ЮЗАО_Зюзино': [(55.663524, 37.613508), (55.664811, 37.601665), (55.666315, 37.596708), (55.665543, 37.594008), (55.665953, 37.581137),
                               (55.661046, 37.575842), (55.654416, 37.568713), (55.649480, 37.563876), (55.647567, 37.572673), (55.644973, 37.581626),
                               (55.642405, 37.590318), (55.642048, 37.594417), (55.641388, 37.605559), (55.646552, 37.605522), (55.653240, 37.605399),
                               (55.656563, 37.605328), (55.658971, 37.608011), (55.661728, 37.611430)],
            'ЮЗАО_Черемушки': [(55.669913, 37.585389), (55.671761, 37.580795), (55.677898, 37.562905), (55.682095, 37.555589), (55.678251, 37.551366),
                                (55.673269, 37.546001), (55.670645, 37.554920), (55.665458, 37.549458), (55.660947, 37.544643), (55.655794, 37.539102),
                                (55.653570, 37.544120), (55.651515, 37.550152), (55.650737, 37.555506), (55.649475, 37.563829), (55.653693, 37.567921),
                                (55.659980, 37.574721), (55.669913, 37.585389)], 
              'ЮЗАО_Ломоносовский': [(55.684029, 37.552164), (55.680811, 37.547574), (55.684468, 37.539711), (55.687083, 37.534987),
                                      (55.690026, 37.530001), (55.690026, 37.530001), (55.683229, 37.517985), (55.679672, 37.511631),
                                      (55.673719, 37.523965), (55.671141, 37.529569), (55.668697, 37.531519), (55.668194, 37.533329),
                                      (55.668506, 37.540614), (55.671396, 37.541800), (55.673242, 37.543608), (55.674223, 37.543082),
                                      (55.673238, 37.546049), (55.678260, 37.551399), (55.682099, 37.555558)], 
              'ЮЗАО_Обручевский': [(55.673693, 37.524134), (55.663417, 37.508942), (55.658964, 37.502304), (55.655591, 37.495627), (55.649461, 37.482457),
                                    (55.646594, 37.487025), (55.642225, 37.498535), (55.649925, 37.508808), (55.648166, 37.513272), (55.654611, 37.521836),
                                    (55.655163, 37.520094), (55.655911, 37.521045), (55.655378, 37.522860), (55.657315, 37.525336), (55.656498, 37.533582),
                                    (55.655787, 37.539088), (55.663408, 37.547328), (55.670652, 37.555089), (55.671465, 37.552107), (55.673286, 37.546015),
                                    (55.674065, 37.543282), (55.673079, 37.543493), (55.671231, 37.541719), (55.668558, 37.540727), (55.668222, 37.533057),
                                    (55.668930, 37.531167), (55.671237, 37.529282), (55.672394, 37.526845)], 
              'ЮЗАО_Коньково': [(55.657275, 37.525361), (55.655337, 37.522864), (55.655904, 37.520907), (55.655217, 37.519995), (55.654649, 37.521952),
                                 (55.650876, 37.516914), (55.648095, 37.513253), (55.649930, 37.508808), (55.642191, 37.498556), (55.635312, 37.516212),
                                 (55.634315, 37.520490), (55.630461, 37.517759), (55.628659, 37.516017), (55.624173, 37.524369), (55.626433, 37.527130),
                                 (55.628695, 37.524200), (55.631817, 37.528247), (55.631055, 37.540055), (55.638240, 37.544710), (55.640864, 37.549092),
                                 (55.643856, 37.558441), (55.645611, 37.561026), (55.649518, 37.563939), (55.651048, 37.553002), (55.652712, 37.546042),
                                 (55.655725, 37.539111), (55.657062, 37.528137)],
               'ЮЗАО_Ясенево': [(55.628577, 37.516050), (55.619112, 37.504928), (55.614869, 37.499144), (55.611158, 37.491841), (55.599637, 37.505397),
                                 (55.595208, 37.513997), (55.591017, 37.530953), (55.581053, 37.572782), (55.592945, 37.577934), (55.597833, 37.580548),
                                 (55.597993, 37.576107), (55.603865, 37.582225), (55.606112, 37.571775), (55.607094, 37.569571), (55.608438, 37.569822),
                                 (55.613295, 37.574557), (55.618977, 37.578643), (55.619990, 37.575014), (55.622143, 37.576853), (55.625422, 37.583205),
                                 (55.631272, 37.585845), (55.632893, 37.576411), (55.634478, 37.577281), (55.638782, 37.581483), (55.642818, 37.579251),
                                 (55.645337, 37.580180), (55.647549, 37.572547), (55.649468, 37.563943), (55.645029, 37.560431), (55.643491, 37.557668),
                                 (55.640624, 37.548666), (55.638156, 37.544767), (55.631001, 37.539891), (55.631638, 37.531715), (55.631462, 37.527660),
                                 (55.630135, 37.524494), (55.628438, 37.524024), (55.626243, 37.526455), (55.625323, 37.525670), (55.624240, 37.524469),
                                 (55.626147, 37.519841)], 
        'ЮЗАО_Теплый_Стан': [(55.649603, 37.482505), (55.638947, 37.459468), (55.625884, 37.474739), (55.611126, 37.491800), (55.617101, 37.502493),
                              (55.626428, 37.513559), (55.630021, 37.517596), (55.634307, 37.520470), (55.635307, 37.516077), (55.641122, 37.501250),
                              (55.646491, 37.487194)],
        'ЮВАО_Лефортово': [(55.769504, 37.692095), (55.767687, 37.688894), (55.763911, 37.685782), (55.762232, 37.685842), (55.760101, 37.683935),
                            (55.759120, 37.681025), (55.759158, 37.676475), (55.759346, 37.673062), (55.758399, 37.671035), (55.756569, 37.670935),
                            (55.753883, 37.672969), (55.752600, 37.672768), (55.750728, 37.670500), (55.749822, 37.674146), (55.748332, 37.682524),
                            (55.744437, 37.691055), (55.742098, 37.695542), (55.742055, 37.697884), (55.743042, 37.698036), (55.743185, 37.700673),
                            (55.744157, 37.700851), (55.743800, 37.706510), (55.743276, 37.711358), (55.739615, 37.713082), (55.737558, 37.714532),
                            (55.736013, 37.716383), (55.735011, 37.718234), (55.734210, 37.720668), (55.733458, 37.728923), (55.736902, 37.731170),
                            (55.740496, 37.733979), (55.743774, 37.735546), (55.746062, 37.736611), (55.747733, 37.730140), (55.749451, 37.723832),
                            (55.751411, 37.720181), (55.753984, 37.717950), (55.756750, 37.717671), (55.759029, 37.718670), (55.762388, 37.720622),
                            (55.766051, 37.720810), (55.770430, 37.717184), (55.775321, 37.713705), (55.771821, 37.706638), (55.770746, 37.704589),
                            (55.770020, 37.702514), (55.766779, 37.697226), (55.768155, 37.694690)],
        'ЮВАО_Нижегородский': [(55.746024, 37.736635), (55.743038, 37.735316), (55.739210, 37.733041), (55.733484, 37.728926), (55.734211, 37.720504),
                                (55.735733, 37.716993), (55.738675, 37.713806), (55.743250, 37.711489), (55.743741, 37.706468), (55.744111, 37.700880),
                                (55.743051, 37.700626), (55.743195, 37.697982), (55.742009, 37.697504), (55.742098, 37.695402), (55.739583, 37.697854),
                                (55.737462, 37.699065), (55.736357, 37.699670), (55.736422, 37.698837), (55.735485, 37.698378), (55.735485, 37.698378),
                                (55.729361, 37.693701), (55.729329, 37.692925), (55.729566, 37.692104), (55.731256, 37.689054), (55.732561, 37.686666),
                                (55.732967, 37.685363), (55.730166, 37.685765), (55.730057, 37.683768), (55.729558, 37.683847), (55.729218, 37.683262),
                                (55.729094, 37.681322), (55.728433, 37.681377), (55.728379, 37.680484), (55.727692, 37.680652), (55.725369, 37.687392),
                                (55.721424, 37.698871), (55.719851, 37.702109), (55.723490, 37.714722), (55.721759, 37.717079), (55.723643, 37.727746),
                                (55.725553, 37.738372), (55.727678, 37.750202), (55.728809, 37.756218), (55.729856, 37.758925), (55.731512, 37.761015),
                                (55.733195, 37.761893), (55.734413, 37.767839), (55.735945, 37.764941), (55.737112, 37.761446), (55.738301, 37.752804),
                                (55.739112, 37.749565), (55.740999, 37.745865), (55.743996, 37.742061), (55.745390, 37.738993)],
        'ЮВАО_Рязанский': [(55.734433, 37.767891), (55.733184, 37.761912), (55.731006, 37.760611), (55.729399, 37.757954), (55.728756, 37.755961),
                            (55.727242, 37.747698), (55.725189, 37.736594), (55.724707, 37.738682), (55.723918, 37.739751), (55.722304, 37.741036),
                            (55.721554, 37.742491), (55.720590, 37.742554), (55.715676, 37.745591), (55.710315, 37.748982), (55.713290, 37.760706),
                            (55.716826, 37.774467), (55.712377, 37.777983), (55.709255, 37.780655), (55.708642, 37.784892), (55.707378, 37.794488),
                            (55.712657, 37.796074), (55.716649, 37.797272), (55.720195, 37.798320), (55.720876, 37.800324), (55.721498, 37.801038),
                            (55.725848, 37.788249), (55.729712, 37.776752), (55.729712, 37.776752)],
        'ЮВАО_Южнопортовый': [(55.731781, 37.666076), (55.730721, 37.660649), (55.727631, 37.662675), (55.725466, 37.663468), (55.723000, 37.663697),
                               (55.719059, 37.663534), (55.717637, 37.663767), (55.712870, 37.665307), (55.708362, 37.666745), (55.705550, 37.667734),
                               (55.703120, 37.667529), (55.700708, 37.667272), (55.700055, 37.672682), (55.698460, 37.675505), (55.697474, 37.683995),
                               (55.698903, 37.685330), (55.700205, 37.686978), (55.704612, 37.690170), (55.705421, 37.686740), (55.706317, 37.686133),
                               (55.711368, 37.681666), (55.714781, 37.679099), (55.717927, 37.690058), (55.720898, 37.699986), (55.721570, 37.698445),
                               (55.724531, 37.689815), (55.728255, 37.678842), (55.727349, 37.677853), (55.725795, 37.670589), (55.730402, 37.667756),
                               (55.730923, 37.667281)], 
            'ЮВАО_Печатники': [(55.720731, 37.699964), (55.714825, 37.679125), (55.711315, 37.681704), (55.705561, 37.686693), (55.704605, 37.690171),
                                (55.700258, 37.687025), (55.698983, 37.685448), (55.697194, 37.683613), (55.693731, 37.704973), (55.690982, 37.709921),
                                (55.687038, 37.712791), (55.680935, 37.711543), (55.679225, 37.707893), (55.677293, 37.704318), (55.675421, 37.698378),
                                (55.671579, 37.687704), (55.669172, 37.677828), (55.667146, 37.674352), (55.663246, 37.672559), (55.654654, 37.671397),
                                (55.652289, 37.673570), (55.647657, 37.683152), (55.643863, 37.692446), (55.646191, 37.694710), (55.649381, 37.700454),
                                (55.658566, 37.713123), (55.665065, 37.721903), (55.669311, 37.725473), (55.678631, 37.731228), (55.687524, 37.735292),
                                (55.693107, 37.733507), (55.694641, 37.731477), (55.702100, 37.729949), (55.709629, 37.729239), (55.715876, 37.710390),
                                (55.718722, 37.704441)], 
        'ЮВАО_Текстильщики': [(55.723508, 37.714918), (55.719821, 37.702123), (55.716324, 37.709295), (55.712515, 37.720696), (55.709644, 37.729181),
                               (55.699115, 37.730151), (55.694527, 37.731420), (55.692970, 37.733717), (55.691875, 37.734023), (55.691875, 37.735860),
                               (55.691673, 37.738718), (55.690059, 37.741678), (55.690370, 37.746026), (55.688986, 37.754243), (55.691428, 37.756105),
                               (55.694188, 37.754267), (55.701912, 37.749212), (55.707918, 37.745189), (55.708341, 37.747773), (55.709911, 37.747500),
                               (55.710299, 37.749034), (55.713119, 37.747168), (55.716900, 37.744861), (55.720765, 37.742516), (55.721550, 37.742546),
                               (55.722449, 37.740851), (55.724191, 37.739582), (55.724869, 37.738410), (55.725167, 37.736402), (55.723651, 37.727802),
                               (55.721773, 37.717113)],
            'ЮВАО_Кузьминки': [(55.716836, 37.774466), (55.713267, 37.760603), (55.709881, 37.747491), (55.708511, 37.747773), (55.707959, 37.745209),
                                (55.702088, 37.749089), (55.697658, 37.752064), (55.692447, 37.755514), (55.691308, 37.756127), (55.690729, 37.755698),
                                (55.689411, 37.758995), (55.687433, 37.763432), (55.686585, 37.765944), (55.685456, 37.772430), (55.684156, 37.778606),
                                (55.683138, 37.782370), (55.683422, 37.785426), (55.681281, 37.789951), (55.683025, 37.792862), (55.680587, 37.802285),
                                (55.687979, 37.804998), (55.689006, 37.798655), (55.687149, 37.790842), (55.689115, 37.787446), (55.690409, 37.787735),
                                (55.690403, 37.789403), (55.696802, 37.791308), (55.701935, 37.792822), (55.707376, 37.794482), (55.708438, 37.786520),
                                (55.709172, 37.780961), (55.709367, 37.780390), (55.712353, 37.777983), (55.715348, 37.775589)], 
            'ЮВАО_Люблино': [(55.691818, 37.734067), (55.688762, 37.735460), (55.687552, 37.735203), (55.678665, 37.731228), (55.667863, 37.724756),
                              (55.665047, 37.721970), (55.658536, 37.713152), (55.658172, 37.715081), (55.660390, 37.720201), (55.656472, 37.727081),
                              (55.659406, 37.734468), (55.660216, 37.737737), (55.661463, 37.739640), (55.662985, 37.743588), (55.666030, 37.750250),
                              (55.666797, 37.754071), (55.667616, 37.759602), (55.668843, 37.763939), (55.668073, 37.766417), (55.667502, 37.771118),
                              (55.667150, 37.774963), (55.666405, 37.776967), (55.663528, 37.782386), (55.661930, 37.784927), (55.658613, 37.787578),
                              (55.657806, 37.791264), (55.657570, 37.797114), (55.657544, 37.809827), (55.657058, 37.823419), (55.656957, 37.839147),
                              (55.660985, 37.839752), (55.663646, 37.839423), (55.672641, 37.835978), (55.678181, 37.833996), (55.677016, 37.824514),
                              (55.680468, 37.818026), (55.675843, 37.809833), (55.680569, 37.802443), (55.683033, 37.792585), (55.681450, 37.789540),
                              (55.683285, 37.785643), (55.683189, 37.782395), (55.685363, 37.772691), (55.686574, 37.765943), (55.688042, 37.762695),
                              (55.690693, 37.755756), (55.688973, 37.754173), (55.690376, 37.746174), (55.690147, 37.741871), (55.691615, 37.738907),
                              (55.691936, 37.735578)], 
              'ЮВАО Марьино': [(55.660372, 37.720258), (55.658195, 37.715339), (55.658478, 37.713009), (55.648527, 37.699364), (55.643950, 37.692302),
                                (55.639465, 37.712455), (55.640884, 37.728349), (55.639821, 37.743592), (55.641185, 37.752077), (55.648601, 37.778802),
                                (55.648601, 37.784365), (55.657723, 37.791535), (55.658552, 37.787646), (55.662121, 37.784855), (55.663458, 37.782454),
                                (55.667004, 37.775773), (55.667526, 37.771279), (55.668203, 37.766118), (55.668730, 37.763753), (55.667591, 37.759481),
                                (55.666477, 37.752134), (55.666052, 37.750271), (55.662693, 37.742931), (55.661449, 37.739555), (55.660193, 37.737584),
                                (55.659241, 37.733893), (55.656461, 37.727112), (55.658665, 37.723096)], 
              'ЮВАО_Капотня': [(55.623225, 37.793407), (55.632229, 37.806805), (55.643009, 37.823566), (55.649707, 37.833538), (55.654127, 37.837850),
                                (55.656993, 37.838684), (55.657254, 37.815879), (55.657588, 37.797886), (55.657769, 37.791148), (55.649388, 37.784821),
                                (55.648263, 37.784244), (55.644475, 37.785091), (55.641608, 37.783808), (55.632926, 37.785348),
                                (55.627008, 37.788364)], 
    'ЮВАО_Выхино-Жулебино': [(55.721467, 37.801068), (55.720870, 37.800348), (55.720511, 37.798612), (55.716655, 37.797271), (55.711119, 37.795641),
                              (55.706308, 37.794133), (55.696802, 37.791303), (55.690307, 37.789474), (55.690416, 37.787196), (55.688523, 37.787561),
                              (55.687037, 37.790995), (55.688777, 37.797540), (55.687998, 37.805016), (55.680246, 37.802392), (55.676414, 37.808668),
                              (55.675888, 37.809662), (55.680547, 37.817909), (55.676992, 37.824403), (55.677734, 37.834222), (55.683471, 37.832552),
                              (55.687976, 37.830111), (55.693460, 37.829110), (55.698821, 37.830238), (55.702688, 37.832251), (55.709718, 37.835996),
                              (55.714814, 37.820735), (55.720250, 37.804832)], 
           'САО_Беговой': [(55.792625, 37.549646), (55.791832, 37.548591), (55.791155, 37.550000), (55.785256, 37.541652), (55.785028, 37.541975),
                            (55.785078, 37.542212), (55.783494, 37.544394), (55.780865, 37.545433), (55.777218, 37.551223), (55.776055, 37.555124),
                            (55.774815, 37.554110), (55.773755, 37.554392), (55.775923, 37.577758), (55.777892, 37.582135), (55.780080, 37.585000),
                            (55.792157, 37.588673), (55.791653, 37.574338), (55.785507, 37.565722)], 
           'САО_Аэропорт': [(55.785498, 37.565714), (55.800836, 37.530741), (55.805987, 37.510911), (55.815197, 37.519490), (55.815634, 37.530772),
                             (55.815039, 37.534391), (55.813891, 37.537229), (55.807306, 37.549052), (55.806726, 37.548383), (55.793750, 37.563545),
                             (55.791452, 37.569809), (55.791650, 37.574251)], 
              'САО_Сокол': [(55.815337, 37.486715), (55.814720, 37.507271), (55.815245, 37.520553), (55.805992, 37.510924), (55.803506, 37.520418),
                             (55.794775, 37.517881), (55.794464, 37.518883), (55.793453, 37.518158), (55.790912, 37.503722), (55.794937, 37.502339),
                             (55.811984, 37.487702)], 
        'САО_Хорошёвский': [(55.792625, 37.549646), (55.791832, 37.548591), (55.791155, 37.550000), (55.785256, 37.541652), (55.785028, 37.541975),
                             (55.785078, 37.542212), (55.783494, 37.544394), (55.780865, 37.545433), (55.777218, 37.551223), (55.776055, 37.555124),
                             (55.773814, 37.551934), (55.771340, 37.539225), (55.763364, 37.532197), (55.760091, 37.524459), (55.777515, 37.507039),
                             (55.790916, 37.503468), (55.793455, 37.518140), (55.794434, 37.518892), (55.794757, 37.517804), (55.803503, 37.520402),
                             (55.800778, 37.530914)], 
        'САО_Савёловский': [(55.807310, 37.549045), (55.805398, 37.553773), (55.804938, 37.556285), (55.804955, 37.559618), (55.808289, 37.580635),
                             (55.805617, 37.582286), (55.799516, 37.583828), (55.795483, 37.584271), (55.794510, 37.584743), (55.792162, 37.588357),
                             (55.791492, 37.569633), (55.793607, 37.563705), (55.806722, 37.548380)],
    'САО_Тимирязевский': [(55.807310, 37.549045), (55.805398, 37.553773), (55.804938, 37.556285), (55.804955, 37.559618), (55.808289, 37.580635),
                           (55.816536, 37.577875), (55.822287, 37.574022), (55.831438, 37.573553), (55.837900, 37.578650), (55.840639, 37.578269),
                           (55.844042, 37.575365), (55.848470, 37.573737), (55.849698, 37.564205), (55.849611, 37.560377), (55.848376, 37.555146),
                           (55.844855, 37.560510), (55.836494, 37.538662), (55.835622, 37.537436), (55.834105, 37.539596), (55.832760, 37.538598),
                           (55.831100, 37.531823), (55.825604, 37.532909), (55.822575, 37.531941), (55.821854, 37.531975), (55.820768, 37.533805),
                           (55.819726, 37.535218), (55.817127, 37.537972), (55.814302, 37.539707), (55.812093, 37.540401)], 
            'САО_Войковский': [(55.815245, 37.520553), (55.817454, 37.522016), (55.818655, 37.517955), (55.821191, 37.519654), (55.827461, 37.514366),
                                (55.827764, 37.514963), (55.832171, 37.507996), (55.833215, 37.508728), (55.835202, 37.507216), (55.837213, 37.505027),
                                (55.839368, 37.493780), (55.839381, 37.481373), (55.841027, 37.480134), (55.839844, 37.471830), (55.831785, 37.477370),
                                (55.829427, 37.475268), (55.826551, 37.478176), (55.827000, 37.480740), (55.826933, 37.482833), (55.826493, 37.485760),
                                (55.826549, 37.488241), (55.827043, 37.490261), (55.824101, 37.492399), (55.821628, 37.488546), (55.818028, 37.486430),
                                (55.815337, 37.486715), (55.814720, 37.507271)], 
              'САО_Коптево': [(55.848376, 37.555146), (55.844855, 37.560510), (55.836494, 37.538662), (55.835622, 37.537436), (55.834105, 37.539596),
                               (55.832760, 37.538598), (55.831100, 37.531823), (55.825604, 37.532909), (55.822575, 37.531941), (55.821854, 37.531975),
                               (55.820768, 37.533805), (55.819726, 37.535218), (55.817127, 37.537972), (55.814302, 37.539707), (55.812093, 37.540401),
                               (55.814912, 37.535052), (55.815627, 37.530770), (55.815245, 37.520553), (55.817454, 37.522016), (55.818655, 37.517955),
                               (55.821191, 37.519654), (55.827461, 37.514366), (55.827764, 37.514963), (55.832171, 37.507996), (55.833215, 37.508728),
                               (55.835202, 37.507216), (55.837213, 37.505027), (55.838040, 37.514960), (55.840847, 37.517524), (55.841437, 37.519473),
                               (55.841243, 37.520911), (55.841728, 37.522339), (55.841077, 37.522999), (55.846868, 37.550359)],
              'САО_Головинский': [(55.848369, 37.555137), (55.846864, 37.550341), (55.841135, 37.522935), (55.841719, 37.522325), (55.841234, 37.520904),
                                   (55.841255, 37.518677), (55.840466, 37.517147), (55.838036, 37.514968), (55.837176, 37.504877), (55.839361, 37.493776),
                                   (55.839378, 37.481374), (55.840789, 37.480184), (55.839839, 37.471832), (55.846835, 37.466231), (55.848488, 37.474939),
                                   (55.849762, 37.474031), (55.850452, 37.479120), (55.850315, 37.485999), (55.850757, 37.491855), (55.857782, 37.511291),
                                   (55.862234, 37.505011), (55.864290, 37.509824), (55.867663, 37.514077), (55.854038, 37.546082)],
        'САО_Левобережный': [(55.852473, 37.496625), (55.859110, 37.489065), (55.858205, 37.485382), (55.866073, 37.476427), (55.866735, 37.479098),
                              (55.876200, 37.469153), (55.876577, 37.469388), (55.880046, 37.465846), (55.880909, 37.464159), (55.882771, 37.461049),
                              (55.879372, 37.437317), (55.867384, 37.456051), (55.847270, 37.468727), (55.848478, 37.474930), (55.849756, 37.474049),
                              (55.850442, 37.479124), (55.850333, 37.483713), (55.850443, 37.488891), (55.850830, 37.492090)],
            'САО_Ховрино': [(55.852473, 37.496625), (55.859110, 37.489065), (55.858205, 37.485382), (55.866073, 37.476427), (55.866735, 37.479098),
                             (55.876200, 37.469153), (55.876577, 37.469388), (55.880046, 37.465846), (55.880909, 37.464159), (55.882771, 37.461049),
                             (55.883271, 37.466724), (55.884922, 37.473909), (55.867633, 37.514149), (55.864605, 37.510297), (55.862235, 37.505026),
                             (55.857736, 37.511188)],
        'САО_Бескудниковский': [(55.877378, 37.546665), (55.878981, 37.555662), (55.878936, 37.557603), (55.875220, 37.557733), (55.863509, 37.566621),
                                 (55.862978, 37.563963), (55.861578, 37.564566), (55.861617, 37.563156), (55.859451, 37.563430), (55.859333, 37.567852),
                                 (55.854454, 37.566338), (55.854835, 37.572979), (55.848612, 37.573689), (55.849596, 37.566818), (55.857381, 37.560325),
                                 (55.872835, 37.529636), (55.873422, 37.531752), (55.874521, 37.530885), (55.875318, 37.534117), (55.876116, 37.533748),
                                 (55.878291, 37.546249)], 
    'САО_Восточное_Дегунино': [(55.897771, 37.552001), (55.891823, 37.551214), (55.891803, 37.550607), (55.891720, 37.550276), (55.890646, 37.550322),
                                (55.890642, 37.550974), (55.887289, 37.550659), (55.887237, 37.543023), (55.882204, 37.543967), (55.877378, 37.546665),
                                (55.878981, 37.555662), (55.878936, 37.557603), (55.875220, 37.557733), (55.863509, 37.566621), (55.862978, 37.563963),
                                (55.861578, 37.564566), (55.861617, 37.563156), (55.859451, 37.563430), (55.859333, 37.567852), (55.854454, 37.566338),
                                (55.854835, 37.572979), (55.859635, 37.574508), (55.862344, 37.574511), (55.878656, 37.568399), (55.890266, 37.567098),
                                (55.893314, 37.564128)], 
    'САО_Западное_Дегунино': [(55.849507, 37.566880), (55.857189, 37.560447), (55.870065, 37.534908), (55.871557, 37.531597), (55.874807, 37.526725),
                               (55.889303, 37.511607), (55.886197, 37.506139), (55.892051, 37.496430), (55.885880, 37.481283), (55.880535, 37.484273),
                               (55.853962, 37.546210), (55.848419, 37.555115), (55.849679, 37.561210)],
        'САО_Дмитровский': [(55.897771, 37.552001), (55.891823, 37.551214), (55.891803, 37.550607), (55.891720, 37.550276), (55.890646, 37.550322),
                             (55.890642, 37.550974), (55.887289, 37.550659), (55.887237, 37.543023), (55.882204, 37.543967), (55.878292, 37.546263),
                             (55.876113, 37.533757), (55.875317, 37.534128), (55.874515, 37.530900), (55.873419, 37.531765), (55.873172, 37.530347),
                             (55.872833, 37.529643), (55.874801, 37.526716), (55.889295, 37.511605), (55.886191, 37.506142), (55.892046, 37.496423),
                             (55.905336, 37.526904), (55.907242, 37.536629), (55.904000, 37.538578), (55.900884, 37.543239)],
    'СЗАО_Северное_Тушино': [(55.852471, 37.456606), (55.848756, 37.421210), (55.850419, 37.413168), (55.850462, 37.403307), (55.851283, 37.398558),
                              (55.851805, 37.393773), (55.859662, 37.398031), (55.865812, 37.403781), (55.870894, 37.412027), (55.876590, 37.428327),
                              (55.879386, 37.437309), (55.876198, 37.439412), (55.868150, 37.454374)], 
        'СЗАО_Южное_Тушино': [(55.852066, 37.452777), (55.848756, 37.421210), (55.850419, 37.413168), (55.850462, 37.403307), (55.851283, 37.398558),
                               (55.851805, 37.393773), (55.846890, 37.391813), (55.838089, 37.395909), (55.833660, 37.395875), (55.832550, 37.402519),
                               (55.831739, 37.405481), (55.833264, 37.408048), (55.832682, 37.410544), (55.833304, 37.412154), (55.832919, 37.413700),
                               (55.833716, 37.415792), (55.834719, 37.415233), (55.835196, 37.414610), (55.836063, 37.415505), (55.836511, 37.416936),
                               (55.836435, 37.418386), (55.837440, 37.424035), (55.832218, 37.426528), (55.831687, 37.427527), (55.834619, 37.428507),
                               (55.836415, 37.434852), (55.836426, 37.437820), (55.835087, 37.459248), (55.836691, 37.465365), (55.839255, 37.467210),
                               (55.845035, 37.467860)],
    'СЗАО_Хорошёво-Мнёвники': [(55.795404, 37.454137), (55.791823, 37.454540), (55.790539, 37.465197), (55.789443, 37.468817), (55.788518, 37.477848),
                                (55.788970, 37.478747), (55.787645, 37.489158), (55.789113, 37.500202), (55.789679, 37.501086), (55.791409, 37.503338),
                                (55.777711, 37.506764), (55.761126, 37.520991), (55.758603, 37.514502), (55.767169, 37.493574), (55.768192, 37.485642),
                                (55.767024, 37.478394), (55.762034, 37.476155), (55.753539, 37.476306), (55.746701, 37.471814), (55.742162, 37.462113),
                                (55.740737, 37.455080), (55.741535, 37.448023), (55.746570, 37.437334), (55.749205, 37.436580), (55.752273, 37.445298),
                                (55.755761, 37.457082), (55.762794, 37.464639), (55.776975, 37.449718), (55.774224, 37.430541), (55.776935, 37.404873),
                                (55.781834, 37.403453), (55.792106, 37.412384), (55.789622, 37.420268), (55.786085, 37.429398), (55.788389, 37.439048),
                                (55.794422, 37.446880)],
'СЗАО_Покровское-Стрешнево': [(55.833660, 37.395875), (55.832550, 37.402519), (55.831739, 37.405481), (55.833264, 37.408048), (55.832682, 37.410544),
                               (55.833304, 37.412154), (55.832919, 37.413700), (55.833716, 37.415792), (55.834719, 37.415233), (55.835196, 37.414610),
                               (55.836063, 37.415505), (55.836511, 37.416936), (55.836435, 37.418386), (55.837440, 37.424035), (55.832218, 37.426528),
                               (55.831687, 37.427527), (55.834619, 37.428507), (55.836415, 37.434852), (55.836426, 37.437820), (55.835087, 37.459248),
                               (55.836691, 37.465365), (55.839255, 37.467210), (55.832069, 37.474147), (55.828597, 37.475832), (55.826591, 37.478207),
                               (55.827009, 37.480737), (55.826501, 37.485762), (55.826556, 37.488236), (55.827050, 37.490261), (55.824194, 37.492608),
                               (55.822396, 37.489366), (55.820133, 37.487438), (55.818025, 37.486432), (55.815320, 37.486725), (55.815397, 37.481965),
                               (55.814993, 37.479176), (55.811707, 37.468588), (55.811175, 37.454992), (55.811756, 37.452764), (55.812547, 37.451064),
                               (55.814098, 37.448923), (55.808170, 37.445132), (55.810018, 37.424714), (55.815226, 37.414833), (55.819463, 37.414038),
                               (55.826889, 37.404497), (55.829164, 37.394514)], 
              'СЗАО_Щукино': [(55.815320, 37.486725), (55.815397, 37.481965), (55.814993, 37.479176), (55.811707, 37.468588), (55.811175, 37.454992),
                               (55.811756, 37.452764), (55.812547, 37.451064), (55.814098, 37.448923), (55.805287, 37.446714), (55.794288, 37.445776),
                               (55.795404, 37.454137), (55.791823, 37.454540), (55.790539, 37.465197), (55.789443, 37.468817), (55.788518, 37.477848),
                               (55.788970, 37.478747), (55.787645, 37.489158), (55.789113, 37.500202), (55.789679, 37.501086), (55.791409, 37.503338),
                               (55.794032, 37.502992), (55.812008, 37.487800)],
            'СЗАО_Строгино': [(55.776935, 37.404873), (55.781834, 37.403453), (55.792106, 37.412384), (55.789622, 37.420268), (55.786085, 37.429398),
                               (55.788389, 37.439048), (55.794290, 37.445784), (55.805286, 37.446668), (55.808892, 37.440318), (55.810104, 37.424504),
                               (55.815250, 37.414821), (55.822678, 37.410599), (55.829170, 37.394491), (55.808831, 37.388360), (55.801712, 37.384263),
                               (55.789695, 37.372809), (55.772827, 37.369540), (55.771164, 37.385820), (55.771200, 37.400045),
                               (55.775115, 37.407936)],
              'СВАО_Марфино': [(55.817039, 37.603760), (55.834684, 37.576320), (55.837350, 37.578453), (55.839847, 37.578604), (55.844169, 37.575269),
                                (55.848065, 37.573862), (55.847080, 37.599376), (55.846548, 37.599175), (55.844820, 37.589997), (55.844812, 37.587161),
                                (55.843536, 37.585880), (55.839586, 37.589187), (55.837628, 37.580307), (55.832751, 37.582471), (55.833615, 37.591716),
                                (55.835840, 37.590914), (55.830189, 37.601164)],
        'СВАО_Марьина_роща': [(55.792477, 37.599286), (55.785119, 37.601739), (55.785257, 37.604545), (55.782567, 37.612372), (55.783379, 37.612952),
                               (55.783900, 37.615315), (55.786528, 37.616812), (55.793129, 37.616328), (55.791871, 37.634621), (55.797143, 37.634972),
                               (55.811624, 37.612126), (55.809889, 37.608297), (55.809389, 37.609348), (55.807196, 37.601966), (55.806898, 37.602285),
                               (55.806355, 37.598860), (55.805780, 37.598476), (55.804872, 37.598879), (55.804286, 37.595591), (55.798652, 37.596395),
                               (55.797303, 37.593631), (55.794612, 37.590427), (55.792166, 37.588645)], 
        'СВАО_Алексеевский': [(55.824398, 37.670506), (55.825028, 37.667311), (55.826002, 37.666673), (55.827365, 37.661797), (55.828413, 37.655015),
                               (55.829240, 37.653680), (55.830405, 37.653467), (55.832193, 37.655119), (55.833251, 37.654239), (55.818442, 37.639035),
                               (55.797128, 37.634977), (55.795139, 37.639603), (55.789964, 37.647691), (55.789103, 37.651629), (55.792769, 37.650161),
                               (55.796242, 37.650291), (55.796265, 37.651005), (55.795356, 37.651179), (55.795876, 37.653662), (55.797797, 37.651326),
                               (55.801909, 37.653093), (55.806406, 37.657135), (55.820082, 37.667900)], 
        'СВАО_Останкинский': [(55.847080, 37.599376), (55.846548, 37.599175), (55.844820, 37.589997), (55.844812, 37.587161), (55.843536, 37.585880),
                               (55.839586, 37.589187), (55.837628, 37.580307), (55.832751, 37.582471), (55.833615, 37.591716), (55.835840, 37.590914),
                               (55.830189, 37.601164), (55.817039, 37.603760), (55.797061, 37.634964), (55.818446, 37.639201), (55.828991, 37.649729),
                               (55.834288, 37.634215), (55.835365, 37.634158), (55.835983, 37.632972), (55.835982, 37.632068), (55.838761, 37.631956),
                               (55.837822, 37.636682), (55.845713, 37.635847), (55.846051, 37.626687), (55.855477, 37.625754), (55.857816, 37.625113),
                               (55.859235, 37.620341), (55.855246, 37.620394), (55.854089, 37.619813), (55.850204, 37.615944),
                               (55.846568, 37.612148)], 
            'СВАО_Ростокино': [(55.828991, 37.649729), (55.834288, 37.634215), (55.835365, 37.634158), (55.835983, 37.632972), (55.835982, 37.632068),
                                (55.838761, 37.631956), (55.837822, 37.636682), (55.845713, 37.635847), (55.844719, 37.649913), (55.838837, 37.669956),
                                (55.827797, 37.671591), (55.824398, 37.670506), (55.825028, 37.667311), (55.826002, 37.666673), (55.827365, 37.661797),
                                (55.828413, 37.655015), (55.829240, 37.653680), (55.830405, 37.653467), (55.832193, 37.655119),
                                (55.833251, 37.654239)], 
            'СВАО_Бутырский': [(55.811624, 37.612126), (55.809889, 37.608297), (55.809389, 37.609348), (55.807196, 37.601966), (55.806898, 37.602285),
                                (55.806355, 37.598860), (55.805780, 37.598476), (55.804872, 37.598879), (55.804286, 37.595591), (55.798652, 37.596395),
                                (55.797303, 37.593631), (55.794612, 37.590427), (55.792166, 37.588645), (55.795193, 37.584327), (55.805617, 37.582285),
                                (55.808297, 37.580606), (55.816557, 37.577856), (55.822317, 37.574000), (55.831412, 37.573540),
                                (55.834880, 37.576065)], 
            'СВАО_Свиблово': [(55.865357, 37.641688), (55.861312, 37.646416), (55.857649, 37.652960), (55.856922, 37.661833), (55.850389, 37.671284),
                               (55.846474, 37.670862), (55.840784, 37.663308), (55.844886, 37.648788), (55.846052, 37.626712),
                               (55.857395, 37.625051)], 
        'СВАО_Бабушкинский': [(55.865357, 37.641688), (55.861312, 37.646416), (55.857649, 37.652960), (55.856922, 37.661833), (55.850389, 37.671284),
                               (55.856576, 37.674655), (55.861461, 37.681780), (55.862783, 37.681285), (55.864389, 37.683529), (55.867504, 37.677082),
                               (55.867953, 37.675314), (55.870741, 37.676035), (55.876917, 37.680706), (55.881329, 37.678185), (55.883295, 37.675207),
                               (55.882359, 37.665096), (55.880852, 37.658879), (55.879433, 37.657296), (55.870950, 37.655671)],
    'СВАО_Южное_Медведково': [(55.865357, 37.641688), (55.870950, 37.655671), (55.876611, 37.657291), (55.878396, 37.636361), (55.884163, 37.641639),
                               (55.885735, 37.620307), (55.883040, 37.614307), (55.880320, 37.617079), (55.875075, 37.616148), (55.868821, 37.628946),
                               (55.862222, 37.636407)],
'СВАО_Северное_Медведково': [(55.882359, 37.665096), (55.880852, 37.658879), (55.879433, 37.657296), (55.876611, 37.657291), (55.878396, 37.636361),
                              (55.884163, 37.641639), (55.885735, 37.620307), (55.888125, 37.623045), (55.897425, 37.643888), (55.895400, 37.664699),
                              (55.894860, 37.685083), (55.886964, 37.681643), (55.884776, 37.679960), (55.883288, 37.675205)],
            'СВАО_Отрадное': [(55.857395, 37.625051), (55.862222, 37.636407), (55.868821, 37.628946), (55.875075, 37.616148), (55.880320, 37.617079),
                               (55.874846, 37.586202), (55.870093, 37.585265), (55.869794, 37.576599), (55.870569, 37.575572), (55.871237, 37.572869),
                               (55.870841, 37.571067), (55.861734, 37.574642), (55.859707, 37.574551), (55.854391, 37.572859), (55.848481, 37.573724),
                               (55.846526, 37.611853), (55.854447, 37.620072), (55.859250, 37.620318), (55.857665, 37.625139)], 
        'СВАО_Лианозово': [(55.907242, 37.536629), (55.904000, 37.538578), (55.900884, 37.543239), (55.893557, 37.563702), (55.890919, 37.566819),
                            (55.887784, 37.567680), (55.887370, 37.570239), (55.886929, 37.586655), (55.907676, 37.587384), (55.909827, 37.588479),
                            (55.911111, 37.576170)], 
        'СВАО_Алтуфьевский': [(55.879227, 37.606956), (55.874846, 37.586202), (55.870093, 37.585265), (55.869794, 37.576599), (55.870569, 37.575572),
                               (55.871237, 37.572869), (55.870841, 37.571067), (55.877175, 37.568700), (55.887782, 37.567685), (55.887377, 37.570239),
                               (55.886834, 37.595310), (55.885828, 37.598141), (55.882915, 37.602721)],
            'СВАО_Бибирево': [(55.886834, 37.595310), (55.885828, 37.598141), (55.882915, 37.602721), (55.879227, 37.606956), (55.880320, 37.617079),
                               (55.883040, 37.614307), (55.885735, 37.620307), (55.888125, 37.623045), (55.897425, 37.643888), (55.900839, 37.622399),
                               (55.909828, 37.588470), (55.907677, 37.587371), (55.886931, 37.586646)],
    'СВАО_Лосиноостровский': [(55.861461, 37.681780), (55.862783, 37.681285), (55.864389, 37.683529), (55.867504, 37.677082), (55.867953, 37.675314),
                               (55.870741, 37.676035), (55.876917, 37.680706), (55.881329, 37.678185), (55.883295, 37.675207), (55.884781, 37.679948),
                               (55.886968, 37.681631), (55.894861, 37.685081), (55.894011, 37.699323), (55.892148, 37.706087),
                               (55.885776, 37.719358)], 
        'СВАО_Ярославский': [(55.837572, 37.674853), (55.848034, 37.690308), (55.846903, 37.714057), (55.850437, 37.720007), (55.852972, 37.721646),
                              (55.860551, 37.718030), (55.860850, 37.709263), (55.864964, 37.706808), (55.869758, 37.714950), (55.873731, 37.719747),
                              (55.873256, 37.720951), (55.873861, 37.721593), (55.872991, 37.724271), (55.874956, 37.731544), (55.876979, 37.730934),
                              (55.878753, 37.733733), (55.885866, 37.719485), (55.856690, 37.674785), (55.854363, 37.672870), (55.846701, 37.671290),
                              (55.840755, 37.663400)],
            'ВАО_Сокольники': [(55.789103, 37.651629), (55.792769, 37.650161), (55.796242, 37.650291), (55.796265, 37.651005), (55.795356, 37.651179),
                                (55.795876, 37.653662), (55.797797, 37.651326), (55.801909, 37.653093), (55.806406, 37.657135), (55.820082, 37.667900),
                                (55.824398, 37.670506), (55.818907, 37.678599), (55.817016, 37.687745), (55.815747, 37.684999), (55.810453, 37.693038),
                                (55.810194, 37.698557), (55.808279, 37.700481), (55.806336, 37.701967), (55.801343, 37.698739), (55.799272, 37.698962),
                                (55.795043, 37.703706), (55.793012, 37.705405), (55.791900, 37.704813), (55.788950, 37.699488), (55.786375, 37.698640),
                                (55.782085, 37.699352), (55.782182, 37.688523)],
        'ВАО_Соколиная_Гора': [(55.782692, 37.699280), (55.782728, 37.700773), (55.783922, 37.700858), (55.784582, 37.702031), (55.783351, 37.705284),
                                (55.784442, 37.705738), (55.785483, 37.717605), (55.787923, 37.716889), (55.787076, 37.719652), (55.787009, 37.720708),
                                (55.787854, 37.725723), (55.790804, 37.730573), (55.793097, 37.732715), (55.794037, 37.735316), (55.794115, 37.738115),
                                (55.793991, 37.744721), (55.788442, 37.743029), (55.775932, 37.744075), (55.770028, 37.745358), (55.770222, 37.748082),
                                (55.759182, 37.756408), (55.752002, 37.719497), (55.755991, 37.717613), (55.763483, 37.720670), (55.766483, 37.720541),
                                (55.775308, 37.713621), (55.780326, 37.707889), (55.781753, 37.703237), (55.782104, 37.699345)],
        'ВАО_Преображенское': [(55.801343, 37.698739), (55.799272, 37.698962), (55.795043, 37.703706), (55.793012, 37.705405), (55.791900, 37.704813),
                                (55.788950, 37.699488), (55.786375, 37.698640), (55.782692, 37.699280), (55.782728, 37.700773), (55.783922, 37.700858),
                                (55.784582, 37.702031), (55.783351, 37.705284), (55.784442, 37.705738), (55.785483, 37.717605), (55.787923, 37.716889),
                                (55.787076, 37.719652), (55.787009, 37.720708), (55.787854, 37.725723), (55.790804, 37.730573), (55.793097, 37.732715),
                                (55.794037, 37.735316), (55.794115, 37.738115), (55.793991, 37.744721), (55.798924, 37.746362), (55.809012, 37.746110),
                                (55.809012, 37.746110), (55.808359, 37.735581), (55.808300, 37.733184), (55.809828, 37.729836), (55.806558, 37.724144),
                                (55.803668, 37.707645)], 
        'ВАО_Богородское': [(55.824398, 37.670506), (55.818907, 37.678599), (55.817016, 37.687745), (55.815747, 37.684999), (55.810453, 37.693038),
                             (55.810194, 37.698557), (55.808279, 37.700481), (55.806336, 37.701967), (55.801343, 37.698739), (55.803664, 37.707658),
                             (55.802903, 37.708744), (55.804345, 37.711620), (55.806556, 37.724162), (55.809823, 37.729835), (55.808360, 37.732719),
                             (55.808347, 37.735594), (55.808997, 37.735646), (55.809008, 37.745942), (55.813275, 37.743805), (55.816725, 37.738052),
                             (55.820547, 37.728508), (55.834317, 37.688039), (55.838742, 37.670319), (55.829698, 37.671343)], 
        'ВАО_Метрогородок': [(55.855762, 37.777861), (55.825877, 37.781285), (55.825583, 37.785207), (55.824320, 37.785189), (55.824338, 37.784328),
                              (55.822508, 37.784385), (55.822495, 37.782810), (55.821062, 37.783641), (55.820705, 37.782046), (55.821321, 37.781655),
                              (55.818318, 37.767596), (55.818318, 37.767596), (55.809937, 37.745856), (55.812794, 37.744295), (55.815849, 37.740291),
                              (55.827328, 37.709944), (55.837572, 37.674853), (55.848034, 37.690308), (55.846903, 37.714057), (55.850437, 37.720007),
                              (55.852972, 37.721646), (55.860551, 37.718030), (55.860850, 37.709263), (55.864964, 37.706808), (55.869758, 37.714950),
                              (55.873731, 37.719747), (55.873256, 37.720951), (55.873861, 37.721593), (55.872991, 37.724271), (55.874956, 37.731544),
                              (55.876979, 37.730934), (55.878753, 37.733733)],
            'ВАО_Измайлово': [(55.793991, 37.744721), (55.788442, 37.743029), (55.775932, 37.744075), (55.770028, 37.745358), (55.770222, 37.748082),
                               (55.759182, 37.756408), (55.761783, 37.769862), (55.763215, 37.769236), (55.765342, 37.775410), (55.766016, 37.778838),
                               (55.764978, 37.786307), (55.768023, 37.801335), (55.793246, 37.799452), (55.793370, 37.805598), (55.802957, 37.805131),
                               (55.802317, 37.751749), (55.801387, 37.746399)], 
        'ВАО_Восточное_Измайлово': [(55.788913, 37.841401), (55.787471, 37.833819), (55.789467, 37.829702), (55.790066, 37.823554), (55.788953, 37.823612),
                                 (55.788127, 37.799941), (55.793242, 37.799442), (55.793343, 37.805595), (55.802963, 37.805121),
                                 (55.803401, 37.840531)], 
              'ВАО_Северное_Измайлово': [(55.803394, 37.840535), (55.802307, 37.751757), (55.809475, 37.779540), (55.810660, 37.811708),
                                          (55.813884, 37.838889)],
              'ВАО_Гольяново': [(55.801364, 37.746403), (55.809476, 37.779561), (55.810656, 37.811708), (55.813919, 37.838878), (55.822955, 37.836338),
                                 (55.829635, 37.828532), (55.855762, 37.777861), (55.825877, 37.781285), (55.825583, 37.785207), (55.824320, 37.785189),
                                 (55.824338, 37.784328), (55.822508, 37.784385), (55.822495, 37.782810), (55.821062, 37.783641), (55.820705, 37.782046),
                                 (55.821321, 37.781655), (55.818318, 37.767596), (55.818318, 37.767596), (55.809937, 37.745856)],
              'ВАО_Перово': [(55.761783, 37.769862), (55.763215, 37.769236), (55.765342, 37.775410), (55.766016, 37.778838), (55.764978, 37.786307),
                              (55.768023, 37.801335), (55.760875, 37.800616), (55.761071, 37.802923), (55.760251, 37.808668), (55.759245, 37.808241),
                              (55.759125, 37.809154), (55.758104, 37.808981), (55.758247, 37.807807), (55.753333, 37.805606), (55.753902, 37.800871),
                              (55.750911, 37.784689), (55.745592, 37.787815), (55.745856, 37.783455), (55.743727, 37.782545), (55.743834, 37.781138),
                              (55.743126, 37.780793), (55.743074, 37.781131), (55.742156, 37.780694), (55.742519, 37.780049), (55.741743, 37.778275),
                              (55.738581, 37.776638), (55.738407, 37.774539), (55.736518, 37.771816), (55.735527, 37.773391), (55.734421, 37.767901),
                              (55.737083, 37.761484), (55.738889, 37.750338), (55.745782, 37.737341), (55.749547, 37.723499),
                              (55.752016, 37.719526)],
        'ВАО_Новогиреево': [(55.760251, 37.808668), (55.759245, 37.808241), (55.759125, 37.809154), (55.758104, 37.808981), (55.758247, 37.807807),
                             (55.753333, 37.805606), (55.753902, 37.800871), (55.750911, 37.784689), (55.745592, 37.787815), (55.745856, 37.783455),
                             (55.743727, 37.782545), (55.743834, 37.781138), (55.743126, 37.780793), (55.743074, 37.781131), (55.742156, 37.780694),
                             (55.742519, 37.780049), (55.741743, 37.778275), (55.738581, 37.776638), (55.738407, 37.774539), (55.736518, 37.771816),
                             (55.735527, 37.773391), (55.746085, 37.827579), (55.746650, 37.827221), (55.749516, 37.830728), (55.750619, 37.823779),
                             (55.757690, 37.826896)], 
        'ВАО_Ивановское': [(55.788127, 37.799941), (55.766370, 37.801420), (55.762374, 37.799773), (55.760866, 37.800615), (55.761064, 37.802924),
                            (55.757688, 37.826885), (55.750616, 37.823769), (55.749514, 37.830716), (55.746655, 37.827221), (55.746086, 37.827577),
                            (55.748968, 37.842111), (55.769100, 37.843193), (55.776938, 37.842722), (55.788913, 37.841401), (55.787471, 37.833819),
                            (55.789467, 37.829702), (55.790066, 37.823554), (55.788953, 37.823612)], 
              'ВАО_Вешняки': [(55.734424, 37.767881), (55.748993, 37.842108), (55.718697, 37.838967), (55.709835, 37.836055),
                               (55.730097, 37.776247)], 
        'ЗАО_Дорогомилово': [(55.722912, 37.498413), (55.731415, 37.518409), (55.736257, 37.538323), (55.733671, 37.542734), (55.728760, 37.547049),
                              (55.736767, 37.564307), (55.745345, 37.572537), (55.748367, 37.572997), (55.751620, 37.570657), (55.752543, 37.567720),
                              (55.752140, 37.560441), (55.750301, 37.553791), (55.746562, 37.545965), (55.743570, 37.533299), (55.744007, 37.526343),
                              (55.746599, 37.521571), (55.744806, 37.516454), (55.737985, 37.508614), (55.734467, 37.499048),
                              (55.733048, 37.489275)], 
              'ЗАО_Раменки': [(55.722912, 37.498413), (55.731415, 37.518409), (55.736257, 37.538323), (55.733671, 37.542734), (55.728760, 37.547049),
                               (55.724233, 37.541815), (55.717624, 37.541588), (55.711446, 37.550490), (55.710319, 37.559198), (55.693310, 37.536097),
                               (55.678914, 37.510235), (55.679540, 37.509142), (55.687850, 37.505341), (55.686666, 37.494026), (55.688425, 37.488409),
                               (55.687366, 37.486999), (55.685190, 37.488782), (55.681100, 37.481345), (55.685915, 37.477538), (55.694530, 37.466318),
                               (55.701643, 37.475205), (55.707525, 37.488487)], 
    'ЗАО_Филевский_парк': [(55.733048, 37.489275), (55.734467, 37.499048), (55.737985, 37.508614), (55.744806, 37.516454), (55.746599, 37.521571),
                            (55.758603, 37.514502), (55.767169, 37.493574), (55.768192, 37.485642), (55.767024, 37.478394), (55.762034, 37.476155),
                            (55.753539, 37.476306), (55.746701, 37.471814), (55.742162, 37.462113), (55.740737, 37.455080), (55.741535, 37.448023),
                            (55.746570, 37.437334), (55.749205, 37.436580), (55.749985, 37.431505), (55.747725, 37.426744), (55.739568, 37.439550),
                            (55.738519, 37.439745), (55.737418, 37.438044), (55.734750, 37.445014), (55.736145, 37.447450), (55.737278, 37.452913),
                            (55.735924, 37.454182), (55.743105, 37.480042)],
    'ЗАО_Проспект_Вернадского': [(55.678914, 37.510235), (55.679540, 37.509142), (55.687850, 37.505341), (55.686666, 37.494026), (55.688425, 37.488409),
                                  (55.687366, 37.486999), (55.685190, 37.488782), (55.681100, 37.481345), (55.685915, 37.477538), (55.683465, 37.473174),
                                  (55.670316, 37.484295), (55.669334, 37.495805), (55.663347, 37.508898), (55.673654, 37.524139),
                                  (55.679720, 37.511627)],
            'ЗАО_Крылатское': [(55.749205, 37.436580), (55.752273, 37.445298), (55.755761, 37.457082), (55.762794, 37.464639), (55.776975, 37.449718),
                                (55.774224, 37.430541), (55.775785, 37.409110), (55.771283, 37.400008), (55.770853, 37.387101), (55.771766, 37.381977),
                                (55.770619, 37.369483), (55.769948, 37.369454), (55.744807, 37.422213), (55.749973, 37.431107)], 
              'ЗАО_Кунцево': [(55.728716, 37.378178), (55.729736, 37.391401), (55.727398, 37.410224), (55.726773, 37.447724), (55.734368, 37.443328),
                               (55.737275, 37.437804), (55.738390, 37.439739), (55.739388, 37.439869), (55.747724, 37.426748), (55.744813, 37.422214),
                               (55.769948, 37.369457), (55.747372, 37.369360)], 
              'ЗАО_Можайский': [(55.709489, 37.448718), (55.706757, 37.447888), (55.705850, 37.436294), (55.703874, 37.429805), (55.707793, 37.428067),
                                 (55.709819, 37.421839), (55.698267, 37.403905), (55.713561, 37.385919), (55.728716, 37.378178), (55.729736, 37.391401),
                                 (55.727398, 37.410224), (55.726773, 37.447724), (55.709057, 37.455046)],
        'ЗАО_Фили-Давыдково': [(55.737418, 37.438044), (55.734750, 37.445014), (55.736145, 37.447450), (55.737278, 37.452913), (55.735924, 37.454182),
                                (55.743105, 37.480042), (55.722915, 37.498413), (55.718009, 37.492499), (55.721161, 37.478517), (55.711270, 37.466776),
                                (55.709979, 37.454652), (55.733346, 37.444481)],
    'ЗАО_Очаково-Матвеевское': [(55.718009, 37.492499), (55.721161, 37.478517), (55.711270, 37.466776), (55.709489, 37.448718), (55.706757, 37.447888),
                                 (55.705850, 37.436294), (55.703874, 37.429805), (55.707793, 37.428067), (55.709819, 37.421839), (55.698267, 37.403905),
                                 (55.692556, 37.411362), (55.682889, 37.416804), (55.662338, 37.432415), (55.662343, 37.432413), (55.664385, 37.439780),
                                 (55.669158, 37.446075), (55.674003, 37.459186), (55.685914, 37.477541), (55.694493, 37.466388), (55.701638, 37.475398),
                                 (55.707520, 37.488411), (55.713462, 37.492938)], 
    'ЗАО_Тропарево-Никулино': [(55.683465, 37.473174), (55.670316, 37.484295), (55.669334, 37.495805), (55.663347, 37.508898), (55.658583, 37.501733),
                                (55.638925, 37.459570), (55.662343, 37.432413), (55.664385, 37.439780), (55.669158, 37.446075),
                                (55.674003, 37.459186)], 
              'ЦАО_Арбат': [(55.756103, 37.607397), (55.754011, 37.605997), (55.754138, 37.603778), (55.753661, 37.602358), (55.753695, 37.600417),
                             (55.754217, 37.600352), (55.754212, 37.599164), (55.756700, 37.589527), (55.756380, 37.589211), (55.758229, 37.584570),
                             (55.755272, 37.583845), (55.755198, 37.576766), (55.752577, 37.574174), (55.749209, 37.575936), (55.745466, 37.575806),
                             (55.745852, 37.580268), (55.745442, 37.583034), (55.743766, 37.583897), (55.744110, 37.587432), (55.746308, 37.585811),
                             (55.746872, 37.588388), (55.747670, 37.595807), (55.747090, 37.599378), (55.748958, 37.599729), (55.748622, 37.602936),
                             (55.750404, 37.604244), (55.749136, 37.608804), (55.754570, 37.612274)], 
              'ЦАО_Тверской': [(55.759501, 37.627873), (55.759067, 37.621495), (55.761717, 37.620097), (55.764815, 37.619951), (55.767035, 37.621292),
                                (55.773703, 37.620976), (55.773643, 37.618974), (55.776421, 37.618562), (55.780346, 37.616875), (55.782570, 37.612372),
                                (55.785339, 37.604653), (55.785188, 37.601728), (55.792409, 37.599319), (55.792157, 37.588680), (55.780900, 37.585571),
                                (55.777896, 37.582130), (55.776594, 37.579602), (55.775442, 37.575514), (55.773783, 37.576561), (55.773181, 37.577191),
                                (55.775246, 37.583309), (55.767420, 37.596252), (55.767846, 37.597013), (55.767408, 37.597926), (55.767058, 37.597772),
                                (55.766265, 37.595973), (55.764293, 37.598945), (55.764856, 37.600601), (55.763194, 37.604170), (55.762543, 37.603540),
                                (55.762307, 37.604689), (55.761261, 37.606208), (55.760848, 37.605585), (55.760358, 37.606603), (55.760491, 37.606770),
                                (55.759684, 37.608083), (55.758314, 37.609795), (55.756219, 37.607013), (55.754570, 37.612264), (55.749141, 37.608801),
                                (55.747427, 37.611729), (55.749708, 37.624412), (55.749588, 37.632236), (55.753420, 37.633494), (55.753777, 37.635104),
                                (55.755019, 37.634183)], 
              'ЦАО_Хамовники': [(55.745466, 37.575806), (55.745852, 37.580268), (55.745442, 37.583034), (55.743766, 37.583897), (55.744110, 37.587432),
                                 (55.746308, 37.585811), (55.746872, 37.588388), (55.747670, 37.595807), (55.747090, 37.599378), (55.748958, 37.599729),
                                 (55.748622, 37.602936), (55.750404, 37.604244), (55.748552, 37.610797), (55.746818, 37.612256), (55.743483, 37.608534),
                                 (55.738203, 37.606711), (55.733972, 37.599047), (55.724810, 37.591573), (55.714101, 37.577269), (55.710341, 37.559283),
                                 (55.714138, 37.545200), (55.722763, 37.541225), (55.727904, 37.547176), (55.735415, 37.564098)], 
               'ЦАО_Якиманка': [(55.746818, 37.612256), (55.743483, 37.608534), (55.738203, 37.606711), (55.733972, 37.599047), (55.724810, 37.591573),
                                 (55.714101, 37.577269), (55.711018, 37.582289), (55.711124, 37.585511), (55.713806, 37.588225), (55.721733, 37.599441),
                                 (55.719512, 37.609334), (55.721404, 37.610077), (55.720995, 37.614427), (55.720082, 37.615518), (55.720301, 37.617775),
                                 (55.726940, 37.615124), (55.727355, 37.619948), (55.729220, 37.619634), (55.729930, 37.623858), (55.734527, 37.624359),
                                 (55.739722, 37.624585), (55.742375, 37.625058), (55.745746, 37.624695), (55.745985, 37.626378),
                                 (55.749096, 37.624820)], 
            'ЦАО_Пресненский': [(55.775911, 37.575200), (55.773783, 37.576561), (55.773181, 37.577191), (55.775246, 37.583309), (55.767420, 37.596252),
                                 (55.767846, 37.597013), (55.767408, 37.597926), (55.767058, 37.597772), (55.766265, 37.595973), (55.764293, 37.598945),
                                 (55.764856, 37.600601), (55.763194, 37.604170), (55.762543, 37.603540), (55.762307, 37.604689), (55.761261, 37.606208),
                                 (55.760848, 37.605585), (55.760358, 37.606603), (55.760491, 37.606770), (55.759684, 37.608083), (55.758314, 37.609795),
                                 (55.756219, 37.607013), (55.756103, 37.607397), (55.754011, 37.605997), (55.754138, 37.603778), (55.753661, 37.602358),
                                 (55.753695, 37.600417), (55.754217, 37.600352), (55.754212, 37.599164), (55.756700, 37.589527), (55.756380, 37.589211),
                                 (55.758229, 37.584570), (55.755272, 37.583845), (55.755198, 37.576766), (55.752577, 37.574174), (55.753126, 37.560390),
                                 (55.747183, 37.545244), (55.744596, 37.532591), (55.747041, 37.522584), (55.758459, 37.514404), (55.761694, 37.522541),
                                 (55.759942, 37.524000), (55.763397, 37.532165), (55.772104, 37.540540)], 
              'ЦАО_Мещанский': [(55.793004, 37.643019), (55.792480, 37.640416), (55.789892, 37.642026), (55.788391, 37.642320), (55.788504, 37.641551),
                                 (55.779767, 37.644810), (55.779724, 37.641301), (55.777167, 37.640037), (55.777142, 37.639348), (55.774759, 37.638205),
                                 (55.774759, 37.638205), (55.772147, 37.637287), (55.772696, 37.632322), (55.769878, 37.632345), (55.765660, 37.631027),
                                 (55.763062, 37.628628), (55.762763, 37.630456), (55.761109, 37.629556), (55.760859, 37.630043), (55.759501, 37.627873),
                                 (55.759067, 37.621495), (55.761717, 37.620097), (55.764815, 37.619951), (55.767035, 37.621292), (55.773703, 37.620976),
                                 (55.773643, 37.618974), (55.776421, 37.618562), (55.780346, 37.616875), (55.782570, 37.612372), (55.783382, 37.612942),
                                 (55.783879, 37.615426), (55.786415, 37.616798), (55.793135, 37.616319), (55.791877, 37.634622), (55.797145, 37.634963),
                                 (55.795143, 37.639525)],
              'ЦАО_Таганский': [(55.748889, 37.631925), (55.748480, 37.634114), (55.747012, 37.638558), (55.736736, 37.647098), (55.731884, 37.649943),
                                 (55.728915, 37.654262), (55.726627, 37.656600), (55.726752, 37.663082), (55.730676, 37.660653), (55.731652, 37.665897),
                                 (55.730884, 37.667245), (55.725818, 37.670569), (55.727783, 37.680443), (55.730153, 37.685798), (55.732958, 37.685318),
                                 (55.729607, 37.693922), (55.736426, 37.699047), (55.740280, 37.697302), (55.743504, 37.692897), (55.748477, 37.681605),
                                 (55.750629, 37.670376), (55.753114, 37.666073), (55.752635, 37.664209), (55.754150, 37.662747), (55.754832, 37.657025),
                                 (55.754372, 37.654352), (55.753551, 37.652489), (55.752983, 37.649015), (55.753020, 37.647199), (55.752686, 37.642875),
                                 (55.753084, 37.639512), (55.753554, 37.637769), (55.754200, 37.637011), (55.753425, 37.633483)], 
        'ЦАО_Замоскворечье': [(55.720301, 37.617775), (55.726940, 37.615124), (55.727355, 37.619948), (55.729220, 37.619634), (55.729930, 37.623858),
                               (55.734527, 37.624359), (55.739722, 37.624585), (55.742375, 37.625058), (55.745746, 37.624695), (55.745985, 37.626378),
                               (55.749096, 37.624820), (55.748480, 37.634114), (55.747012, 37.638558), (55.736736, 37.647098), (55.731884, 37.649943),
                               (55.728915, 37.654262), (55.727953, 37.652081), (55.729472, 37.645931), (55.727978, 37.644866), (55.722386, 37.643701),
                               (55.722183, 37.641858), (55.722183, 37.641858), (55.723509, 37.636449), (55.722034, 37.635824), (55.722521, 37.633376),
                               (55.721339, 37.627579), (55.720909, 37.622375)], 
        'ЦАО_Красносельский': [(55.759501, 37.627873), (55.760139, 37.630797), (55.760929, 37.631960), (55.762476, 37.634588), (55.765167, 37.637377),
                                (55.768502, 37.644706), (55.768754, 37.649702), (55.769574, 37.653448), (55.769754, 37.663020), (55.773268, 37.664304),
                                (55.778603, 37.676317), (55.777238, 37.679814), (55.782139, 37.686559), (55.784733, 37.675302), (55.789910, 37.648713),
                                (55.793004, 37.643019), (55.792480, 37.640416), (55.789892, 37.642026), (55.788391, 37.642320), (55.788504, 37.641551),
                                (55.779767, 37.644810), (55.779724, 37.641301), (55.777167, 37.640037), (55.777142, 37.639348), (55.774759, 37.638205),
                                (55.774759, 37.638205), (55.772147, 37.637287), (55.772696, 37.632322), (55.769878, 37.632345), (55.765660, 37.631027),
                                (55.763062, 37.628628), (55.762763, 37.630456), (55.761109, 37.629556), (55.760859, 37.630043)], 
            'ЦАО_Басманный': [(55.750629, 37.670376), (55.753114, 37.666073), (55.752635, 37.664209), (55.754150, 37.662747), (55.754832, 37.657025),
                               (55.754372, 37.654352), (55.753551, 37.652489), (55.752983, 37.649015), (55.753020, 37.647199), (55.752686, 37.642875),
                               (55.753084, 37.639512), (55.753554, 37.637769), (55.754200, 37.637011), (55.753425, 37.633483), (55.759501, 37.627873),
                               (55.760139, 37.630797), (55.760929, 37.631960), (55.762476, 37.634588), (55.765167, 37.637377), (55.768502, 37.644706),
                               (55.768754, 37.649702), (55.769574, 37.653448), (55.769754, 37.663020), (55.773268, 37.664304), (55.778603, 37.676317),
                               (55.777238, 37.679814), (55.782139, 37.686559), (55.782087, 37.699380), (55.781027, 37.706223), (55.775327, 37.713663),
                               (55.766751, 37.697216), (55.769495, 37.692178), (55.765771, 37.687209), (55.762044, 37.685755), (55.759880, 37.683443),
                               (55.759078, 37.678195), (55.759291, 37.672919), (55.758104, 37.670807), (55.753213, 37.672993)]
                        }

In [33]:
def point_inside_polygon(x, y, poly, include_edges = True):
    '''
    Проверьте, находится ли точка (x,y) внутри многоугольника poly.

    poly - это многоугольник с N вершинами, определенный как 
    [(x1,y1),...,(xN,yN)] или [(x1,y1),...,(xN,yN),(x1,y1)]
    (функция отлично работает в обоих случаях)

    Геометрическая идея: точка находится внутри многоугольника, если горизонтальный луч
    справа от точки пересекает многоугольник четное число раз. 
    Отлично работает для невыпуклых многоугольников.
    '''
    n = len(poly)
    inside = False

    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if p1y == p2y:
            if y == p1y:
                if min(p1x, p2x) <= x <= max(p1x, p2x):
                    # point is on horisontal edge
                    inside = include_edges
                    break
                elif x < min(p1x, p2x):  # point is to the left from current edge
                    inside = not inside
        else:  # p1y!= p2y
            if min(p1y, p2y) <= y <= max(p1y, p2y):
                xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x

                if x == xinters:  # point is right on the edge
                    inside = include_edges
                    break

                if x < xinters:  # point is to the left from current edge
                    inside = not inside

        p1x, p1y = p2x, p2y

    return inside

In [34]:
from tqdm import tqdm

k = len(df_310['lat']) #Широта

districts_r = []

for i in tqdm(range(0, k, 1), 'Обработано'):
    for j, c, p in zip(range(1, len(polygons_r.keys()) + 1, 1), polygons_r.keys(), polygons_r.values()): #enumerate(polygons.items())
        poly = p
        x = df_310['lat'][i] 
        y = df_310['lon'][i]
        res = point_inside_polygon(x, y, poly, include_edges = True)

        if res == True:
            district = c
            districts_r.append(district)
            break
        
        if (j == len(polygons_r.keys()) and res == False):
            district = 999
            districts_r.append(district)

Обработано: 100%|██████████| 3257/3257 [00:10<00:00, 305.48it/s]


In [35]:
 len(districts_r)

3257

In [36]:
df_310['poly_r'] = pd.DataFrame(districts_r, columns = ['poly_r'])
df_310

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,999,999
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,999,999
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,999,999
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,999,999
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка


### Полигоны кольца

In [37]:
polygon_circle = {'Бульварное_кольцо': [(55.766593, 37.631433), (55.766813, 37.622955), (55.768195, 37.613502), (55.764721, 37.605550),
                                   (55.757618, 37.598710), (55.754607, 37.600678), (55.751084, 37.600638), (55.746588, 37.599971),
                                   (55.745187, 37.600939), (55.742703, 37.606907), (55.746507, 37.610037), (55.747849, 37.613331),
                                   (55.749677, 37.624429), (55.749505, 37.632287), (55.747741, 37.639336), (55.750427, 37.642446),
                                   (55.751293, 37.645645), (55.755647, 37.648446), (55.758044, 37.646185), (55.761645, 37.643401),
                                   (55.763893, 37.639845), (55.765852, 37.635925), (55.766398, 37.633339)],
            'Остров_Балчуг': [(55.738995, 37.609030), (55.739635, 37.608403), (55.742481, 37.608922), (55.744242, 37.609765),
                               (55.746289, 37.612249), (55.747849, 37.616905), (55.748604, 37.624240), (55.748836, 37.628291),
                               (55.748007, 37.633887), (55.746752, 37.637938), (55.746752, 37.637938), (55.740658, 37.644020),
                               (55.735961, 37.645972), (55.733561, 37.642105), (55.734029, 37.641411), (55.738518, 37.639187),
                               (55.743282, 37.636822), (55.745513, 37.633538), (55.746307, 37.626134), (55.744421, 37.618526),
                               (55.741303, 37.612530), (55.739013, 37.609450)],
            'Замоскворечье_до_СК': [(55.742414, 37.615657), (55.741348, 37.613369), (55.733646, 37.600325), (55.730009, 37.612045),
                                     (55.729785, 37.623269), (55.730148, 37.629014), (55.731283, 37.637698), (55.733294, 37.641585),
                                     (55.736734, 37.639367), (55.740877, 37.638044), (55.742455, 37.637014), (55.745022, 37.634019),
                                     (55.745778, 37.629914), (55.746074, 37.626217), (55.745354, 37.622389), (55.743868, 37.618675)], 
            'Садовое_кольцо': [(55.772801, 37.632343), (55.773709, 37.623209), (55.773357, 37.611216), (55.772480, 37.604441), (55.769956, 37.597275),
                                (55.767658, 37.593231), (55.763994, 37.588351), (55.758915, 37.584667), (55.752638, 37.583364), (55.746907, 37.582464),
                                (55.741359, 37.584676), (55.738589, 37.586218), (55.735472, 37.594305), (55.734231, 37.598275), (55.738535, 37.606438),
                                (55.742595, 37.607903), (55.745952, 37.610598), (55.748086, 37.615935), (55.749215, 37.625820), (55.748646, 37.634073),
                                (55.747078, 37.638884), (55.742750, 37.643761), (55.739438, 37.645915), (55.736791, 37.647110), (55.739095, 37.650481),
                                (55.740644, 37.652681), (55.743164, 37.654840), (55.748797, 37.655765), (55.752767, 37.656199), (55.756926, 37.657708),
                                (55.761408, 37.657581), (55.764086, 37.656258), (55.768579, 37.649843), (55.771850, 37.638452),
                                (55.772500, 37.634895)], 
            'ТТК': [(55.791869, 37.634621), (55.792123, 37.630207), (55.792772, 37.623659), (55.792969, 37.609931), (55.792211, 37.595919),
                     (55.791938, 37.579470), (55.791627, 37.574327), (55.780143, 37.558235), (55.775021, 37.553967), (55.774058, 37.550999),
                     (55.773293, 37.546085), (55.770301, 37.542429), (55.767642, 37.538222), (55.762065, 37.535431), (55.752344, 37.531421),
                     (55.749561, 37.532031), (55.741745, 37.534448), (55.736805, 37.538807), (55.733777, 37.543092), (55.728720, 37.546256),
                     (55.723864, 37.551282), (55.720469, 37.560816), (55.714755, 37.576462), (55.707522, 37.586511), (55.705136, 37.594236),
                     (55.701107, 37.608447), (55.701795, 37.614108), (55.705491, 37.619376), (55.705491, 37.619376), (55.703160, 37.651317),
                     (55.703360, 37.657627), (55.711748, 37.672771), (55.715221, 37.683228), (55.720332, 37.701375), (55.724869, 37.712599),
                     (55.736416, 37.698879), (55.739177, 37.697365), (55.743102, 37.697837), (55.747062, 37.699821), (55.750477, 37.697345),
                     (55.755275, 37.693410), (55.758257, 37.685758), (55.760404, 37.684438), (55.767099, 37.687742), (55.769539, 37.691549),
                     (55.774795, 37.683952), (55.777225, 37.679960), (55.781318, 37.669678), (55.784418, 37.664697), (55.792308, 37.654451),
                     (55.793584, 37.651265), (55.793555, 37.645278), (55.792223, 37.638670)], 
            'ЧТК': [(55.841194, 37.663900), (55.845322, 37.650087), (55.846718, 37.628816), (55.847646, 37.596730), (55.848426, 37.576373),
                     (55.846900, 37.573207), (55.845083, 37.561273), (55.839657, 37.547262), (55.836327, 37.538443), (55.833308, 37.533289),
                     (55.829441, 37.530669), (55.825142, 37.529622), (55.814107, 37.518450), (55.805582, 37.510543), (55.801306, 37.506666),
                     (55.788131, 37.484703), (55.785320, 37.482274), (55.769080, 37.474022), (55.764187, 37.468352), (55.757850, 37.464451),
                     (55.751745, 37.455100), (55.748496, 37.441098), (55.749926, 37.430700), (55.744760, 37.422061), (55.734643, 37.442960),
                     (55.731952, 37.445545), (55.712571, 37.453788), (55.705049, 37.456693), (55.698882, 37.460911), (55.684777, 37.478896),
                     (55.675504, 37.485054), (55.671672, 37.492008), (55.666968, 37.504143), (55.658817, 37.516450), (55.657501, 37.523871),
                     (55.655658, 37.539177), (55.653240, 37.544735), (55.651051, 37.553406), (55.648534, 37.568363), (55.642412, 37.590100),
                     (55.640727, 37.619723), (55.640201, 37.633782), (55.638579, 37.647991), (55.636588, 37.682396), (55.636677, 37.685875),
                     (55.646847, 37.694954), (55.651459, 37.702499), (55.666643, 37.722435), (55.677135, 37.729540), (55.688196, 37.733967),
                     (55.701973, 37.727946), (55.712183, 37.726841), (55.721962, 37.729273), (55.728256, 37.724367), (55.736669, 37.728514),
                     (55.736669, 37.728514), (55.751361, 37.744687), (55.757666, 37.747389), (55.771637, 37.746575), (55.783297, 37.744122),
                     (55.792884, 37.745780), (55.800911, 37.749136), (55.811707, 37.746540), (55.818248, 37.735731),
                     (55.834796, 37.688236)], 
            'МКАД': [(55.882269, 37.725866), (55.891444, 37.707537), (55.893068, 37.702976), (55.894652, 37.691894), (55.895578, 37.662685),
                      (55.898074, 37.638348), (55.902918, 37.613724), (55.909941, 37.587828), (55.911014, 37.574859), (55.906587, 37.533088),
                      (55.906587, 37.533088), (55.888962, 37.489085), (55.882871, 37.463461), (55.881653, 37.444970), (55.872404, 37.415310),
                      (55.863882, 37.401596), (55.859963, 37.398166), (55.848552, 37.392150), (55.843268, 37.393070), (55.833943, 37.395923),
                      (55.806867, 37.387534), (55.789693, 37.372754), (55.783687, 37.369744), (55.748423, 37.369177), (55.741867, 37.371296),
                      (55.711846, 37.386963), (55.707476, 37.391532), (55.692148, 37.411750), (55.682873, 37.416764), (55.662374, 37.432410),
                      (55.657749, 37.437256), (55.629113, 37.470857), (55.599102, 37.506219), (55.594157, 37.517849), (55.577035, 37.589274),
                      (55.575933, 37.598001), (55.571949, 37.667614), (55.572677, 37.677145), (55.575211, 37.685976), (55.591813, 37.729330),
                      (55.602723, 37.755015), (55.623668, 37.792930), (55.640466, 37.819542), (55.650582, 37.834628), (55.657054, 37.839159),
                      (55.664158, 37.839185), (55.687358, 37.830264), (55.694727, 37.829303), (55.700761, 37.831104), (55.715764, 37.838587),
                      (55.736831, 37.841125), (55.755340, 37.842445), (55.772284, 37.843286), (55.786808, 37.841605), (55.814212, 37.838921),
                      (55.822598, 37.836423), (55.829831, 37.828129), (55.860582, 37.768335), (55.876871, 37.736913)]
           }

In [39]:
def point_inside_polygon(x, y, poly, include_edges = True):
    '''
    Проверьте, находится ли точка (x,y) внутри многоугольника poly.

    poly - это многоугольник с N вершинами, определенный как 
    [(x1,y1),...,(xN,yN)] или [(x1,y1),...,(xN,yN),(x1,y1)]
    (функция отлично работает в обоих случаях)

    Геометрическая идея: точка находится внутри многоугольника, если горизонтальный луч
    справа от точки пересекает многоугольник четное число раз. 
    Отлично работает для невыпуклых многоугольников.
    '''
    n = len(poly)
    inside = False

    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if p1y == p2y:
            if y == p1y:
                if min(p1x, p2x) <= x <= max(p1x, p2x):
                    # point is on horisontal edge
                    inside = include_edges
                    break
                elif x < min(p1x, p2x):  # point is to the left from current edge
                    inside = not inside
        else:  # p1y!= p2y
            if min(p1y, p2y) <= y <= max(p1y, p2y):
                xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x

                if x == xinters:  # point is right on the edge
                    inside = include_edges
                    break

                if x < xinters:  # point is to the left from current edge
                    inside = not inside

        p1x, p1y = p2x, p2y

    return inside

In [40]:
from tqdm import tqdm

k = len(df_310['lat']) #Широта

districts_circle = []

for i in tqdm(range(0, k, 1), 'Обработано'):
    for j, c, p in zip(range(1, len(polygon_circle.keys()) + 1, 1), polygon_circle.keys(), polygon_circle.values()): #enumerate(polygons.items())
        poly = p
        x = df_310['lat'][i] 
        y = df_310['lon'][i]
        res = point_inside_polygon(x, y, poly, include_edges = True)

        if res == True:
            district = c
            districts_circle.append(district)
            break
        
        if (j == len(polygon_circle.keys()) and res == False):
            district = 999
            districts_circle.append(district)

Обработано: 100%|██████████| 3257/3257 [00:00<00:00, 3460.57it/s]


In [41]:
 len(districts_circle)

3257

In [42]:
df_310['poly_circle'] = pd.DataFrame(districts_circle, columns = ['poly_circle'])
df_310

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,999,999,999
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,999,999,999
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,999,999,999
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,999,999,999
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,999,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,999
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,999
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,999
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,999


In [43]:
df_310['poly_circle'].value_counts()

ЧТК                    1135
999                    1057
МКАД                    643
ТТК                     247
Садовое_кольцо           77
Бульварное_кольцо        57
Замоскворечье_до_СК      35
Остров_Балчуг             6
Name: poly_circle, dtype: int64

## Geo_region

In [44]:
df_310['geo_region'] = df_310['poly_okrug'].copy()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
# Зеленоград
df_310['geo_region'] = np.where(df_310['geo_region'] == 999, 'Московская область', 'Москва')
df_310['geo_region'].value_counts()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Москва                2676
Московская область     581
Name: geo_region, dtype: int64

## Дорабатываем полигоны
### Районы

In [46]:
df_310[(df_310['geo_region'] == 'Москва') & (df_310['poly_r'] == 999)]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region
622,1851,1637,3630000000107,77:10:0002001:1441,55.3,2024-11-01,7000000.0,2024-05-14,NaN,"Россия, Москва, Зеленоград, 3с2",55.971904,37.186201,МСК,999,Зеленоград,999,999,Москва
2328,3559,3175,S.0.126.1.2863,77:10:0002005:6574,51.8,2024-11-01,9583000.0,2024-02-01,"Москва г, г Зеленоград, к. 931, кв. 92","Россия, Москва, Зеленоград, к931, подъезд 2, э...",55.982618,37.180317,МСК,999,Зеленоград,999,999,Москва
2862,4093,1920,27366219594,77:10:0001003:2486,49.8,2024-11-01,7983000.0,2021-08-05,"РОССИЯ,,г Москва,,г Зеленоград,,,корпус 1107,117","Россия, Москва, Зеленоград, к1107",55.998675,37.191340,МСК,999,Зеленоград,999,999,Москва
2947,4180,1000,238450093363127021367377358053634738279,77:10:0004005:1871,54.0,2024-11-01,10865908.0,2023-11-16,"124482, г Москва, г Зеленоград, к 337","Россия, Москва, Зеленоград, к337",55.995770,37.215576,МСК,999,Зеленоград,999,999,Москва


In [48]:
df_310['poly_r'] = np.where((df_310['geo_region'] == 'Москва') & (df_310['poly_r'] == 999), 'Зеленоград', df_310['poly_r'])

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
df_310['poly_r'] = np.where(df_310['geo_region'] == 'Московская область', df_310['poly_sputnik'], df_310['poly_r']) 
df_310['poly_r'].unique()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['Заречье', 'Дзержинский', 'Люберцы', 'ВНИИССОК', 'Котельники',
       'Красногорск', 'Балашиха-2', 'Долгопрудный', 'Одинцово',
       'Развилка', 'Химки', 'НАО_Коммунарка', 'Королев',
       'Глухово_Ильинское-Усово_Бузланово', 'Бутово_Парк_ЖК', 'Отрадное',
       'Реутов', 'Путилково', 'Дрожжино', 'НАО_Филимонковский',
       'Трехгорка', 'Янтарный_мкр', 'НАО_Щербинка', 'Железнодорожный_мкр',
       'Мытищи', 'Лыткарино', 'Сапроново', 'Малаховский_квартал_Заречье',
       'Измайлово_рабпос', 'Октябрьский', 'Саввино_мкр', 'Клязьма',
       'Балашиха-1', 'Ромашково', 'САО_Савёловский', 'ЮВАО Марьино',
       'ЗАО_Можайский', 'ЦАО_Арбат', 'ЮАО_Чертаново_Северное',
       'ЗАО_Филевский_парк', 'ЮЗАО_Гагаринский', 'ЦАО_Тверской',
       'ЮЗАО_Ломоносовский', 'ЦАО_Таганский', 'ЗАО_Ново-Переделкино',
       'ЗАО_Тропарево-Никулино', 'ЮВАО_Кузьминки', 'ЦАО_Пресненский',
       'ЮЗАО_Черемушки', 'ЦАО_Якиманка', 'ЮВАО_Лефортово',
       'СВАО_Лосиноостровский', 'ЮВАО_Выхино-Жулебино',
  

### Кольца

In [53]:
df_310['poly_circle'].value_counts()

ЧТК                    1135
999                    1057
МКАД                    643
ТТК                     247
Садовое_кольцо           77
Бульварное_кольцо        57
Замоскворечье_до_СК      35
Остров_Балчуг             6
Name: poly_circle, dtype: int64

In [56]:
df_310['poly_circle'] = np.where((df_310['poly_circle'] == 'Бульварное_кольцо') | (df_310['poly_circle'] == 'Замоскворечье_до_СК') | (df_310['poly_circle'] == 'Остров_Балчуг'), 'Садовое_кольцо', df_310['poly_circle'])
df_310['poly_circle'] = np.where(df_310['poly_circle'] == 999, 'За_МКАД', df_310['poly_circle'])
df_310['poly_circle'].value_counts()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ЧТК               1135
За_МКАД           1057
МКАД               643
ТТК                247
Садовое_кольцо     175
Name: poly_circle, dtype: int64

### Округа

In [57]:
df_310['poly_okrug'].unique()

array([999, 'Новомосковский_административный_округ', 'САО', 'ЮВАО', 'ЗАО',
       'ЦАО', 'ЮАО', 'ЮЗАО', 'ЗАО_Ново-Переделкино', 'СВАО', 'СЗАО',
       'ВАО', 'Зеленоград', 'СЗАО_Митино',
       'Троицкий_административный_округ', 'ВАО_Жулебино', 'ЗАО_Солнцево',
       'СВАО_Северный', 'ВАО_Новокосино', 'ЮЗАО_Южное Бутово',
       'СЗАО_Куркино', 'ЮЗАО_Северное Бутово', 'ВАО_Косино-Ухтомский',
       'ВАО_Некрасовка', 'ВАО_Восточный'], dtype=object)

In [59]:
conditions = [df_310['poly_okrug'].isin(['ЗАО_Ново-Переделкино', 'ЗАО_Солнцево', 'ЗАО_ИЦ_Сколково', 'ЗАО_Рублёво-Архангельское']),
             df_310['poly_okrug'].isin(['САО_Молжаниновский']),
             df_310['poly_okrug'].isin(['ЮЗАО_Южное Бутово', 'ЮЗАО_Северное Бутово']),
             df_310['poly_okrug'].isin(['СЗАО_Куркино', 'СЗАО_Митино']),
             df_310['poly_okrug'].isin(['ВАО_Некрасовка', 'ВАО_Жулебино', 'ВАО_Новокосино', 'ВАО_Косино-Ухтомский', 'ВАО_Восточный']),
             df_310['poly_okrug'].isin(['СВАО_Северный'])
             ]
choises = ['ЗАО', 'САО', 'ЮЗАО', 'СЗАО', 'ВАО', 'СВАО']
df_310['poly_okrug'] = np.select(conditions, choises, default = df_310['poly_okrug'])
df_310['poly_okrug'].unique()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


array([999, 'Новомосковский_административный_округ', 'САО', 'ЮВАО', 'ЗАО',
       'ЦАО', 'ЮАО', 'ЮЗАО', 'СВАО', 'СЗАО', 'ВАО', 'Зеленоград',
       'Троицкий_административный_округ'], dtype=object)

In [60]:
df_310['poly_okrug']= np.where(df_310['geo_region'] == 'Московская область', df_310['poly_sputnik'], df_310['poly_okrug']) 

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
conditions_2 = [df_310['poly_okrug'].isin(['Одинцово', 'Трехгорка', 'Ромашково', 'Немчиновка', 'Новоивановское', 'Заречье']),
             df_310['poly_okrug'].isin(['Красногорск', 'Архангельское', 'Новый', 'Глухово_Ильинское-Усово_Бузланово', 'Путилково', 'Отрадное']),
             df_310['poly_okrug'].isin(['Химки', 'Планерная_Новогорск_парк', 'Клязьма']),
             df_310['poly_okrug'].isin(['Долгопрудный']),
             df_310['poly_okrug'].isin(['Мытищи', 'Королев', 'Нагорное', 'Бородино']),
             df_310['poly_okrug'].isin(['Измайлово_рабпос', 'Булатниково_мкр', 'Живописный_ЖК', 'Южная_Битца_ЖК', 'Бутово_Парк_ЖК', 'Лопатино', 'Восточное_Бутово_ЖК', 'Дрожжино', 'Видное',
            'Расторгуево', 'Первый_Южный_ЖК', 'Новый_мкр', 'Усадьба_Суханово_ЖК', 'Сапроново', 'Совхоз_им_Ленина', 'Развилка', 'Мисайлово', 'Молоково']),
             df_310['poly_okrug'].isin(['Дзержинский', 'Лыткарино', 'Томилино_Парк_ЖК', 'Октябрьский', 'Островцы', 'Жилино-1', 'Люберцы', 'Котельники', 'Томилино', 'Красково', 'Коренёво', 
                                   'Малаховка', 'Малаховский_квартал_Заречье', 'Зенино', 'Марусино', 'Мотяково']),                      
             df_310['poly_okrug'].isin(['Павлино_мкр', 'Реутов', 'Салтыковка_мкр', 'Кучино_мкр', 'Керамик_мкр', 'Ольгино_мкр', 'Железнодорожный_мкр', 'Саввино_мкр', 'Балашиха-1', 
            'Измайловский_Лес_ЖК', 'Новое_Измайлово_мкр', 'Дзержинского_мкр', 'Балашиха-2', 'Балашиха-2', '1_Мая_мкр', 'Янтарный_мкр'])]
choises_2 = ['МО_ЮЗ', 'МО_З', 'МО_СЗ', 'МО_С', 'МО_СВ', 'МО_Ю', 'МО_ЮВ', 'МО_В']

In [64]:
df_310['poly_okrug'] = np.select(conditions_2, choises_2, default = df_310['poly_okrug'])
df_310['poly_okrug'].unique()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['МО_ЮЗ', 'МО_ЮВ', 'ВНИИССОК', 'МО_З', 'МО_В', 'МО_С', 'МО_Ю',
       'МО_СЗ', 'Новомосковский_административный_округ', 'МО_СВ', 'САО',
       'ЮВАО', 'ЗАО', 'ЦАО', 'ЮАО', 'ЮЗАО', 'СВАО', 'СЗАО', 'ВАО',
       'Зеленоград', 'Троицкий_административный_округ'], dtype=object)

### Rings_MKAD

In [65]:
df_310['rings_MKAD'] = df_310['poly_circle'].copy()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
df_310['rings_MKAD'].value_counts()

ЧТК               1135
За_МКАД           1057
МКАД               643
ТТК                247
Садовое_кольцо     175
Name: rings_MKAD, dtype: int64

In [67]:
df_310['rings_MKAD'] = np.where((df_310['rings_MKAD'] == 'ЧТК') | (df_310['rings_MKAD'] == 'МКАД') | (df_310['rings_MKAD'] == 'ТТК') | (df_310['rings_MKAD'] == 'Садовое_кольцо'), 'внутри_МКАД', df_310['rings_MKAD'])

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
df_310['rings_MKAD'].value_counts()

внутри_МКАД    2200
За_МКАД        1057
Name: rings_MKAD, dtype: int64

# Отделка

In [73]:
df_310['repair_type'] = 'cosmetic'

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Интервалы площади

In [77]:
df_310[df_310.area_val.isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type


от_42_до_65     26005
от_28_до_42     18241
от_65_до_85     10903
более_105        6517
от_85_до_105     4380
от_17_до_28      2519

In [78]:
s_labels = ['от_17_до_28', 'от_28_до_42', 'от_42_до_65', 'от_65_до_85', 'от_85_до_105', 'более_105']  
s_bins = [0, 28, 42, 65, 85, 105, 10000]

In [79]:
df_310['ОПлощадь_cut'] = pd.cut(df_310['area_val'], bins = s_bins, labels = s_labels)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
df_310['ОПлощадь_cut'].value_counts()

от_42_до_65     1019
от_65_до_85      699
от_28_до_42      515
более_105        493
от_85_до_105     372
от_17_до_28      159
Name: ОПлощадь_cut, dtype: int64

# Объем рынка *****************

In [81]:
df = pd.read_csv('White_ЦИАН_Мск_МО.csv')
df.columns

Index(['Unnamed: 0', 'cian_id', 'photo', 'url', 'bargainterms_price',
       'bargainterms_price_LN', 'area_total', 'area_total_1', 'unit_price',
       'unit_price_LN', 'rooms_count', 'rooms_count_1', 'floor_1', 'floor',
       'floors_count', 'wall_type', 'last_edited', 'repair_type', 'lat', 'lng',
       'lat_lng', 'geo_region', 'geo_settlement_1', 'geo_city_okrug_1',
       'geo_city_district_1', 'geo_street', 'geo_house_num',
       'distance_to_centre', 'distance_to_centre_1', 'rings_MSK', 'rings_MKAD',
       'subway_name', 'subway_travel_type', 'subway_time_1', 'subway_time',
       'nearest_metro', 'distance_to_nearest_metro', 'description', 'ptn_dadd',
       'Регион_enc', 'med_price', 'Районы_Kmeans', 'type', 'address_elit',
       'bti_year_built', 'is_elit', 'address_full', 'is_renovation',
       'address_renovation', 'Объем_рынка', 'Районы_Kmeans_str'],
      dtype='object')

In [85]:
mapping = dict(zip(df['area_total_1'], df['Объем_рынка']))
df_310['Объем_рынка'] = df_310['ОПлощадь_cut'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [113]:
df_310['Объем_рынка'].value_counts()

25994    1019
10900     699
18230     515
6517      493
4379      372
2517      159
Name: Объем_рынка, dtype: int64

In [115]:
df_310['Объем_рынка'] = df_310['Объем_рынка'].astype(int)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
df_310['Объем_рынка'].dtype

dtype('int64')

# Расстояние до центра

In [89]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в километрах
    lat1_rad = np.radians(lat1) #широта квартиры
    lon1_rad = np.radians(lon1) #долгота квартиры
    lat2_rad = np.radians(lat2) #широта центр
    lon2_rad = np.radians(lon2) #долгота центр

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c  # Расстояние в километрах
    return distance

# Загрузка данных
apartments = df_310[['lat', 'lon']] #.iloc[0:1000]
#metro_stations =  metro[['Название_станции', 'Широта', 'Долгота']]

apartments['distance_to_centre'] = None
## Координата центра
lat_center = 55.753348
lng_center = 37.621568

# Вычисляем расстояния
for i, apartment in tqdm(apartments.iterrows(), total = apartments.shape[0], position = 0):
    distance = haversine(apartment['lat'], apartment['lon'], lat_center, lng_center)
        
    apartments.at[i, 'distance_to_centre'] = distance

apartments

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 3257/3257 [00:00<00:00, 6036.68it/s]


,lat,lon,distance_to_centre
0,55.677143,37.392005,16.690113
1,55.631829,37.861357,20.209697
2,55.631829,37.861357,20.209697
3,55.686508,37.913540,19.738682
4,55.660516,37.225395,26.881163
...,...,...,...
3252,55.570986,37.474210,22.284767
3253,55.570858,37.473015,22.328898
3254,55.570858,37.473015,22.328898
3255,55.531912,37.447413,26.938959


In [90]:
df_310['distance_to_centre'] = apartments['distance_to_centre']

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [106]:
df_310['distance_to_centre'] = df_310['distance_to_centre'].astype(float)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
df_310['distance_to_centre'].dtype

dtype('float64')

In [108]:
df_310['distance_to_centre'].describe()

count    3257.000000
mean       13.364914
std         8.804713
min         0.465150
25%         7.491509
50%         9.981345
75%        19.245997
max        53.447191
Name: distance_to_centre, dtype: float64

# Расстояние до центра (категория)

In [109]:
с_labels = ['до_3_км ', 'от_3_до_5_км', 'от_5_до_8_км', 'от_8_до_14_км', 'от_14_до_19_км', 'от_19_до_30_км', 'от_30_км']  
с_bins = [0, 3, 5, 8, 14, 19, 30, 1000]

In [110]:
df_310['distance_to_centre_1'] = pd.cut(df_310['distance_to_centre'], bins = с_bins, labels = с_labels)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
df_310['distance_to_centre_1'].value_counts()

от_8_до_14_км     1194
от_19_до_30_км     840
от_5_до_8_км       489
от_14_до_19_км     268
от_3_до_5_км       226
до_3_км            190
от_30_км            50
Name: distance_to_centre_1, dtype: int64

In [117]:
df_310.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3257 entries, 0 to 3256
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    3257 non-null   int64         
 1   bank_code             3257 non-null   int64         
 2   obj_cd                3257 non-null   object        
 3   kadastr_no            3257 non-null   object        
 4   area_val              3257 non-null   float64       
 5   report_date           3257 non-null   object        
 6   market_tot_val        3257 non-null   float64       
 7   market_tot_dt         3257 non-null   datetime64[ns]
 8   Адрес                 2532 non-null   object        
 9   Адрес Yandex          3257 non-null   object        
 10  lat                   3257 non-null   float64       
 11  lon                   3257 non-null   float64       
 12  М_МО                  3257 non-null   object        
 13  poly_sputnik      

In [118]:
#writer = pd.ExcelWriter('!310_обогащение.xlsx')
#df_310.to_excel(writer, '310_обогащение')
#writer.save()

# Элитка

In [121]:
df_coord_oo = df_310[['lat', 'lon']]
df_coord_oo

,lat,lon
0,55.677143,37.392005
1,55.631829,37.861357
2,55.631829,37.861357
3,55.686508,37.913540
4,55.660516,37.225395
...,...,...
3252,55.570986,37.474210
3253,55.570858,37.473015
3254,55.570858,37.473015
3255,55.531912,37.447413


In [134]:
df_elit = pd.read_excel('БД_Элитка_Москва.xlsx', sheet_name = 0)
df_elit = df_elit[['Широта', 'Долгота', 'Адрес', 'Тип']]
df_elit

,Широта,Долгота,Адрес,Тип
0,37.557322,55.761269,"улица Костикова, 4к1, Москва",престижный+
1,37.556756,55.761102,"улица Костикова, 4к2, Москва",престижный+
2,37.555966,55.761097,"улица Костикова, 4к3, Москва",престижный+
3,37.555238,55.761178,"улица Костикова, 4к4, Москва",престижный+
4,37.555616,55.761512,"улица Костикова, 4к5, Москва",престижный+
...,...,...,...,...
399,37.609047,55.759289,"Брюсов переулок, 8-10с2, Москва",престижный
400,37.604592,55.761720,"Большой Гнездниковский переулок, 3, Москва",престижный+
401,37.602696,55.754512,"Малый Кисловский переулок, 3, Москва",престижный+
402,37.592635,55.753990,"улица Малая Молчановка, 8с1, Москва",престижный+


In [135]:
df_coord_oa = df_elit[['Долгота', 'Широта']]
df_coord_oa = df_coord_oa.rename(columns = {'Широта': 'lon', 'Долгота': 'lat'})
df_coord_oa

,lat,lon
0,55.761269,37.557322
1,55.761102,37.556756
2,55.761097,37.555966
3,55.761178,37.555238
4,55.761512,37.555616
...,...,...
399,55.759289,37.609047
400,55.761720,37.604592
401,55.754512,37.602696
402,55.753990,37.592635


In [136]:
combined_data = pd.concat([df_coord_oo, df_coord_oa])

In [137]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

In [138]:
table_oo_scaled = scaled_data[:len(df_coord_oo), :]
table_oa_scaled = scaled_data[len(df_coord_oo):, :]

In [139]:
print(len(table_oo_scaled))
print(len(table_oa_scaled))

3257
404


In [140]:
n_neighbors = 1  # Количество ближайших соседей
metric = 'manhattan' # Список метрик для перебора, minkowski, manhattan
algorithm = 'ball_tree' # Список алгоритмов для перебора

knn = NearestNeighbors(
                       n_neighbors = n_neighbors,
                       algorithm = algorithm,
                       metric = metric
                      )

knn.fit(table_oa_scaled)

distances, neighbors_indices = knn.kneighbors(table_oo_scaled)

In [141]:
df_310['nn_index'] = neighbors_indices.flatten()
df_310['dist'] = distances.flatten()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [142]:
df_310['nn_index'].unique()

array([243, 238, 266,  11, 307, 260, 262, 265, 284, 263, 218, 228,  10,
        78, 236, 223, 280, 273, 267, 208, 293, 227, 241, 242,  37,  79,
       347, 197, 362, 356, 363, 322,  17, 112,  63, 252, 261, 256, 214,
       205, 232, 294, 304, 219, 264, 355, 237,  45, 146,  14, 255, 160,
        36, 296, 259, 332,  77, 147, 395, 199,  93, 400, 277,  30, 145,
        95, 379, 383, 403,  86, 148, 161, 200, 377,   7,  64,  22, 158,
       215, 240, 253, 390, 349, 181, 101, 367, 331, 392, 391, 364,  75,
       233, 398,  41,  25, 258, 376, 389,  15, 235,  60, 346, 348, 165,
       246, 388, 203, 108,  31, 282, 316, 271,   4, 315, 402,  13, 204,
       250, 330,   9, 167, 257, 365,  68,  42, 239, 153,  80,   0, 295,
        65, 401, 234, 337, 168, 368, 382, 217, 335,  18, 184,  40,  20,
       113,  67, 396, 374,  26, 341, 206, 319,  34, 114, 139, 340,  61,
       386, 220, 140, 179, 279, 303, 275,  24, 202,  84, 209, 244,  33,
       162, 305,  32, 328,   8, 326])

In [143]:
mapping = dict(zip(df_elit.index, df_elit['Адрес']))
df_310['OA_add'] = df_310['nn_index'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lat']))
df_310['lat_OA'] = df_310['nn_index'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lon']))
df_310['lon_OA'] = df_310['nn_index'].apply(lambda x: mapping.get(x))

df_310

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,26.938959,от_19_до_30_км,265,0.300428,"улица Академика Волгина, 2, Москва",55.656099,37.522117


In [144]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Создаем массив для координат
    coords_1 = np.array([[lat1_rad, lon1_rad]])
    coords_2 = np.array([[lat2_rad, lon2_rad]])
    
    # Вычисляем расстояние
    return haversine_distances(coords_1, coords_2) * 6371000  # Умножаем на радиус Земли в метрах

In [145]:
len(df_310['nn_index'])

3257

In [146]:
metrs = []

for j in tqdm(range(len(df_310['nn_index'])), total = df_310.shape[0], position = 0):
    metrs.append(round(calculate_distance(df_310['lat'].iloc[j], df_310['lon'].iloc[j], df_310['lat_OA'].iloc[j], df_310['lon_OA'].iloc[j])[0][0], 0))

#metrs
df_310['metrs'] = metrs
df_310

100%|██████████| 3257/3257 [00:00<00:00, 9128.23it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506.0
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359.0
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931.0
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,26.938959,от_19_до_30_км,265,0.300428,"ул

In [147]:
sim = []

for i in tqdm(range(len(df_310['nn_index'])), total = df_310.shape[0], position = 0):
    token_set_ratio = fuzz.token_set_ratio(df_310['Адрес Yandex'][i], df_310['OA_add'][i])
    sim.append(token_set_ratio)

df_310['sim'] = sim
df_310

100%|██████████| 3257/3257 [00:01<00:00, 2332.94it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506.0,20
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359.0,41
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931.0,38
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Новомосковский административны...",55.531912,37.447413,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,26.938959

In [148]:
s_1 = df_310['metrs'] <= 30
s_2 = df_310['sim'] >= 94

df_310[s_1 & s_2]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim
642,1871,3395,S.0.126.1.502,77:09:0005004:7060,166.7,2024-11-01,113332700.0,2024-06-05,NaN,"Россия, Москва, Чапаевский переулок, 3",55.798485,37.520680,МСК,999,САО,САО_Хорошёвский,ЧТК,Москва,внутри_МКАД,cosmetic,более_105,6517,8.062255,от_8_до_14_км,355,0.000000,"Чапаевский переулок, 3, Москва",55.798485,37.520680,0.0,100
651,1880,2275,322586812552,77:01:0004042:23269,284.9,2024-11-01,287464000.0,2024-07-16,"Российская Федерация, город Москва, вн.тер.г. ...","Россия, Москва, Краснопресненская набережная, ...",55.751128,37.549633,МСК,999,ЦАО,ЦАО_Пресненский,ТТК,Москва,внутри_МКАД,cosmetic,более_105,6517,4.508273,от_3_до_5_км,45,0.000000,"Краснопресненская набережная, 14Ак1, Москва",55.751128,37.549633,0.0,100
652,1881,2275,322586812636,77:01:0004042:23273,119.9,2024-11-01,96280000.0,2024-07-16,"Российская Федерация, город Москва, вн.тер.г. ...","Россия, Москва, Краснопресненская набережная, ...",55.751128,37.549633,МСК,999,ЦАО,ЦАО_Пресненский,ТТК,Москва,внутри_МКАД,cosmetic,более_105,6517,4.508273,от_3_до_5_км,45,0.000000,"Краснопресненская набережная, 14Ак1, Москва",55.751128,37.549633,0.0,100
653,1882,2275,322591542138,77:01:0004042:23264,285.5,2024-11-01,286357000.0,2024-07-16,"г Москва, наб Краснопресненская, д. 14А, к. 1,...","Россия, Москва, Краснопресненская набережная, ...",55.751128,37.549633,МСК,999,ЦАО,ЦАО_Пресненский,ТТК,Москва,внутри_МКАД,cosmetic,более_105,6517,4.508273,от_3_до_5_км,45,0.000000,"Краснопресненская набережная, 14Ак1, Москва",55.751128,37.549633,0.0,100
654,1883,2275,322591542193,77:01:0004042:23268,120.3,2024-11-01,96481000.0,2024-07-16,"г Москва, наб Краснопресненская, д. 14А, к. 1,...","Россия, Москва, Краснопресненская набережная, ...",55.751128,37.549633,МСК,999,ЦАО,ЦАО_Пресненский,ТТК,Москва,внутри_МКАД,cosmetic,более_105,6517,4.508273,от_3_до_5_км,45,0.000000,"Краснопресненская набережная, 14Ак1, Москва",55.751128,37.549633,0.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014,4247,1000,305095650215170968909740179962450377573,77:01:0002019:2709,220.0,2024-11-01,203832725.0,2024-09-01,"115184, г Москва, р-н Замоскворечье, ул Больша...","Россия, Москва, Большая Татарская улица, 7к1, ...",55.742210,37.633625,МСК,999,ЦАО,ЦАО_Замоскворечье,Садовое_кольцо,Москва,внутри_МКАД,cosmetic,более_105,6517,1.450259,до_3_км,386,0.000264,"Большая Татарская улица, 7к1, Москва",55.742265,37.633805,13.0,100
3019,4252,1000,339910634208937079085948710924091896302,77:01:0001020:2757,87.0,2024-11-01,81311038.0,2023-09-05,"109028, г Москва, Таганский р-н, Серебряническ...","Россия, Москва, Серебрянический переулок, 6",55.749851,37.646112,МСК,999,ЦАО,ЦАО_Таганский,Садовое_кольцо,Москва,внутри_МКАД,cosmetic,от_85_до_105,4379,1.584384,до_3_км,24,0.000000,"Серебрянический переулок, 6, Москва",55.749851,37.646112,0.0,100
3091,4324,2210,602052136162,77:07:0013002:9231,69.8,2024-11-01,33480000.0,2024-08-09,"119361, г Москва, ул Янковского, д. 1, к. 1, к...","Россия, Москва, улица Янковского, 1к1",55.697346,37.473770,МСК,999,ЗАО,ЗАО_Раменки,ЧТК,Москва,внутри_МКАД,cosmetic,от_65_до_85,10900,11.155083,от_8_до_14_км,241,0.000000,"улица Янковского, 1к1, Москва",55.697346,37.473770,0.0,100
3166,4399,3436,12068870497,77:01:0001088:2993,119.9,2024-11-01,223133900.0,2024-09-27,"РОССИЯ,107045,,,г Москва,,,пер. Костянский,д. ...","Россия, Москва, Костянский переулок, 13",55.769403,37.635575,МСК,999,ЦАО,ЦАО_Красносельский,Садовое_кольцо,Москва,внутри_МКАД,cosmetic,более_105,6517,1.988717,до_3_км,64,0.000000,"Костянский переулок, 13, Москва",55.769403,37.635575,0.0,100


In [152]:
df_310['Сцепка_к'] = (df_310['lat'].astype(str)).str.cat(df_310['lon'].astype(str), sep = ' ')
df_310

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506.0,20,55.677143 37.392005
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36,55.631829 37.861357
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36,55.631829 37.861357
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359.0,41,55.686508 37.91354
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592.0,27,55.660516 37.225395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931.0,38,55.570986 37.47421
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37,55.570858 37.473015
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37,55.570858 37.473015
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п Воскресен...","Россия, Москва, Ново

In [153]:
s_1 = df_310['metrs'] <= 30
s_2 = df_310['sim'] >= 94

df_map_el = df_310[s_1 & s_2]
df_map_el.reset_index(drop = True , inplace = True)

mapping = dict(zip(df_map_el['Сцепка_к'], ['elit'] * len(df_map_el['Сцепка_к'])))
df_310['is_elit'] = df_310['Сцепка_к'].apply(lambda x: mapping.get(x))

df_310['is_elit'] = np.where(df_310['is_elit'].isnull(), 'no', df_310['is_elit'])

df_310

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506.0,20,55.677143 37.392005,no
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36,55.631829 37.861357,no
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36,55.631829 37.861357,no
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359.0,41,55.686508 37.91354,no
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592.0,27,55.660516 37.225395,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931.0,38,55.570986 37.47421,no
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37,55.570858 37.473015,no
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37,55.570858 37.473015,no
3255,4489,2210,483699063951,50:21:0130202:6548,39.9,2024-11-01,7110000.0,2024-08-15,"108803, г Москва, п Воскресенское, п

In [154]:
df_310['is_elit'].value_counts()

no      2650
elit     607
Name: is_elit, dtype: int64

In [159]:
# Проставляем тип для элитки
df_elit['Сцепка_к'] = (df_elit['Долгота'].astype(str)).str.cat(df_elit['Широта'].astype(str), sep = ' ')
df_elit

,Широта,Долгота,Адрес,Тип,Сцепка_к
0,37.557322,55.761269,"улица Костикова, 4к1, Москва",престижный+,55.761269 37.557322
1,37.556756,55.761102,"улица Костикова, 4к2, Москва",престижный+,55.761102 37.556756
2,37.555966,55.761097,"улица Костикова, 4к3, Москва",престижный+,55.761097 37.555966
3,37.555238,55.761178,"улица Костикова, 4к4, Москва",престижный+,55.761178 37.555238
4,37.555616,55.761512,"улица Костикова, 4к5, Москва",престижный+,55.761512 37.555616
...,...,...,...,...,...
399,37.609047,55.759289,"Брюсов переулок, 8-10с2, Москва",престижный,55.759289 37.609047
400,37.604592,55.761720,"Большой Гнездниковский переулок, 3, Москва",престижный+,55.76172 37.604592
401,37.602696,55.754512,"Малый Кисловский переулок, 3, Москва",престижный+,55.754512 37.602696
402,37.592635,55.753990,"улица Малая Молчановка, 8с1, Москва",престижный+,55.75399 37.592635


In [160]:
df_310['Сцепка_к_ОА'] = (df_310['lat_OA'].astype(str)).str.cat(df_310['lon_OA'].astype(str), sep = ' ')
df_310

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506.0,20,55.677143 37.392005,no,55.695611 37.47351
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36,55.631829 37.861357,no,55.690522 37.668606
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739.0,36,55.631829 37.861357,no,55.690522 37.668606
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359.0,41,55.686508 37.91354,no,55.690522 37.668606
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592.0,27,55.660516 37.225395,no,55.656543 37.521707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931.0,38,55.570986 37.47421,no,55.656099 37.522117
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967.0,37,55.570858 37.473015,no,55.656099 37.522117
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117

In [162]:
mapping = dict(zip(df_elit['Сцепка_к'], df_elit['Тип']))
df_310['type'] = df_310['Сцепка_к_ОА'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [167]:
df_310['type'] = np.where(df_310['is_elit'] == 'no', 'нет', df_310['type'])

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
df_310['type'].value_counts()

нет            2650
престижный      544
престижный+      63
Name: type, dtype: int64

In [169]:
544+63

607

In [170]:
#writer = pd.ExcelWriter('!310_обогащение.xlsx')
#df_310.to_excel(writer, '310_обогащение')
#writer.save()

# Восстановление данных по выборке
## Восстановление по первичке

In [175]:
df_310 = pd.read_excel('!310_обогащение.xlsx', sheet_name = 0)
df_310 = df_310.drop(columns = ['Unnamed: 0'])
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931,38,55.570986 37.47421,no,55.656099 37.522117,нет
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9967,37,55.570858 37.473015,no,55.656099 37.522117,нет
3254,4488,1000,59099667286840978882315639119735665122,50:21:0120303:2818,40.0,2024-11-01,8541000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_28_до_42,18230,22.328898,от_19_до_30_км,265,0.204266,"улица Академика Волгина, 2, Моск

# 1

In [177]:
df_coord_oo = df_310[['lat', 'lon']]
df_coord_oo

,lat,lon
0,55.677143,37.392005
1,55.631829,37.861357
2,55.631829,37.861357
3,55.686508,37.913540
4,55.660516,37.225395
...,...,...
3252,55.570986,37.474210
3253,55.570858,37.473015
3254,55.570858,37.473015
3255,55.531912,37.447413


In [241]:
df_data = pd.read_csv('Выборка_МиМО_out_second.csv')
df_data = df_data.drop(columns = ['Unnamed: 0'])
df_data

,cian_id,photo,url,bargainterms_price,bargainterms_price_LN,area_total,area_total_1,unit_price,unit_price_LN,rooms_count,rooms_count_1,floor_1,floor,floors_count,wall_type,last_edited,repair_type,lat,lng,lat_lng,geo_region,geo_settlement_1,geo_city_okrug_1,geo_city_district_1,geo_street,geo_house_num,distance_to_centre,distance_to_centre_1,rings_MSK,rings_MKAD,subway_name,subway_travel_type,subway_time_1,subway_time,nearest_metro,distance_to_nearest_metro,description,ptn_dadd,Регион_enc,med_price,Районы_Kmeans,type,address_elit,bti_year_built,is_elit,address_full,is_renovation,address_renovation,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,238345845,https://images.cdn-cian.ru/images/78/391/501/k...,https://www.cian.ru/sale/flat/238345845,13140000,16.391172,65.7,от_65_до_85,199999.0,12.206068,2-комн.,3-к,последний,8,8,Монолитный,2024-10-04 00:00:00,no,55.81953,37.24588,55.81953 37.24588,Московская область,Красногорск,МО_З,Красногорск,улица Пришвина,15,24.614772,от_19_до_30_км,За_МКАД,За_МКАД,Опалиха,walk,до_6_минут,6.0,D2_Опалиха,0.416612,Вашему вниманию предлагается 2-х комнатная ква...,2024-10-16 00:00:00,1,208979.0,4,нет,нет,2020,нет,"Московская область, Красногорск, улица Пришвин...",нет,нет,10900,4,0,1,0,0,0,0,0,1,2,0
1,239198805,https://images.cdn-cian.ru/images/3/648/639/93...,https://www.cian.ru/sale/flat/239198805,7550000,15.837058,50.0,от_42_до_65,151000.0,11.925035,2-комн.,2-к,первый,1,2,Кирпичный,2024-09-28 00:00:00,cosmetic,55.63384,37.84267,55.63384 37.84267,Московская область,Дзержинский,МО_ЮВ,Дзержинский,улица Строителей,10,19.198938,от_19_до_30_км,За_МКАД,За_МКАД,Котельники,transport,нет_пешей_доступности,10.0,Таганско_Краснопресненская_Котельники,4.589888,"Квартира для тех, кто ценит тишину и спокойст...",2024-10-16 00:00:00,1,159635.0,2,нет,нет,1956,нет,"Московская область, Дзержинский, улица Строите...",нет,нет,25994,2,0,1,0,0,0,0,0,0,1,0
2,240154473,https://images.cdn-cian.ru/images/1/673/449/kv...,https://www.cian.ru/sale/flat/240154473,6100000,15.623799,42.2,от_42_до_65,144549.0,11.881374,1-комн.,2-к,этажи_2-5,4,17,Панельный,2024-10-03 00:00:00,euro,55.83250,37.97867,55.8325 37.97867,Московская область,Балашиха-2,МО_В,Балашиха-2,улица 40 лет Победы,29,23.995573,от_19_до_30_км,За_МКАД,За_МКАД,Щёлковская,transport,нет_пешей_доступности,16.0,D5_Балашиха_Балашиха,3.110049,Ян. Квартира в хорошем состоянии! Один взрослы...,2024-10-16 00:00:00,1,152876.0,2,нет,нет,2008,нет,"Московская область, Балашиха-2, улица 40 лет П...",нет,нет,25994,2,0,0,0,0,0,0,0,0,0,0
3,239293843,https://images.cdn-cian.ru/images/2071196285-1...,https://www.cian.ru/sale/flat/239293843,75000000,18.132999,130.0,более_105,576923.0,13.265464,3-комн.,4+,этажи_24-33,27,34,Монолитный,2024-09-25 00:00:00,design,55.76035,37.41940,55.76035 37.4194,Москва,Москва,ЗАО,ЗАО_Крылатское,улица Крылатские Холмы,37,12.673599,от_8_до_14_км,МКАД,внутри_МКАД,Крылатское,walk,от_6_до_12_минут,11.0,Арбатско_Покровская_Крылатское,0.805016,Лот 57704 Вашему вниманию предлагается 3-комна...,2024-10-16 00:00:00,0,376190.0,12,нет,нет,2006,нет,"Москва, улица Крылатские Холмы, 37",нет,нет,6517,12,0,1,0,1,0,0,0,0,2,0
4,239383173,https://images.cdn-cian.ru/images/5/153/839/kv...,https://www.cian.ru/sale/flat/239383173,264200000,19.392217,330.0,более_105,800606.0,13.593124,4+-комн.,4+,этажи_2-5,2,12,Монолитный,2024-09-30 00:00:00,design,55.83210,37.48396,55.8321 37.48396,Москва,Москва,САО,САО_Войковский,Ленинградское шоссе,37к1,12.275179,от_8_до_14_км,МКАД,внутри_МКАД,Войковская,transport,нет_пешей_доступности,3.0,Замоскворецкая_Водный_стадион,0.873577,"ЖК ""Город Яхт"" 2007 года постройки. Охраняемая...",2024-10-16 00:00:00,0,357142.0,10,нет,нет,2007,нет,"Москва, Ленинградское шоссе, 37к1",нет,нет,6517,10,1,1,1,0,0,1,1,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [178]:
df_coord_oa = df_data[['lat', 'lng']]
df_coord_oa = df_coord_oa.rename(columns = {'lng': 'lon'})
df_coord_oa

,lat,lon
0,55.81953,37.24588
1,55.63384,37.84267
2,55.83250,37.97867
3,55.76035,37.41940
4,55.83210,37.48396
...,...,...
68532,55.59151,37.44731
68533,55.68776,37.46253
68534,55.84874,37.63874
68535,55.70990,37.81053


In [179]:
combined_data = pd.concat([df_coord_oo, df_coord_oa])

In [180]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

In [181]:
table_oo_scaled = scaled_data[:len(df_coord_oo), :]
table_oa_scaled = scaled_data[len(df_coord_oo):, :]

In [182]:
print(len(table_oo_scaled))
print(len(table_oa_scaled))

3257
68537


# 2

In [184]:
n_neighbors = 1  # Количество ближайших соседей
metric = 'manhattan' # Список метрик для перебора, minkowski, manhattan
algorithm = 'ball_tree' # Список алгоритмов для перебора

knn = NearestNeighbors(
                       n_neighbors = n_neighbors,
                       algorithm = algorithm,
                       metric = metric
                      )

knn.fit(table_oa_scaled)

distances, neighbors_indices = knn.kneighbors(table_oo_scaled)

In [185]:
df_310['nn_index_data'] = neighbors_indices.flatten()
df_310['dist_data'] = distances.flatten()

In [186]:
df_310['nn_index_data'].unique()

array([18987,   675,  5362,  4571, 18150, 17144, 68517, 26410, 63367,
       44818, 55946,  9734,   240, 65606, 12965, 12212, 66635, 39529,
       48823,  9602, 47897, 19227, 33827, 66200, 27218, 11017,   153,
       60877, 47952, 51951, 16668,  3941, 39326, 15746, 59875,  9596,
       11712, 24144, 64203, 52721, 41843, 21779, 22588, 52773, 49495,
       31517, 18393, 34015, 32035, 16630, 28926, 13879, 36446, 39651,
        9043, 24485, 22863,  6882, 10025,  7879, 38673,  5666,  3017,
       27420, 15309, 48055, 40365, 13966, 23461, 42247, 52540, 58676,
       30272, 67818, 66831, 27166,  3940,  9389,  4167, 33258, 36629,
       42150, 63837, 42028, 37720, 23554, 50966, 10000, 33482, 18107,
       24781, 49756, 53182,  1274, 34698,  9281, 66992, 67604, 40044,
       27212, 18921, 46454, 33061, 43916,  1341, 48499, 35525, 29717,
       62955, 24138, 16870, 46532,   169,    61,  7321, 31261, 35562,
       19810, 50425, 39657, 52770, 15828, 56861, 15355, 14109, 13041,
       20689,  1592,

In [187]:
mapping = dict(zip(df_data.index, df_data['address_full']))
df_310['OA_add_data'] = df_310['nn_index_data'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lat']))
df_310['lat_OA_data'] = df_310['nn_index_data'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lon']))
df_310['lon_OA_data'] = df_310['nn_index_data'].apply(lambda x: mapping.get(x))

df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931,38,55.570986 37.47421,no,55.656099 37.522117,нет,29717,0.000667,"Москва, 0, 20А",55.57084,37.47364
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.328898,от_19_до_30

# 3

In [190]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Создаем массив для координат
    coords_1 = np.array([[lat1_rad, lon1_rad]])
    coords_2 = np.array([[lat2_rad, lon2_rad]])
    
    # Вычисляем расстояние
    return haversine_distances(coords_1, coords_2) * 6371000  # Умножаем на радиус Земли в метрах

In [191]:
len(df_310['nn_index_data'])

3257

In [192]:
metrs = []

for j in tqdm(range(len(df_310['nn_index_data'])), total = df_310.shape[0], position = 0):
    metrs.append(round(calculate_distance(df_310['lat'].iloc[j], df_310['lon'].iloc[j], df_310['lat_OA_data'].iloc[j], df_310['lon_OA_data'].iloc[j])[0][0], 0))

#metrs
df_310['metrs_data'] = metrs
df_310

100%|██████████| 3257/3257 [00:00<00:00, 9039.41it/s]


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931,38,55.570986 37.47421,no,55.656099 37.522117,нет,29717,0.000667,"Москва, 0, 20А",55.57084,37.47364,39.0
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmeti

# 4

In [193]:
sim = []

for i in tqdm(range(len(df_310['nn_index_data'])), total = df_310.shape[0], position = 0):
    token_set_ratio = fuzz.token_set_ratio(df_310['Адрес Yandex'][i], df_310['OA_add_data'][i])
    sim.append(token_set_ratio)

df_310['sim_data'] = sim
df_310

100%|██████████| 3257/3257 [00:01<00:00, 2551.27it/s]


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931,38,55.570986 37.47421,no,55.656099 37.522117,нет,29717,0.000667,"Москва, 0, 20А",55.57084,37.47364,39.0,91
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковский_административный_округ,НАО_Коммун

In [198]:
s_1 = df_310['metrs_data'] <= 30
s_2 = df_310['sim_data'] >= 80

df_310[s_1 & s_2]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100
5,41,2275,288133341290,50:22:0050101:15356,38.6,2024-11-01,8151355.0,2023-06-14,NaN,"Россия, Московская область, Котельники, Соснов...",55.674717,37.854530,МО,Котельники,МО_ЮВ,Котельники,За_МКАД,Московская область,За_МКАД,cosmetic,от_28_до_42,18230,17.011335,от_14_до_19_км,238,0.196055,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,11787,33,55.674717 37.85453,no,55.690522 37.668606,нет,18150,0.000004,"Московская область, Котельники, улица Сосновая...",55.67472,37.85453,0.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,4476,963,S.0.126.1.97928,77:01:0001055:2308,333.6,2024-11-01,315777420.0,2023-02-07,NaN,"Россия, Москва, Малый Лёвшинский переулок, 5к2",55.741632,37.589473,МСК,999,ЦАО,ЦАО_Хамовники,Садовое_кольцо,Москва,внутри_МКАД,cosmetic,более_105,6517,2.394145,до_3_км,326,0.002005,"Большой Лёвшинский переулок, 11, Москва",55.742164,37.588350,92,83,55.741632 37.589473,no,55.742164 37.58835,нет,57591,0.000014,"Москва, Малый Левшинский переулок, 5К2",55.74163,37.58946,1.0,89
3243,4477,3251,3541045,77:07:0001002:23726,50.8,2024-11-01,14664000.0,2024-10-09,NaN,"Россия, Москва, улица Крылатские Холмы, 36к3",55.765777,37.409837,МСК,999,ЗАО,ЗАО_Крылатское,МКАД,Москва,внутри_МКАД,cosmetic,от_42_до_65,25994,13.318993,от_8_до_14_км,140,0.043573,"Островной проезд, 11к5, Москва",55.760697,37.447863,2445,41,55.765777 37.409837,no,55.760

In [199]:
s_1 = df_310['metrs_data'] <= 30
s_2 = df_310['sim_data'] >= 80

df_map_data = df_310[s_1 & s_2]
df_map_data.reset_index(drop = True , inplace = True)

mapping = dict(zip(df_map_data['Сцепка_к'], ['yes'] * len(df_map_data['Сцепка_к'])))
df_310['in_second'] = df_310['Сцепка_к'].apply(lambda x: mapping.get(x))

df_310['in_second'] = np.where(df_310['in_second'].isnull(), 'no', df_310['in_second'])

df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931,38,55.570986 37.47421,no,55.656099 37.522117,нет,29717,0.000667,"Москва, 0, 20А",55.57084,37.47364,39.0,91,no
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570858,37.473015,МСК,999,Новомосковск

In [200]:
df_310['in_second'].value_counts()

yes    2008
no     1249
Name: in_second, dtype: int64

In [201]:
# сцепка для mapping по вторичке
df_310['Сцепка_к_ОА_second'] = (df_310['lat_OA_data'].astype(str)).str.cat(df_310['lon_OA_data'].astype(str), sep = ' ')
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомосковский административны...",55.570986,37.474210,МСК,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,от_42_до_65,25994,22.284767,от_19_до_30_км,265,0.202957,"улица Академика Волгина, 2, Москва",55.656099,37.522117,9931,38,55.570986 37.47421,no,55.656099 37.522117,нет,29717,0.000667,"Москва, 0, 20А",55.57084,37.47364,39.0,91,no,55.57084 37.47364
3253,4487,1000,306192768438041933162280255567369773885,50:21:0120303:2490,45.0,2024-11-01,10242000.0,2023-07-01,"108814, г Мо

## Восстанавливаем по первичке

# 1

In [202]:
df_coord_oo = df_310[['lat', 'lon']]
df_coord_oo

,lat,lon
0,55.677143,37.392005
1,55.631829,37.861357
2,55.631829,37.861357
3,55.686508,37.913540
4,55.660516,37.225395
...,...,...
3252,55.570986,37.474210
3253,55.570858,37.473015
3254,55.570858,37.473015
3255,55.531912,37.447413


In [204]:
df_data_1 = pd.read_csv('Выборка_МиМО_out.csv')
df_data_1 = df_data_1.drop(columns = ['Unnamed: 0'])
df_data_1

,cian_id,photo,url,ОПлощадь,ОПлощадь_cut,Цена,LN(Цена),Цена(кв.м.),Этаж,Этаж_cut,Этажность,address,Название_ЖК,Класс_ЖК,От_застройщика,Регион,Районы_2,Районы_Kmeans,Кольца,Округ,Раст_метро_1,Раст_метро_2,Раст_центр,Раст_центр_2,МКАД,ГП,Статус_сдачи,decoration,Отделка,lat_lng,lat,lng,geo_regiondistrict,geo_settlement,geo_city,geo_street,geo_house_num,subway_name,subway_time,subway_travel_type,highway_name,highway_distance,railway_name,railway_distance,railway_time,railway_travel_type,description,poly_sputnik,poly_okrug,poly_r,До_ввода,nearest_metro,subway_name_4,subway_time_4,subway_travel_type_4,LN(Цена(кв.м.)),Регион_enc,Округ_enc,med_price,med_distance,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,263217614,https://images.cdn-cian.ru/images/2117855995-1...,https://www.cian.ru/sale/flat/263217614,41.10,от_29_до_42,12985861.0,16.379372,315958.0,4,от_2_до_16,6,"Московская область, Красногорск, микрорайон Оп...",8 кленов,бизнес,да,Московская_область,Красногорск,5,за_МКАД,МО_З,от_16_до_30_минут,1.315881,25.166201,от_15_до_30_км,за_МКАД,2024,сдан,NaN,without,55.83499 37.24608,55.83499,37.24608,Красногорский район,Красногорск,Красногорск,аллея Золотая,1,Опалиха,17.0,walk,Волоколамское,11.0,Аникеевка,3.0,7.0,byCar,Эстетика неоклассицизма. Уникальность окружени...,Красногорск,Московская область,Московская область,-198,D2_Опалиха,Опалиха,17.0,walk,12.663365,1,4,253720.0,25.0,22255,5,1,1,1,0,1,0,0,0,4,0
1,263729848,https://images.cdn-cian.ru/images/2294396030-1...,https://www.cian.ru/sale/flat/263729848,61.30,от_43_до_65,13071428.0,16.385939,213237.0,2,от_2_до_16,15,"Московская область, Ленинский район, СНТ Полян...",Римский,комфорт_эконом,да,Московская_область,Развилка,3,за_МКАД,МО_Ю,нет_пешей_доступности,2.311419,19.730815,от_15_до_30_км,за_МКАД,2024,сдан,NaN,without,55.5979 37.77332,55.59790,37.77332,Ленинский городской округ,пос. Развилка,NaN,Римский ЖК,NaN,Зябликово,9.0,transport,Каширское,3.0,Москворечье,8.0,106.0,byFoot,Прямая продажа от застройщика! Жилой Комплекс ...,Развилка,Московская область,Московская область,-15,Люблинско_Дмитровская_Зябликово,Зябликово,9.0,transport,12.270160,1,8,234000.0,20.0,25178,3,0,0,0,0,0,0,0,0,0,0
2,263729993,https://images.cdn-cian.ru/images/2294396053-1...,https://www.cian.ru/sale/flat/263729993,65.80,от_66_до_85,14693140.0,16.502891,223300.0,2,от_2_до_16,8,"Московская область, Ленинский район, СНТ Полян...",Римский,комфорт_эконом,да,Московская_область,Развилка,3,за_МКАД,МО_Ю,нет_пешей_доступности,2.311419,19.730815,от_15_до_30_км,за_МКАД,2024,сдан,without,without,55.5979 37.77332,55.59790,37.77332,Ленинский городской округ,пос. Развилка,NaN,Римский ЖК,NaN,Зябликово,9.0,transport,Каширское,3.0,Москворечье,8.0,106.0,byFoot,Прямая продажа от застройщика! Жилой Комплекс ...,Развилка,Московская область,Московская область,-15,Люблинско_Дмитровская_Зябликово,Зябликово,9.0,transport,12.316271,1,8,234000.0,20.0,11448,3,0,0,0,0,0,0,0,0,0,0
3,263729680,https://images.cdn-cian.ru/images/2294880638-1...,https://www.cian.ru/sale/flat/263729680,37.00,от_29_до_42,8424123.0,15.946610,227679.0,2,от_2_до_16,9,"Московская область, Ленинский район, СНТ Полян...",Римский,комфорт_эконом,да,Московская_область,Развилка,3,за_МКАД,МО_Ю,нет_пешей_доступности,2.311419,19.730815,от_15_до_30_км,за_МКАД,2024,сдан,without,without,55.5979 37.77332,55.59790,37.77332,Ленинский городской округ,пос. Развилка,NaN,Римский ЖК,NaN,Зябликово,9.0,transport,Каширское,3.0,Москворечье,8.0,106.0,byFoot,Прямая продажа от застройщика! Жилой Комплекс ...,Развилка,Московская область,Московская область,-15,Люблинско_Дмитровская_Зябликово,Зябликово,9.0,transport,12.335692,1,8,234000.0,20.0,22255,3,0,0,0,0,0,0,0,0,0,0
4,263591293,https://images.cdn-cian.ru/images/2287688499-1...,https://www.cian.ru/sale/flat/263591293,64.70,от_43_до_65,15851500.0,16.578775,245000.0,14,от_2_до_16,16,"Московская область, Ленинский городской округ,...",Римский,комфорт_эконом,да,Мо

In [205]:
df_coord_oa = df_data_1[['lat', 'lng']]
df_coord_oa = df_coord_oa.rename(columns = {'lng': 'lon'})
df_coord_oa

,lat,lon
0,55.83499,37.24608
1,55.59790,37.77332
2,55.59790,37.77332
3,55.59790,37.77332
4,55.59857,37.77042
...,...,...
76689,55.72840,37.75743
76690,55.69109,37.45996
76691,55.83001,37.40578
76692,55.83025,37.39846


In [206]:
combined_data = pd.concat([df_coord_oo, df_coord_oa])

In [207]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

In [208]:
table_oo_scaled = scaled_data[:len(df_coord_oo), :]
table_oa_scaled = scaled_data[len(df_coord_oo):, :]

In [209]:
print(len(table_oo_scaled))
print(len(table_oa_scaled))

3257
76694


# 2

In [213]:
n_neighbors = 1  # Количество ближайших соседей
metric = 'manhattan' # Список метрик для перебора, minkowski, manhattan
algorithm = 'ball_tree' # Список алгоритмов для перебора

knn = NearestNeighbors(
                       n_neighbors = n_neighbors,
                       algorithm = algorithm,
                       metric = metric
                      )

knn.fit(table_oa_scaled)

distances, neighbors_indices = knn.kneighbors(table_oo_scaled)

In [214]:
df_310['nn_index_data_1'] = neighbors_indices.flatten()
df_310['dist_data_1'] = distances.flatten()

In [215]:
df_310['nn_index_data_1'].unique()

array([13389, 15995,  5125,  2711,  9808, 11520, 10747,  6646, 12594,
        1021, 11054, 61233, 20993,  7952,  9445,   356, 11989, 12652,
        6669,  3718, 12567, 76211, 13779, 15032, 15084,  1030, 58693,
       13776,   618,  9927, 12823,   836,  9303,   168,  6330, 14223,
       13724, 13614, 73366, 15987,  4407,  2840, 11387,  8024, 16579,
        2977, 15948, 14028, 15475,  2032,  2530, 15069,  5044, 16324,
        2428, 10707, 10395,  1582,  4120,  4573,  8703,  7322,   328,
        7492,  1947,  7561,  7915,  6870, 14199,   357,  6241,  7531,
       12486,  5677, 12936,  2225,  5676,  8291, 13031, 44274, 11678,
        6027,   386,  7277, 15491,  7668, 18325,  4644, 55185, 52582,
       11782,  5951,  6551, 16082,  9409, 46024, 17305, 36061, 21461,
       29845, 32613, 65969, 29269, 20894, 36217, 46796, 37957, 19541,
       73789, 47481, 29901, 75716, 19299,  7513, 51630, 50672, 29605,
       45068, 37091, 23116, 40640, 22562, 47459, 76311, 66421, 69094,
       33008, 26939,

In [216]:
mapping = dict(zip(df_data_1.index, df_data_1['address']))
df_310['OA_add_data_1'] = df_310['nn_index_data_1'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lat']))
df_310['lat_OA_data_1'] = df_310['nn_index_data_1'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lon']))
df_310['lon_OA_data_1'] = df_310['nn_index_data_1'].apply(lambda x: mapping.get(x))

df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселение Сосенское, Новомос...","Россия, Москва, Новомо

# 3

In [219]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Создаем массив для координат
    coords_1 = np.array([[lat1_rad, lon1_rad]])
    coords_2 = np.array([[lat2_rad, lon2_rad]])
    
    # Вычисляем расстояние
    return haversine_distances(coords_1, coords_2) * 6371000  # Умножаем на радиус Земли в метрах

In [220]:
len(df_310['nn_index_data_1'])

3257

In [221]:
metrs = []

for j in tqdm(range(len(df_310['nn_index_data_1'])), total = df_310.shape[0], position = 0):
    metrs.append(round(calculate_distance(df_310['lat'].iloc[j], df_310['lon'].iloc[j], df_310['lat_OA_data_1'].iloc[j], df_310['lon_OA_data_1'].iloc[j])[0][0], 0))

#metrs
df_310['metrs_data_1'] = metrs
df_310

100%|██████████| 3257/3257 [00:00<00:00, 9006.73it/s]


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617.0
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-07-01,"108814, г Москва, поселе

# 4

In [222]:
sim = []

for i in tqdm(range(len(df_310['nn_index_data_1'])), total = df_310.shape[0], position = 0):
    token_set_ratio = fuzz.token_set_ratio(df_310['Адрес Yandex'][i], df_310['OA_add_data_1'][i])
    sim.append(token_set_ratio)

df_310['sim_data_1'] = sim
df_310

100%|██████████| 3257/3257 [00:01<00:00, 2189.93it/s]


,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617.0,69
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447.0,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000.0,2023-0

In [226]:
s_1 = df_310['metrs_data_1'] <= 30
s_2 = df_310['sim_data_1'] >= 80

df_310[s_1 & s_2]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1
10,158,1809,50:20:0040111:4357,50:20:0040111:4357,50.0,2024-11-01,8125609.0,2023-10-19,NaN,"Россия, Московская область, Одинцово, микрорай...",55.678900,37.234495,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,25.618768,от_19_до_30_км,243,0.245253,"улица Янковского, 1к3, Москва",55.695611,37.473510,15097,24,55.6789 37.234495,no,55.695611 37.47351,нет,44818,0.000194,"Московская область, Одинцово, 0, 2",55.67883,37.23462,11.0,97,yes,55.67883 37.23462,12594,0.000239,"Московская область, Одинцово, микрорайон Клубн...",55.67883,37.23462,11.0,100
40,223,1481,НККЖ0000000000100362046,50:22:0050203:19701,26.2,2024-11-01,5225000.0,2024-08-23,NaN,"Россия, Московская область, Котельники, микрор...",55.640370,37.854728,МО,Котельники,МО_ЮВ,Котельники,За_МКАД,Московская область,За_МКАД,cosmetic,от_17_до_28,2517,19.269325,от_19_до_30_км,238,0.260839,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12937,37,55.64037 37.854728,no,55.690522 37.668606,нет,11712,0.000002,"Московская область, Котельники, Новые Котельни...",55.64037,37.85473,0.0,95,yes,55.64037 37.85473,15995,0.000002,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,0.0,96
41,224,1481,НККЖ0000000000100363438,50:22:0050203:19772,27.7,2024-11-01,5524000.0,2024-08-23,NaN,"Россия, Московская область, Котельники, микрор...",55.640370,37.854728,МО,Котельники,МО_ЮВ,Котельники,За_МКАД,Московская область,За_МКАД,cosmetic,от_17_до_28,2517,19.269325,от_19_до_30_км,238,0.260839,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12937,37,55.64037 37.854728,no,55.690522 37.668606,нет,11712,0.000002,"Московская область, Котельники, Новые Котельни...",55.64037,37.85473,0.0,95,yes,55.64037 37.85473,15995,0.000002,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,0.0,96
42,225,1481,НККЖ0000000000100363695,50:22:0050203:19707,26.2,2024-11-01,5225000.0,2024-08-23,NaN,"Россия, Московская область, Котельники, микрор...",55.640370,37.854728,МО,Котельники,МО_ЮВ,Котельники,За_МКАД,Московская область,За_МКАД,cosmetic,от_17_до_28,2517,19.269325,от_19_до_30_км,238,0.260839,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12937,37,55.64037 37.854728,no,55.690522 37.668606,нет,11712,0.000002,"Московская область, Котельники, Новые Котельни...",55.64037,37.85473,0.0,95,yes,55.64037 37.85473,15995,0.000002,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,0.0,96
43,226,1481,НККЖ0000000000100364098,50:22:0050203:19694,26.2,2024-11-01,5225000.0,2024-08-23,NaN,"Россия, Московская область, Котельники, микрор...",55.640370,37.854728,МО,Котельники,МО_ЮВ,Котельники,За_МКАД,Московская область,За_МКАД,cosmetic,от_17_до_28,2517,19.269325,от_19_до_30_км,238,0.260839,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12937,37,55.64037 37.854728,no,55.690522 37.668606,нет,11712,0.000002,"Московская область, Котельники, Новые Котельни...",55.64037,37.85473,0.0,95,yes,55.64037 37.85473,15995,0.000002,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,0.0,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,4166,1000,130324669878675530338172038628351898163,77:09:0005008:9488,64.0,2024-11-01,42299000.0,2024-09-01,"125167, г Москва, Хорошевский р-н, Ленинградск...","Россия, Москва, Ленинградский проспе

In [227]:
s_1 = df_310['metrs_data_1'] <= 30
s_2 = df_310['sim_data_1'] >= 80

df_map_data_1 = df_310[s_1 & s_2]
df_map_data_1.reset_index(drop = True , inplace = True)

mapping = dict(zip(df_map_data_1['Сцепка_к'], ['yes'] * len(df_map_data_1['Сцепка_к'])))
df_310['in_first'] = df_310['Сцепка_к'].apply(lambda x: mapping.get(x))

df_310['in_first'] = np.where(df_310['in_first'].isnull(), 'no', df_310['in_first'])

df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54,no
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54,no
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617.0,69,no
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447.0,64,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,

In [228]:
df_310['in_first'].value_counts()

no     2252
yes    1005
Name: in_first, dtype: int64

In [229]:
# сцепка для mapping по первичке
df_310['Сцепка_к_ОА_first'] = (df_310['lat_OA_data_1'].astype(str)).str.cat(df_310['lon_OA_data_1'].astype(str), sep = ' ')
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54,no,55.64037 37.85473
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54,no,55.64037 37.85473
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617.0,69,no,55.68096 37.91318
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447.0,64,no,55.67152 37.23772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [250]:
no_info = df_310[(df_310['in_first'] == 'no') & (df_310['in_second'] == 'no')]
no_info

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183
8,46,1151,10000001363,50:15:0011002:330,100.7,2024-11-01,15105000.0,2024-05-29,NaN,"Россия, Московская область, Балашиха, микрорай...",55.826321,37.923790,МО,Балашиха-2,МО_В,Балашиха-2,За_МКАД,Московская область,За_МКАД,cosmetic,от_85_до_105,4379,20.562710,от_19_до_30_км,307,0.270563,"улица Вильгельма Пика, 1, Москва",55.834699,37.638961,17812,25,55.826321 37.92379,no,55.834699 37.638961,нет,26410,0.000649,"Московская область, Балашиха-2, Гагарина мкр, 29",55.82627,37.92448,43.0,93,no,55.82627 37.92448,10747,0.012322,"Московская область, Балашиха, жилой комплекс И...",55.82727,37.93569,751.0,69,no,55.82727 37.93569
11,159,1067,48634340937,50:21:0050101:6807,56.9,2024-11-01,9195000.0,2024-05-27,NaN,"Россия, Московская область, Ленинский городско...",55.588287,37.762686,МО,Развилка,МО_Ю,Развилка_Совхоз_им_Ленина,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.375862,от_19_до_30_км,238,0.276469,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12810,31,55.588287 37.762686,no,55.690522 37.668606,нет,55946,0.002211,"Московская область, Развилка_Совхоз_им_Ленина,...",55.59004,37.76270,195.0,54,no,55.59004 37.7627,1021,0.006394,"Московская область, Ленинский район, ЖК Римский",55.58933,37.75764,338.0,77,no,55.58933 37.75764
14,183,2440,3630000030756,50:10:0080107:689,126.0,2024-11-01,19000000.0,2024-09-16,NaN,"Россия, Москва, Новомосковский административны...",55.633185,37.462056,МО,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,более_105,6517,16.687379,от_14_до_19_км,265,0.096832,"улица Академика Волгина, 2, Москва",55.656099,37.522117,4549,38,55.633185 37.462056,no,55.656099 37.522117,нет,65606,0.008449,"Москва, улица Академика Варги, 22",55.63358,37.47143,590.0,37,no,55.63358 37.47143,20993,0.017906,"Москва, Мосрентген район, Дудкино д., д.2, стр...",55.62931,37.44988,877.0,41,no,55.62931 37.44988
15,184,2440,3630000008346,50:20:0000000:173579,48.0,2024-11-01,10500000.0,2024-01-15,NaN,"Россия, Московская область, Одинцово, Северная...",55.689218,37.255075,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.034582,от_19_до_30_км,243,0.207434,"улица Янковского, 1к3, Москва",55.695611,37.473510,13709,37,55.689218 37.255075,no,55.695611 37.47351,нет,12965,0.002216,"Московская область, Одинцово, Северная улица, 55",55.68780,37.25559,161.0,97,no,55.6878 37.25559,7952,0.019641,"Московская область, Одинцово, улица Маршала Жу...",55.68088,37.26005,978.0,81,no,55.68088 37.26005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:0120303:2660,47.0,2024-11-01,10533000

In [249]:
df_310['poly_r'] = df_310['poly_r'].replace('Марусино', 'Марусино_Мотяково_Малаховка_Жилино_1_Коренёво')
df_310['poly_r'] = df_310['poly_r'].replace('Мотяково', 'Марусино_Мотяково_Малаховка_Жилино_1_Коренёво')
df_310['poly_r'] = df_310['poly_r'].replace('Малаховка', 'Марусино_Мотяково_Малаховка_Жилино_1_Коренёво')
df_310['poly_r'] = df_310['poly_r'].replace('Жилино-1', 'Марусино_Мотяково_Малаховка_Жилино_1_Коренёво')
df_310['poly_r'] = df_310['poly_r'].replace('Коренёво', 'Марусино_Мотяково_Малаховка_Жилино_1_Коренёво')

df_310['poly_r'] = df_310['poly_r'].replace('Расторгуево', 'Расторгуево_Усадьба_Суханово_Измайлово_Булатниково')
df_310['poly_r'] = df_310['poly_r'].replace('Усадьба_Суханово_ЖК', 'Расторгуево_Усадьба_Суханово_Измайлово_Булатниково')
df_310['poly_r'] = df_310['poly_r'].replace('Измайлово_рабпос', 'Расторгуево_Усадьба_Суханово_Измайлово_Булатниково')
df_310['poly_r'] = df_310['poly_r'].replace('Булатниково_мкр', 'Расторгуево_Усадьба_Суханово_Измайлово_Булатниково')

df_310['poly_r'] = df_310['poly_r'].replace('Керамик_мкр', 'Керамик_мкр_Кучино_мкр')
df_310['poly_r'] = df_310['poly_r'].replace('Кучино_мкр', 'Керамик_мкр_Кучино_мкр')

df_310['poly_r'] = df_310['poly_r'].replace('Совхоз_им_Ленина', 'Развилка_Совхоз_им_Ленина')
df_310['poly_r'] = df_310['poly_r'].replace('Развилка', 'Развилка_Совхоз_им_Ленина')

df_310['poly_r'] = df_310['poly_r'].replace('Салтыковка_мкр', 'Салтыковка_мкр_Дзержинского_мкр')
df_310['poly_r'] = df_310['poly_r'].replace('Дзержинского_мкр', 'Салтыковка_мкр_Дзержинского_мкр')

df_310['poly_r'] = df_310['poly_r'].replace('Дрожжино', 'Дрожжино_Бутово_Парк')
df_310['poly_r'] = df_310['poly_r'].replace('Бутово_Парк_ЖК', 'Дрожжино_Бутово_Парк')

df_310['poly_r'] = df_310['poly_r'].replace('СВАО_Алтуфьевский', 'СВАО_Алтуфьевский_СВАО_Лианозово')
df_310['poly_r'] = df_310['poly_r'].replace('СВАО_Лианозово', 'СВАО_Алтуфьевский_СВАО_Лианозово')

df_310['poly_r'] = df_310['poly_r'].replace('ТАО_Краснопахорский', 'ТАО_Краснопахорский_Бекасово_Вороново')
df_310['poly_r'] = df_310['poly_r'].replace('ТАО_Бекасово', 'ТАО_Краснопахорский_Бекасово_Вороново')
df_310['poly_r'] = df_310['poly_r'].replace('ТАО_Вороново', 'ТАО_Краснопахорский_Бекасово_Вороново')

In [251]:
#Количество уникальных объектов
len(no_info['Сцепка_к'].unique())

450

In [252]:
len(no_info['poly_r'].unique())

128

## mapping из вторички

In [242]:
# Вторичка
df_data.head(5)

,cian_id,photo,url,bargainterms_price,bargainterms_price_LN,area_total,area_total_1,unit_price,unit_price_LN,rooms_count,rooms_count_1,floor_1,floor,floors_count,wall_type,last_edited,repair_type,lat,lng,lat_lng,geo_region,geo_settlement_1,geo_city_okrug_1,geo_city_district_1,geo_street,geo_house_num,distance_to_centre,distance_to_centre_1,rings_MSK,rings_MKAD,subway_name,subway_travel_type,subway_time_1,subway_time,nearest_metro,distance_to_nearest_metro,description,ptn_dadd,Регион_enc,med_price,Районы_Kmeans,type,address_elit,bti_year_built,is_elit,address_full,is_renovation,address_renovation,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,238345845,https://images.cdn-cian.ru/images/78/391/501/k...,https://www.cian.ru/sale/flat/238345845,13140000,16.391172,65.7,от_65_до_85,199999.0,12.206068,2-комн.,3-к,последний,8,8,Монолитный,2024-10-04 00:00:00,no,55.81953,37.24588,55.81953 37.24588,Московская область,Красногорск,МО_З,Красногорск,улица Пришвина,15,24.614772,от_19_до_30_км,За_МКАД,За_МКАД,Опалиха,walk,до_6_минут,6.0,D2_Опалиха,0.416612,Вашему вниманию предлагается 2-х комнатная ква...,2024-10-16 00:00:00,1,208979.0,4,нет,нет,2020,нет,"Московская область, Красногорск, улица Пришвин...",нет,нет,10900,4,0,1,0,0,0,0,0,1,2,0
1,239198805,https://images.cdn-cian.ru/images/3/648/639/93...,https://www.cian.ru/sale/flat/239198805,7550000,15.837058,50.0,от_42_до_65,151000.0,11.925035,2-комн.,2-к,первый,1,2,Кирпичный,2024-09-28 00:00:00,cosmetic,55.63384,37.84267,55.63384 37.84267,Московская область,Дзержинский,МО_ЮВ,Дзержинский,улица Строителей,10,19.198938,от_19_до_30_км,За_МКАД,За_МКАД,Котельники,transport,нет_пешей_доступности,10.0,Таганско_Краснопресненская_Котельники,4.589888,"Квартира для тех, кто ценит тишину и спокойст...",2024-10-16 00:00:00,1,159635.0,2,нет,нет,1956,нет,"Московская область, Дзержинский, улица Строите...",нет,нет,25994,2,0,1,0,0,0,0,0,0,1,0
2,240154473,https://images.cdn-cian.ru/images/1/673/449/kv...,https://www.cian.ru/sale/flat/240154473,6100000,15.623799,42.2,от_42_до_65,144549.0,11.881374,1-комн.,2-к,этажи_2-5,4,17,Панельный,2024-10-03 00:00:00,euro,55.83250,37.97867,55.8325 37.97867,Московская область,Балашиха-2,МО_В,Балашиха-2,улица 40 лет Победы,29,23.995573,от_19_до_30_км,За_МКАД,За_МКАД,Щёлковская,transport,нет_пешей_доступности,16.0,D5_Балашиха_Балашиха,3.110049,Ян. Квартира в хорошем состоянии! Один взрослы...,2024-10-16 00:00:00,1,152876.0,2,нет,нет,2008,нет,"Московская область, Балашиха-2, улица 40 лет П...",нет,нет,25994,2,0,0,0,0,0,0,0,0,0,0
3,239293843,https://images.cdn-cian.ru/images/2071196285-1...,https://www.cian.ru/sale/flat/239293843,75000000,18.132999,130.0,более_105,576923.0,13.265464,3-комн.,4+,этажи_24-33,27,34,Монолитный,2024-09-25 00:00:00,design,55.76035,37.41940,55.76035 37.4194,Москва,Москва,ЗАО,ЗАО_Крылатское,улица Крылатские Холмы,37,12.673599,от_8_до_14_км,МКАД,внутри_МКАД,Крылатское,walk,от_6_до_12_минут,11.0,Арбатско_Покровская_Крылатское,0.805016,Лот 57704 Вашему вниманию предлагается 3-комна...,2024-10-16 00:00:00,0,376190.0,12,нет,нет,2006,нет,"Москва, улица Крылатские Холмы, 37",нет,нет,6517,12,0,1,0,1,0,0,0,0,2,0
4,239383173,https://images.cdn-cian.ru/images/5/153/839/kv...,https://www.cian.ru/sale/flat/239383173,264200000,19.392217,330.0,более_105,800606.0,13.593124,4+-комн.,4+,этажи_2-5,2,12,Монолитный,2024-09-30 00:00:00,design,55.83210,37.48396,55.8321 37.48396,Москва,Москва,САО,САО_Войковский,Ленинградское шоссе,37к1,12.275179,от_8_до_14_км,МКАД,внутри_МКАД,Войковская,transport,нет_пешей_доступности,3.0,Замоскворецкая_Водный_стадион,0.873577,"ЖК ""Город Яхт"" 2007 года постройки. Охраняемая...",2024-10-16 00:00:00,0,357142.0,10,нет,нет,2007,нет,"Москва, Ленинградское шоссе, 37к1",нет,нет,6517,10,1,1,1,0,0,1,1,1,4,2


In [287]:
df_data.head(5)

,cian_id,photo,url,bargainterms_price,bargainterms_price_LN,area_total,area_total_1,unit_price,unit_price_LN,rooms_count,rooms_count_1,floor_1,floor,floors_count,wall_type,last_edited,repair_type,lat,lng,lat_lng,geo_region,geo_settlement_1,geo_city_okrug_1,geo_city_district_1,geo_street,geo_house_num,distance_to_centre,distance_to_centre_1,rings_MSK,rings_MKAD,subway_name,subway_travel_type,subway_time_1,subway_time,nearest_metro,distance_to_nearest_metro,description,ptn_dadd,Регион_enc,med_price,Районы_Kmeans,type,address_elit,bti_year_built,is_elit,address_full,is_renovation,address_renovation,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,238345845,https://images.cdn-cian.ru/images/78/391/501/k...,https://www.cian.ru/sale/flat/238345845,13140000,16.391172,65.7,от_65_до_85,199999.0,12.206068,2-комн.,3-к,последний,8,8,Монолитный,2024-10-04 00:00:00,no,55.81953,37.24588,55.81953 37.24588,Московская область,Красногорск,МО_З,Красногорск,улица Пришвина,15,24.614772,от_19_до_30_км,За_МКАД,За_МКАД,Опалиха,walk,до_6_минут,6.0,D2_Опалиха,0.416612,Вашему вниманию предлагается 2-х комнатная ква...,2024-10-16 00:00:00,1,208979.0,4,нет,нет,2020,нет,"Московская область, Красногорск, улица Пришвин...",нет,нет,10900,4,0,1,0,0,0,0,0,1,2,0
1,239198805,https://images.cdn-cian.ru/images/3/648/639/93...,https://www.cian.ru/sale/flat/239198805,7550000,15.837058,50.0,от_42_до_65,151000.0,11.925035,2-комн.,2-к,первый,1,2,Кирпичный,2024-09-28 00:00:00,cosmetic,55.63384,37.84267,55.63384 37.84267,Московская область,Дзержинский,МО_ЮВ,Дзержинский,улица Строителей,10,19.198938,от_19_до_30_км,За_МКАД,За_МКАД,Котельники,transport,нет_пешей_доступности,10.0,Таганско_Краснопресненская_Котельники,4.589888,"Квартира для тех, кто ценит тишину и спокойст...",2024-10-16 00:00:00,1,159635.0,2,нет,нет,1956,нет,"Московская область, Дзержинский, улица Строите...",нет,нет,25994,2,0,1,0,0,0,0,0,0,1,0
2,240154473,https://images.cdn-cian.ru/images/1/673/449/kv...,https://www.cian.ru/sale/flat/240154473,6100000,15.623799,42.2,от_42_до_65,144549.0,11.881374,1-комн.,2-к,этажи_2-5,4,17,Панельный,2024-10-03 00:00:00,euro,55.83250,37.97867,55.8325 37.97867,Московская область,Балашиха-2,МО_В,Балашиха-2,улица 40 лет Победы,29,23.995573,от_19_до_30_км,За_МКАД,За_МКАД,Щёлковская,transport,нет_пешей_доступности,16.0,D5_Балашиха_Балашиха,3.110049,Ян. Квартира в хорошем состоянии! Один взрослы...,2024-10-16 00:00:00,1,152876.0,2,нет,нет,2008,нет,"Московская область, Балашиха-2, улица 40 лет П...",нет,нет,25994,2,0,0,0,0,0,0,0,0,0,0
3,239293843,https://images.cdn-cian.ru/images/2071196285-1...,https://www.cian.ru/sale/flat/239293843,75000000,18.132999,130.0,более_105,576923.0,13.265464,3-комн.,4+,этажи_24-33,27,34,Монолитный,2024-09-25 00:00:00,design,55.76035,37.41940,55.76035 37.4194,Москва,Москва,ЗАО,ЗАО_Крылатское,улица Крылатские Холмы,37,12.673599,от_8_до_14_км,МКАД,внутри_МКАД,Крылатское,walk,от_6_до_12_минут,11.0,Арбатско_Покровская_Крылатское,0.805016,Лот 57704 Вашему вниманию предлагается 3-комна...,2024-10-16 00:00:00,0,376190.0,12,нет,нет,2006,нет,"Москва, улица Крылатские Холмы, 37",нет,нет,6517,12,0,1,0,1,0,0,0,0,2,0
4,239383173,https://images.cdn-cian.ru/images/5/153/839/kv...,https://www.cian.ru/sale/flat/239383173,264200000,19.392217,330.0,более_105,800606.0,13.593124,4+-комн.,4+,этажи_2-5,2,12,Монолитный,2024-09-30 00:00:00,design,55.83210,37.48396,55.8321 37.48396,Москва,Москва,САО,САО_Войковский,Ленинградское шоссе,37к1,12.275179,от_8_до_14_км,МКАД,внутри_МКАД,Войковская,transport,нет_пешей_доступности,3.0,Замоскворецкая_Водный_стадион,0.873577,"ЖК ""Город Яхт"" 2007 года постройки. Охраняемая...",2024-10-16 00:00:00,0,357142.0,10,нет,нет,2007,нет,"Москва, Ленинградское шоссе, 37к1",нет,нет,6517,10,1,1,1,0,0,1,1,1,4,2


## Количество этажей

In [275]:
df_in_second = df_310[df_310['in_second'] == 'yes']
df_in_second.reset_index(drop = True , inplace = True)
df_in_second

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count
0,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54,no,55.64037 37.85473,NaN
1,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0.0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037.0,54,no,55.64037 37.85473,NaN
2,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7.0,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617.0,69,no,55.68096 37.91318,NaN
3,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1.0,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447.0,64,no,55.67152 37.23772,NaN
4,41,2275,288133341290,50:22:0050101:15356,38.6,2024-11-01,8151355.0,2023-06-14,NaN,"Россия, Московская область, Котельники, Соснов...",55.674717,37.854530,МО,Котельники,МО_ЮВ,Котельники,За_МКАД,Московская область,За_МКАД,cosmetic,от_28_до_42,18230,17.011335,от_14_до_19_км,238,0.196055,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,11787,33,55.674717 37.85453,no,55.690522 37.668606,нет,18150,0.000004,"Московская область, Котельники, улица Сосновая...",55.67472,37.85453,0.0,100,yes,55.67472 37.85453,9808,0.004129,"Россия, Московская область, Котельники, жилой ...",55.67401,37.85134,215.0,79,no,55.67401 37.85134,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [276]:
mapping = dict(zip(df_data['lat_lng'], df_data['floors_count']))
df_in_second['floors_count'] = df_in_second['Сцепка_к_ОА_second'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [266]:
df_in_second.columns

Index(['ID ', 'bank_code', 'obj_cd', 'kadastr_no', 'area_val', 'report_date',
       'market_tot_val', 'market_tot_dt', 'Адрес', 'Адрес Yandex', 'lat',
       'lon', 'М_МО', 'poly_sputnik', 'poly_okrug', 'poly_r', 'poly_circle',
       'geo_region', 'rings_MKAD', 'repair_type', 'ОПлощадь_cut',
       'Объем_рынка', 'distance_to_centre', 'distance_to_centre_1', 'nn_index',
       'dist', 'OA_add', 'lat_OA', 'lon_OA', 'metrs', 'sim', 'Сцепка_к',
       'is_elit', 'Сцепка_к_ОА', 'type', 'nn_index_data', 'dist_data',
       'OA_add_data', 'lat_OA_data', 'lon_OA_data', 'metrs_data', 'sim_data',
       'in_second', 'Сцепка_к_ОА_second', 'nn_index_data_1', 'dist_data_1',
       'OA_add_data_1', 'lat_OA_data_1', 'lon_OA_data_1', 'metrs_data_1',
       'sim_data_1', 'in_first', 'Сцепка_к_ОА_first', 'floors_count'],
      dtype='object')

In [267]:
df_310.columns

Index(['ID ', 'bank_code', 'obj_cd', 'kadastr_no', 'area_val', 'report_date',
       'market_tot_val', 'market_tot_dt', 'Адрес', 'Адрес Yandex', 'lat',
       'lon', 'М_МО', 'poly_sputnik', 'poly_okrug', 'poly_r', 'poly_circle',
       'geo_region', 'rings_MKAD', 'repair_type', 'ОПлощадь_cut',
       'Объем_рынка', 'distance_to_centre', 'distance_to_centre_1', 'nn_index',
       'dist', 'OA_add', 'lat_OA', 'lon_OA', 'metrs', 'sim', 'Сцепка_к',
       'is_elit', 'Сцепка_к_ОА', 'type', 'nn_index_data', 'dist_data',
       'OA_add_data', 'lat_OA_data', 'lon_OA_data', 'metrs_data', 'sim_data',
       'in_second', 'Сцепка_к_ОА_second', 'nn_index_data_1', 'dist_data_1',
       'OA_add_data_1', 'lat_OA_data_1', 'lon_OA_data_1', 'metrs_data_1',
       'sim_data_1', 'in_first', 'Сцепка_к_ОА_first', 'floors_count'],
      dtype='object')

In [271]:
df_data_1.head(2)

,cian_id,photo,url,ОПлощадь,ОПлощадь_cut,Цена,LN(Цена),Цена(кв.м.),Этаж,Этаж_cut,Этажность,address,Название_ЖК,Класс_ЖК,От_застройщика,Регион,Районы_2,Районы_Kmeans,Кольца,Округ,Раст_метро_1,Раст_метро_2,Раст_центр,Раст_центр_2,МКАД,ГП,Статус_сдачи,decoration,Отделка,lat_lng,lat,lng,geo_regiondistrict,geo_settlement,geo_city,geo_street,geo_house_num,subway_name,subway_time,subway_travel_type,highway_name,highway_distance,railway_name,railway_distance,railway_time,railway_travel_type,description,poly_sputnik,poly_okrug,poly_r,До_ввода,nearest_metro,subway_name_4,subway_time_4,subway_travel_type_4,LN(Цена(кв.м.)),Регион_enc,Округ_enc,med_price,med_distance,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,263217614,https://images.cdn-cian.ru/images/2117855995-1...,https://www.cian.ru/sale/flat/263217614,41.1,от_29_до_42,12985861.0,16.379372,315958.0,4,от_2_до_16,6,"Московская область, Красногорск, микрорайон Оп...",8 кленов,бизнес,да,Московская_область,Красногорск,5,за_МКАД,МО_З,от_16_до_30_минут,1.315881,25.166201,от_15_до_30_км,за_МКАД,2024,сдан,NaN,without,55.83499 37.24608,55.83499,37.24608,Красногорский район,Красногорск,Красногорск,аллея Золотая,1,Опалиха,17.0,walk,Волоколамское,11.0,Аникеевка,3.0,7.0,byCar,Эстетика неоклассицизма. Уникальность окружени...,Красногорск,Московская область,Московская область,-198,D2_Опалиха,Опалиха,17.0,walk,12.663365,1,4,253720.0,25.0,22255,5,1,1,1,0,1,0,0,0,4,0
1,263729848,https://images.cdn-cian.ru/images/2294396030-1...,https://www.cian.ru/sale/flat/263729848,61.3,от_43_до_65,13071428.0,16.385939,213237.0,2,от_2_до_16,15,"Московская область, Ленинский район, СНТ Полян...",Римский,комфорт_эконом,да,Московская_область,Развилка,3,за_МКАД,МО_Ю,нет_пешей_доступности,2.311419,19.730815,от_15_до_30_км,за_МКАД,2024,сдан,NaN,without,55.5979 37.77332,55.59790,37.77332,Ленинский городской округ,пос. Развилка,NaN,Римский ЖК,NaN,Зябликово,9.0,transport,Каширское,3.0,Москворечье,8.0,106.0,byFoot,Прямая продажа от застройщика! Жилой Комплекс ...,Развилка,Московская область,Московская область,-15,Люблинско_Дмитровская_Зябликово,Зябликово,9.0,transport,12.270160,1,8,234000.0,20.0,25178,3,0,0,0,0,0,0,0,0,0,0


In [277]:
df_in_first = df_310[(df_310['in_first'] == 'yes') & (df_310['in_second'] == 'no')]
df_in_first.reset_index(drop = True , inplace = True)
df_in_first

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count
0,623,1481,НККЖ0000000000068954144,50:10:0000000:20112,38.5,2024-11-01,7746000.0,2024-05-24,NaN,"Россия, Московская область, Химки, улица Генер...",55.957348,37.452776,МО,Клязьма,МО_СЗ,Клязьма,За_МКАД,Московская область,За_МКАД,cosmetic,от_28_до_42,18230,25.010615,от_19_до_30_км,260,0.259076,"Ильменский проезд, 14к8, Москва",55.862378,37.542689,11955,32,55.957348 37.452776,no,55.862378 37.542689,нет,50966,0.000904,"Московская область, Клязьма, улица Генерала Дь...",55.95722,37.45190,56.0,90,no,55.95722 37.4519,7531,0.000007,"Московская область, Химки, улица Генерала Дьяк...",55.95735,37.45278,0.0,100,yes,55.95735 37.45278,8.0
1,626,1481,НККЖ0000000000068954158,50:10:0000000:20113,40.1,2024-11-01,8068000.0,2024-05-24,NaN,"Россия, Московская область, Химки, улица Генер...",55.957348,37.452776,МО,Клязьма,МО_СЗ,Клязьма,За_МКАД,Московская область,За_МКАД,cosmetic,от_28_до_42,18230,25.010615,от_19_до_30_км,260,0.259076,"Ильменский проезд, 14к8, Москва",55.862378,37.542689,11955,32,55.957348 37.452776,no,55.862378 37.542689,нет,50966,0.000904,"Московская область, Клязьма, улица Генерала Дь...",55.95722,37.45190,56.0,90,no,55.95722 37.4519,7531,0.000007,"Московская область, Химки, улица Генерала Дьяк...",55.95735,37.45278,0.0,100,yes,55.95735 37.45278,8.0
2,627,1481,НККЖ0000000000068954163,50:10:0000000:20048,44.9,2024-11-01,9034000.0,2024-05-24,NaN,"Россия, Московская область, Химки, улица Генер...",55.957348,37.452776,МО,Клязьма,МО_СЗ,Клязьма,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,25.010615,от_19_до_30_км,260,0.259076,"Ильменский проезд, 14к8, Москва",55.862378,37.542689,11955,32,55.957348 37.452776,no,55.862378 37.542689,нет,50966,0.000904,"Московская область, Клязьма, улица Генерала Дь...",55.95722,37.45190,56.0,90,no,55.95722 37.4519,7531,0.000007,"Московская область, Химки, улица Генерала Дьяк...",55.95735,37.45278,0.0,100,yes,55.95735 37.45278,8.0
3,628,1481,НККЖ0000000000068954167,50:10:0000000:20282,46.4,2024-11-01,9336000.0,2024-05-24,NaN,"Россия, Московская область, Химки, улица Генер...",55.957207,37.453100,МО,Клязьма,МО_СЗ,Клязьма,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.987888,от_19_до_30_км,260,0.258521,"Ильменский проезд, 14к8, Москва",55.862378,37.542689,11932,33,55.957207 37.4531,no,55.862378 37.542689,нет,50966,0.001034,"Московская область, Клязьма, улица Генерала Дь...",55.95722,37.45190,75.0,92,no,55.95722 37.4519,7531,0.000546,"Московская область, Химки, улица Генерала Дьяк...",55.95735,37.45278,25.0,100,yes,55.95735 37.45278,8.0
4,629,1481,НККЖ0000000000068954172,50:10:0000000:20331,46.4,2024-11-01,9336000.0,2024-05-24,NaN,"Россия, Московская область, Химки, улица Генер...",55.957469,37.452929,МО,Клязьма,МО_СЗ,Клязьма,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,25.018793,от_19_до_30_км,260,0.259167,"Ильменский проезд, 14к8, Москва",55.862378,37.542689,11962,31,55.957469 37.452929,no,55.862378 37.542689,нет,50966,0.001185,"Московская область, Клязьма, улица Генерала Дь...",55.95722,37.45190,70.0,90,no,55.95722 37.4519,6241,0.000350,"Московская область, Химки, улица Генерала Дьяк...",55.95735,37.45278,16.0,100,yes,55.95735 37.45278,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,4063,2312,40000000004796653,

In [ ]:
mapping = dict(zip(df_data_1['lat_lng'], df_data_1['Этажность']))
df_in_first['floors_count'] = df_in_first['Сцепка_к_ОА_first'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [283]:
df_has_info = pd.concat([df_in_second, df_in_first])

In [284]:
mapping = dict(zip(df_has_info['ID '], df_has_info['floors_count']))
df_310['floors_count'] = df_310['ID '].apply(lambda x: mapping.get(x))

In [285]:
df_310[df_310['floors_count'].isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183,NaN
8,46,1151,10000001363,50:15:0011002:330,100.7,2024-11-01,15105000.0,2024-05-29,NaN,"Россия, Московская область, Балашиха, микрорай...",55.826321,37.923790,МО,Балашиха-2,МО_В,Балашиха-2,За_МКАД,Московская область,За_МКАД,cosmetic,от_85_до_105,4379,20.562710,от_19_до_30_км,307,0.270563,"улица Вильгельма Пика, 1, Москва",55.834699,37.638961,17812,25,55.826321 37.92379,no,55.834699 37.638961,нет,26410,0.000649,"Московская область, Балашиха-2, Гагарина мкр, 29",55.82627,37.92448,43.0,93,no,55.82627 37.92448,10747,0.012322,"Московская область, Балашиха, жилой комплекс И...",55.82727,37.93569,751.0,69,no,55.82727 37.93569,NaN
11,159,1067,48634340937,50:21:0050101:6807,56.9,2024-11-01,9195000.0,2024-05-27,NaN,"Россия, Московская область, Ленинский городско...",55.588287,37.762686,МО,Развилка,МО_Ю,Развилка_Совхоз_им_Ленина,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.375862,от_19_до_30_км,238,0.276469,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12810,31,55.588287 37.762686,no,55.690522 37.668606,нет,55946,0.002211,"Московская область, Развилка_Совхоз_им_Ленина,...",55.59004,37.76270,195.0,54,no,55.59004 37.7627,1021,0.006394,"Московская область, Ленинский район, ЖК Римский",55.58933,37.75764,338.0,77,no,55.58933 37.75764,NaN
14,183,2440,3630000030756,50:10:0080107:689,126.0,2024-11-01,19000000.0,2024-09-16,NaN,"Россия, Москва, Новомосковский административны...",55.633185,37.462056,МО,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,более_105,6517,16.687379,от_14_до_19_км,265,0.096832,"улица Академика Волгина, 2, Москва",55.656099,37.522117,4549,38,55.633185 37.462056,no,55.656099 37.522117,нет,65606,0.008449,"Москва, улица Академика Варги, 22",55.63358,37.47143,590.0,37,no,55.63358 37.47143,20993,0.017906,"Москва, Мосрентген район, Дудкино д., д.2, стр...",55.62931,37.44988,877.0,41,no,55.62931 37.44988,NaN
15,184,2440,3630000008346,50:20:0000000:173579,48.0,2024-11-01,10500000.0,2024-01-15,NaN,"Россия, Московская область, Одинцово, Северная...",55.689218,37.255075,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.034582,от_19_до_30_км,243,0.207434,"улица Янковского, 1к3, Москва",55.695611,37.473510,13709,37,55.689218 37.255075,no,55.695611 37.47351,нет,12965,0.002216,"Московская область, Одинцово, Северная улица, 55",55.68780,37.25559,161.0,97,no,55.6878 37.25559,7952,0.019641,"Московская область, Одинцово, улица Маршала Жу...",55.68088,37.26005,978.0,81,no,55.68088 37.26005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250727077080959939521089096122649957214,50:21:

# subway_time_1

In [288]:
mapping = dict(zip(df_data['lat_lng'], df_data['subway_time_1']))
df_in_second['subway_time_1'] = df_in_second['Сцепка_к_ОА_second'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [289]:
mapping = dict(zip(df_data_1['lat_lng'], df_data_1['Раст_метро_1']))
df_in_first['subway_time_1'] = df_in_first['Сцепка_к_ОА_first'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [290]:
df_has_info = pd.concat([df_in_second, df_in_first])

In [291]:
mapping = dict(zip(df_has_info['ID '], df_has_info['subway_time_1']))
df_310['subway_time_1'] = df_310['ID '].apply(lambda x: mapping.get(x))

In [292]:
df_310[df_310['subway_time_1'].isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183,NaN,None
8,46,1151,10000001363,50:15:0011002:330,100.7,2024-11-01,15105000.0,2024-05-29,NaN,"Россия, Московская область, Балашиха, микрорай...",55.826321,37.923790,МО,Балашиха-2,МО_В,Балашиха-2,За_МКАД,Московская область,За_МКАД,cosmetic,от_85_до_105,4379,20.562710,от_19_до_30_км,307,0.270563,"улица Вильгельма Пика, 1, Москва",55.834699,37.638961,17812,25,55.826321 37.92379,no,55.834699 37.638961,нет,26410,0.000649,"Московская область, Балашиха-2, Гагарина мкр, 29",55.82627,37.92448,43.0,93,no,55.82627 37.92448,10747,0.012322,"Московская область, Балашиха, жилой комплекс И...",55.82727,37.93569,751.0,69,no,55.82727 37.93569,NaN,None
11,159,1067,48634340937,50:21:0050101:6807,56.9,2024-11-01,9195000.0,2024-05-27,NaN,"Россия, Московская область, Ленинский городско...",55.588287,37.762686,МО,Развилка,МО_Ю,Развилка_Совхоз_им_Ленина,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.375862,от_19_до_30_км,238,0.276469,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12810,31,55.588287 37.762686,no,55.690522 37.668606,нет,55946,0.002211,"Московская область, Развилка_Совхоз_им_Ленина,...",55.59004,37.76270,195.0,54,no,55.59004 37.7627,1021,0.006394,"Московская область, Ленинский район, ЖК Римский",55.58933,37.75764,338.0,77,no,55.58933 37.75764,NaN,None
14,183,2440,3630000030756,50:10:0080107:689,126.0,2024-11-01,19000000.0,2024-09-16,NaN,"Россия, Москва, Новомосковский административны...",55.633185,37.462056,МО,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,более_105,6517,16.687379,от_14_до_19_км,265,0.096832,"улица Академика Волгина, 2, Москва",55.656099,37.522117,4549,38,55.633185 37.462056,no,55.656099 37.522117,нет,65606,0.008449,"Москва, улица Академика Варги, 22",55.63358,37.47143,590.0,37,no,55.63358 37.47143,20993,0.017906,"Москва, Мосрентген район, Дудкино д., д.2, стр...",55.62931,37.44988,877.0,41,no,55.62931 37.44988,NaN,None
15,184,2440,3630000008346,50:20:0000000:173579,48.0,2024-11-01,10500000.0,2024-01-15,NaN,"Россия, Московская область, Одинцово, Северная...",55.689218,37.255075,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.034582,от_19_до_30_км,243,0.207434,"улица Янковского, 1к3, Москва",55.695611,37.473510,13709,37,55.689218 37.255075,no,55.695611 37.47351,нет,12965,0.002216,"Московская область, Одинцово, Северная улица, 55",55.68780,37.25559,161.0,97,no,55.6878 37.25559,7952,0.019641,"Московская область, Одинцово, улица Маршала Жу...",55.68088,37.26005,978.0,81,no,55.68088 37.26005,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,4486,1000,250

# Год постройки

In [293]:
mapping = dict(zip(df_data['lat_lng'], df_data['bti_year_built']))
df_in_second['bti_year_built'] = df_in_second['Сцепка_к_ОА_second'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [294]:
mapping = dict(zip(df_data_1['lat_lng'], df_data_1['ГП']))
df_in_first['bti_year_built'] = df_in_first['Сцепка_к_ОА_first'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [296]:
df_has_info = pd.concat([df_in_second, df_in_first])

In [297]:
mapping = dict(zip(df_has_info['ID '], df_has_info['bti_year_built']))
df_310['bti_year_built'] = df_310['ID '].apply(lambda x: mapping.get(x))

In [298]:
df_310[df_310['bti_year_built'].isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183,NaN,None,NaN
8,46,1151,10000001363,50:15:0011002:330,100.7,2024-11-01,15105000.0,2024-05-29,NaN,"Россия, Московская область, Балашиха, микрорай...",55.826321,37.923790,МО,Балашиха-2,МО_В,Балашиха-2,За_МКАД,Московская область,За_МКАД,cosmetic,от_85_до_105,4379,20.562710,от_19_до_30_км,307,0.270563,"улица Вильгельма Пика, 1, Москва",55.834699,37.638961,17812,25,55.826321 37.92379,no,55.834699 37.638961,нет,26410,0.000649,"Московская область, Балашиха-2, Гагарина мкр, 29",55.82627,37.92448,43.0,93,no,55.82627 37.92448,10747,0.012322,"Московская область, Балашиха, жилой комплекс И...",55.82727,37.93569,751.0,69,no,55.82727 37.93569,NaN,None,NaN
11,159,1067,48634340937,50:21:0050101:6807,56.9,2024-11-01,9195000.0,2024-05-27,NaN,"Россия, Московская область, Ленинский городско...",55.588287,37.762686,МО,Развилка,МО_Ю,Развилка_Совхоз_им_Ленина,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.375862,от_19_до_30_км,238,0.276469,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12810,31,55.588287 37.762686,no,55.690522 37.668606,нет,55946,0.002211,"Московская область, Развилка_Совхоз_им_Ленина,...",55.59004,37.76270,195.0,54,no,55.59004 37.7627,1021,0.006394,"Московская область, Ленинский район, ЖК Римский",55.58933,37.75764,338.0,77,no,55.58933 37.75764,NaN,None,NaN
14,183,2440,3630000030756,50:10:0080107:689,126.0,2024-11-01,19000000.0,2024-09-16,NaN,"Россия, Москва, Новомосковский административны...",55.633185,37.462056,МО,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,более_105,6517,16.687379,от_14_до_19_км,265,0.096832,"улица Академика Волгина, 2, Москва",55.656099,37.522117,4549,38,55.633185 37.462056,no,55.656099 37.522117,нет,65606,0.008449,"Москва, улица Академика Варги, 22",55.63358,37.47143,590.0,37,no,55.63358 37.47143,20993,0.017906,"Москва, Мосрентген район, Дудкино д., д.2, стр...",55.62931,37.44988,877.0,41,no,55.62931 37.44988,NaN,None,NaN
15,184,2440,3630000008346,50:20:0000000:173579,48.0,2024-11-01,10500000.0,2024-01-15,NaN,"Россия, Московская область, Одинцово, Северная...",55.689218,37.255075,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.034582,от_19_до_30_км,243,0.207434,"улица Янковского, 1к3, Москва",55.695611,37.473510,13709,37,55.689218 37.255075,no,55.695611 37.47351,нет,12965,0.002216,"Московская область, Одинцово, Северная улица, 55",55.68780,37.25559,161.0,97,no,55.6878 37.25559,7952,0.019641,"Московская область, Одинцово, улица Маршала Жу...",55.68088,37.26005,978.0,81,no,55.68088 37.26005,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [299]:
#writer = pd.ExcelWriter('!310_обогащение.xlsx')
#df_310.to_excel(writer, '310_обогащение')
#writer.save()

In [310]:
#df_310 = pd.read_excel('!310_обогащение.xlsx', sheet_name = 0)
#df_310 = df_310.drop(columns = ['Unnamed: 0'])
#df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502,89,no,55.68166 37.39183,NaN,NaN,NaN
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14.0,нет_пешей_доступности,1999.0
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14.0,нет_пешей_доступности,1999.0
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617,69,no,55.68096 37.91318,27.0,от_17_до_30_минут,2015.0
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447,64,no,55.67152 37.23772,17.0,нет_пешей_доступности,2013.0
...,...,...,...,.

# Заполнение пропусков средними значениями

In [311]:
df_data.head(2)

,cian_id,photo,url,bargainterms_price,bargainterms_price_LN,area_total,area_total_1,unit_price,unit_price_LN,rooms_count,rooms_count_1,floor_1,floor,floors_count,wall_type,last_edited,repair_type,lat,lng,lat_lng,geo_region,geo_settlement_1,geo_city_okrug_1,geo_city_district_1,geo_street,geo_house_num,distance_to_centre,distance_to_centre_1,rings_MSK,rings_MKAD,subway_name,subway_travel_type,subway_time_1,subway_time,nearest_metro,distance_to_nearest_metro,description,ptn_dadd,Регион_enc,med_price,Районы_Kmeans,type,address_elit,bti_year_built,is_elit,address_full,is_renovation,address_renovation,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,238345845,https://images.cdn-cian.ru/images/78/391/501/k...,https://www.cian.ru/sale/flat/238345845,13140000,16.391172,65.7,от_65_до_85,199999.0,12.206068,2-комн.,3-к,последний,8,8,Монолитный,2024-10-04 00:00:00,no,55.81953,37.24588,55.81953 37.24588,Московская область,Красногорск,МО_З,Красногорск,улица Пришвина,15,24.614772,от_19_до_30_км,За_МКАД,За_МКАД,Опалиха,walk,до_6_минут,6.0,D2_Опалиха,0.416612,Вашему вниманию предлагается 2-х комнатная ква...,2024-10-16 00:00:00,1,208979.0,4,нет,нет,2020,нет,"Московская область, Красногорск, улица Пришвин...",нет,нет,10900,4,0,1,0,0,0,0,0,1,2,0
1,239198805,https://images.cdn-cian.ru/images/3/648/639/93...,https://www.cian.ru/sale/flat/239198805,7550000,15.837058,50.0,от_42_до_65,151000.0,11.925035,2-комн.,2-к,первый,1,2,Кирпичный,2024-09-28 00:00:00,cosmetic,55.63384,37.84267,55.63384 37.84267,Московская область,Дзержинский,МО_ЮВ,Дзержинский,улица Строителей,10,19.198938,от_19_до_30_км,За_МКАД,За_МКАД,Котельники,transport,нет_пешей_доступности,10.0,Таганско_Краснопресненская_Котельники,4.589888,"Квартира для тех, кто ценит тишину и спокойст...",2024-10-16 00:00:00,1,159635.0,2,нет,нет,1956,нет,"Московская область, Дзержинский, улица Строите...",нет,нет,25994,2,0,1,0,0,0,0,0,0,1,0


In [312]:
df_310.head(3)

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502,89,no,55.68166 37.39183,NaN,NaN,NaN
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14.0,нет_пешей_доступности,1999.0
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14.0,нет_пешей_доступности,1999.0


In [307]:
no_info = df_310[(df_310['in_first'] == 'no') & (df_310['in_second'] == 'no')]
no_info

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183,NaN,None,NaN
8,46,1151,10000001363,50:15:0011002:330,100.7,2024-11-01,15105000.0,2024-05-29,NaN,"Россия, Московская область, Балашиха, микрорай...",55.826321,37.923790,МО,Балашиха-2,МО_В,Балашиха-2,За_МКАД,Московская область,За_МКАД,cosmetic,от_85_до_105,4379,20.562710,от_19_до_30_км,307,0.270563,"улица Вильгельма Пика, 1, Москва",55.834699,37.638961,17812,25,55.826321 37.92379,no,55.834699 37.638961,нет,26410,0.000649,"Московская область, Балашиха-2, Гагарина мкр, 29",55.82627,37.92448,43.0,93,no,55.82627 37.92448,10747,0.012322,"Московская область, Балашиха, жилой комплекс И...",55.82727,37.93569,751.0,69,no,55.82727 37.93569,NaN,None,NaN
11,159,1067,48634340937,50:21:0050101:6807,56.9,2024-11-01,9195000.0,2024-05-27,NaN,"Россия, Московская область, Ленинский городско...",55.588287,37.762686,МО,Развилка,МО_Ю,Развилка_Совхоз_им_Ленина,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.375862,от_19_до_30_км,238,0.276469,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12810,31,55.588287 37.762686,no,55.690522 37.668606,нет,55946,0.002211,"Московская область, Развилка_Совхоз_им_Ленина,...",55.59004,37.76270,195.0,54,no,55.59004 37.7627,1021,0.006394,"Московская область, Ленинский район, ЖК Римский",55.58933,37.75764,338.0,77,no,55.58933 37.75764,NaN,None,NaN
14,183,2440,3630000030756,50:10:0080107:689,126.0,2024-11-01,19000000.0,2024-09-16,NaN,"Россия, Москва, Новомосковский административны...",55.633185,37.462056,МО,999,Новомосковский_административный_округ,НАО_Коммунарка,За_МКАД,Москва,За_МКАД,cosmetic,более_105,6517,16.687379,от_14_до_19_км,265,0.096832,"улица Академика Волгина, 2, Москва",55.656099,37.522117,4549,38,55.633185 37.462056,no,55.656099 37.522117,нет,65606,0.008449,"Москва, улица Академика Варги, 22",55.63358,37.47143,590.0,37,no,55.63358 37.47143,20993,0.017906,"Москва, Мосрентген район, Дудкино д., д.2, стр...",55.62931,37.44988,877.0,41,no,55.62931 37.44988,NaN,None,NaN
15,184,2440,3630000008346,50:20:0000000:173579,48.0,2024-11-01,10500000.0,2024-01-15,NaN,"Россия, Московская область, Одинцово, Северная...",55.689218,37.255075,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.034582,от_19_до_30_км,243,0.207434,"улица Янковского, 1к3, Москва",55.695611,37.473510,13709,37,55.689218 37.255075,no,55.695611 37.47351,нет,12965,0.002216,"Московская область, Одинцово, Северная улица, 55",55.68780,37.25559,161.0,97,no,55.6878 37.25559,7952,0.019641,"Московская область, Одинцово, улица Маршала Жу...",55.68088,37.26005,978.0,81,no,55.68088 37.26005,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [306]:
no_info[no_info['metrs_data'] <= 200]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65.0,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502.0,89,no,55.68166 37.39183,NaN,None,NaN
8,46,1151,10000001363,50:15:0011002:330,100.7,2024-11-01,15105000.0,2024-05-29,NaN,"Россия, Московская область, Балашиха, микрорай...",55.826321,37.923790,МО,Балашиха-2,МО_В,Балашиха-2,За_МКАД,Московская область,За_МКАД,cosmetic,от_85_до_105,4379,20.562710,от_19_до_30_км,307,0.270563,"улица Вильгельма Пика, 1, Москва",55.834699,37.638961,17812,25,55.826321 37.92379,no,55.834699 37.638961,нет,26410,0.000649,"Московская область, Балашиха-2, Гагарина мкр, 29",55.82627,37.92448,43.0,93,no,55.82627 37.92448,10747,0.012322,"Московская область, Балашиха, жилой комплекс И...",55.82727,37.93569,751.0,69,no,55.82727 37.93569,NaN,None,NaN
11,159,1067,48634340937,50:21:0050101:6807,56.9,2024-11-01,9195000.0,2024-05-27,NaN,"Россия, Московская область, Ленинский городско...",55.588287,37.762686,МО,Развилка,МО_Ю,Развилка_Совхоз_им_Ленина,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.375862,от_19_до_30_км,238,0.276469,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,12810,31,55.588287 37.762686,no,55.690522 37.668606,нет,55946,0.002211,"Московская область, Развилка_Совхоз_им_Ленина,...",55.59004,37.76270,195.0,54,no,55.59004 37.7627,1021,0.006394,"Московская область, Ленинский район, ЖК Римский",55.58933,37.75764,338.0,77,no,55.58933 37.75764,NaN,None,NaN
15,184,2440,3630000008346,50:20:0000000:173579,48.0,2024-11-01,10500000.0,2024-01-15,NaN,"Россия, Московская область, Одинцово, Северная...",55.689218,37.255075,МО,Одинцово,МО_ЮЗ,Одинцово,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.034582,от_19_до_30_км,243,0.207434,"улица Янковского, 1к3, Москва",55.695611,37.473510,13709,37,55.689218 37.255075,no,55.695611 37.47351,нет,12965,0.002216,"Московская область, Одинцово, Северная улица, 55",55.68780,37.25559,161.0,97,no,55.6878 37.25559,7952,0.019641,"Московская область, Одинцово, улица Маршала Жу...",55.68088,37.26005,978.0,81,no,55.68088 37.26005,NaN,None,NaN
16,185,2440,3630000008347,50:45:0000000:49529,54.0,2024-11-01,7700000.0,2024-01-15,NaN,"Россия, Московская область, Королёв, микрорайо...",55.930574,37.851009,МО,Королев,МО_СВ,Королев,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,24.362874,от_19_до_30_км,284,0.333166,"проезд Серебрякова, 11к2, Москва",55.848897,37.650325,15462,31,55.930574 37.851009,no,55.848897 37.650325,нет,12212,0.000962,"Московская область, Королев, улица Маяковского...",55.92984,37.85096,82.0,66,no,55.92984 37.85096,9445,0.032921,"Московская область, Королёв, микрорайон Первом...",55.94013,37.86841,1518.0,77,no,55.94013 37.86841,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### Метро по ближайшему соседу

In [313]:
df_310 = df_310[(df_310['in_first'] == 'yes') | (df_310['in_second'] == 'yes') | ((df_310['in_first'] == 'no') & (df_310['in_second'] == 'no') & (df_310['metrs_data'] <= 200))]
df_310.reset_index(drop = True , inplace = True)
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502,89,no,55.68166 37.39183,NaN,NaN,NaN
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14.0,нет_пешей_доступности,1999.0
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14.0,нет_пешей_доступности,1999.0
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617,69,no,55.68096 37.91318,27.0,от_17_до_30_минут,2015.0
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина, 2, Москва",55.656543,37.521707,18592,27,55.660516 37.225395,no,55.656543 37.521707,нет,4571,0.000012,"Московская область, ВНИИССОК, улица Дениса Дав...",55.66051,37.22539,1,100,yes,55.66051 37.22539,2711,0.031046,"Московская область, Одинцовский район, Одинцов...",55.67152,37.23772,1447,64,no,55.67152 37.23772,17.0,нет_пешей_доступности,2013.0
...,...,...,...,.

In [314]:
mapping = dict(zip(df_data['lat_lng'], df_data['subway_time_1']))
df_310['subway_time_2'] = df_310['Сцепка_к_ОА_second'].apply(lambda x: mapping.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [315]:
df_310['subway_time_1'] = np.where(df_310['subway_time_1'].isnull(), df_310['subway_time_2'], df_310['subway_time_1'])

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [330]:
df_mean = df_data.groupby('geo_city_district_1')[['floors_count', 'bti_year_built']].mean().round()
df_mean = df_mean.reset_index()
df_mean

,geo_city_district_1,floors_count,bti_year_built
0,1_Мая_мкр,17.0,2004.0
1,Архангельское,6.0,1989.0
2,Балашиха-1,13.0,1988.0
3,Балашиха-2,19.0,2008.0
4,Бородино,15.0,2019.0
...,...,...,...
177,ЮЗАО_Черемушки,15.0,1987.0
178,ЮЗАО_Южное Бутово,17.0,2006.0
179,ЮЗАО_Ясенево,13.0,1980.0
180,Южная_Битца_ЖК,24.0,2022.0


In [343]:
df_mean['floors_count'].describe()

count    182.000000
mean      15.093407
std        4.335012
min        5.000000
25%       12.000000
50%       15.000000
75%       18.000000
max       29.000000
Name: floors_count, dtype: float64

## Этажность по среднему

In [ ]:
mapping_1 = dict(zip(df_mean['geo_city_district_1'], df_mean['floors_count']))
df_310['floors_count_mean'] = df_310['poly_r'].apply(lambda x: mapping_1.get(x))

In [336]:
df_310['floors_count'] = np.where(df_310['floors_count'].isnull(), df_310['floors_count_mean'], df_310['floors_count'])

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [337]:
df_310[df_310['floors_count'].isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built,subway_time_2,floors_count_mean


## ГП по среднему

In [338]:
mapping_2 = dict(zip(df_mean['geo_city_district_1'], df_mean['bti_year_built']))
df_310['bti_year_built_mean'] = df_310['poly_r'].apply(lambda x: mapping_2.get(x))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [339]:
df_310['bti_year_built'] = np.where(df_310['bti_year_built'].isnull(), df_310['bti_year_built_mean'], df_310['bti_year_built'])

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [340]:
df_310[df_310['bti_year_built'].isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built,subway_time_2,floors_count_mean,bti_year_built_mean


In [341]:
#writer = pd.ExcelWriter('!310_обогащение.xlsx')
#df_310.to_excel(writer, '310_обогащение')
#writer.save()

In [353]:
df_310.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227 entries, 0 to 3226
Data columns (total 60 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    3227 non-null   int64         
 1   bank_code             3227 non-null   int64         
 2   obj_cd                3227 non-null   object        
 3   kadastr_no            3227 non-null   object        
 4   area_val              3227 non-null   float64       
 5   report_date           3227 non-null   object        
 6   market_tot_val        3227 non-null   float64       
 7   market_tot_dt         3227 non-null   datetime64[ns]
 8   Адрес                 2515 non-null   object        
 9   Адрес Yandex          3227 non-null   object        
 10  lat                   3227 non-null   float64       
 11  lon                   3227 non-null   float64       
 12  М_МО                  3227 non-null   object        
 13  poly_sputnik      

In [349]:
df_310['floors_count'] = df_310['floors_count'].astype(int)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [351]:
df_310['floor_0.5'] = (df_310['floors_count'] / 2).astype(int)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_310['floor_0.5'].describe()

count    3227.000000
mean        9.547877
std         4.422206
min         1.000000
25%         6.000000
50%         9.000000
75%        12.000000
max        33.000000
Name: floor_0.5, dtype: float64

In [ ]:
df_data.head(2)

,cian_id,photo,url,bargainterms_price,bargainterms_price_LN,area_total,area_total_1,unit_price,unit_price_LN,rooms_count,rooms_count_1,floor_1,floor,floors_count,wall_type,last_edited,repair_type,lat,lng,lat_lng,geo_region,geo_settlement_1,geo_city_okrug_1,geo_city_district_1,geo_street,geo_house_num,distance_to_centre,distance_to_centre_1,rings_MSK,rings_MKAD,subway_name,subway_travel_type,subway_time_1,subway_time,nearest_metro,distance_to_nearest_metro,description,ptn_dadd,Регион_enc,med_price,Районы_Kmeans,type,address_elit,bti_year_built,is_elit,address_full,is_renovation,address_renovation,Объем_рынка,Районы_Kmeans_str,IsolationForest,OneClassSVM,LOF,COPOD,PCA,out_iqr,out_iqr_kv,Manual_autoencoder,sum_out_ml,sum_out_iqr
0,238345845,https://images.cdn-cian.ru/images/78/391/501/k...,https://www.cian.ru/sale/flat/238345845,13140000,16.391172,65.7,от_65_до_85,199999.0,12.206068,2-комн.,3-к,последний,8,8,Монолитный,2024-10-04 00:00:00,no,55.81953,37.24588,55.81953 37.24588,Московская область,Красногорск,МО_З,Красногорск,улица Пришвина,15,24.614772,от_19_до_30_км,За_МКАД,За_МКАД,Опалиха,walk,до_6_минут,6.0,D2_Опалиха,0.416612,Вашему вниманию предлагается 2-х комнатная ква...,2024-10-16 00:00:00,1,208979.0,4,нет,нет,2020,нет,"Московская область, Красногорск, улица Пришвин...",нет,нет,10900,4,0,1,0,0,0,0,0,1,2,0
1,239198805,https://images.cdn-cian.ru/images/3/648/639/93...,https://www.cian.ru/sale/flat/239198805,7550000,15.837058,50.0,от_42_до_65,151000.0,11.925035,2-комн.,2-к,первый,1,2,Кирпичный,2024-09-28 00:00:00,cosmetic,55.63384,37.84267,55.63384 37.84267,Московская область,Дзержинский,МО_ЮВ,Дзержинский,улица Строителей,10,19.198938,от_19_до_30_км,За_МКАД,За_МКАД,Котельники,transport,нет_пешей_доступности,10.0,Таганско_Краснопресненская_Котельники,4.589888,"Квартира для тех, кто ценит тишину и спокойст...",2024-10-16 00:00:00,1,159635.0,2,нет,нет,1956,нет,"Московская область, Дзержинский, улица Строите...",нет,нет,25994,2,0,1,0,0,0,0,0,0,1,0


In [ ]:
df_data.floor_1.unique()

array(['последний', 'первый', 'этажи_2-5', 'этажи_24-33', 'этажи_6-9',
       'этажи_10-15', 'этажи_16-23', 'более_34'], dtype=object)

In [356]:
f_labels = ['этажи_2-5', 'этажи_6-9', 'этажи_10-15', 'этажи_16-23', 'этажи_24-33', 'более_34']  
f_bins = [0, 5, 9, 15, 23, 33, 10000]

In [357]:
df_310['floor_1'] = pd.cut(df_310['floor_0.5'], bins = f_bins, labels = f_labels)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [358]:
df_310[df_310['floor_1'].isnull()]

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built,subway_time_2,floors_count_mean,bti_year_built_mean,floor_0.5,floor_1


In [359]:
df_310['floor_1'].value_counts()

этажи_10-15    1360
этажи_6-9      1105
этажи_2-5       590
этажи_16-23      87
этажи_24-33      85
более_34          0
Name: floor_1, dtype: int64

In [360]:
df_310['floor_1'].unique()

['этажи_2-5', 'этажи_6-9', 'этажи_10-15', 'этажи_16-23', 'этажи_24-33']
Categories (6, object): ['этажи_2-5' < 'этажи_6-9' < 'этажи_10-15' < 'этажи_16-23' < 'этажи_24-33' < 'более_34']

In [363]:
df_310.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227 entries, 0 to 3226
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    3227 non-null   int64         
 1   bank_code             3227 non-null   int64         
 2   obj_cd                3227 non-null   object        
 3   kadastr_no            3227 non-null   object        
 4   area_val              3227 non-null   float64       
 5   report_date           3227 non-null   object        
 6   market_tot_val        3227 non-null   float64       
 7   market_tot_dt         3227 non-null   datetime64[ns]
 8   Адрес                 2515 non-null   object        
 9   Адрес Yandex          3227 non-null   object        
 10  lat                   3227 non-null   float64       
 11  lon                   3227 non-null   float64       
 12  М_МО                  3227 non-null   object        
 13  poly_sputnik      

In [362]:
df_310['bti_year_built'] = df_310['bti_year_built'].astype(int)

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [367]:
df_310['subway_time_1'].unique()

array(['нет_пешей_доступности', 'от_17_до_30_минут', 'до_6_минут',
       'от_12_до_17_минут', 'от_6_до_12_минут', 'неважно'], dtype=object)

In [365]:
df_data['subway_time_1'].unique()

array(['до_6_минут', 'нет_пешей_доступности', 'от_6_до_12_минут',
       'от_12_до_17_минут', 'неважно', 'от_17_до_30_минут'], dtype=object)

In [366]:
df_310['subway_time_1'] = df_310['subway_time_1'].replace('до_7_минут', 'до_6_минут')
df_310['subway_time_1'] = df_310['subway_time_1'].replace('от_16_до_30_минут', 'от_17_до_30_минут')

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [368]:
#writer = pd.ExcelWriter('!310_обогащение.xlsx')
#df_310.to_excel(writer, '310_обогащение')
#writer.save()

# CATBOOST

In [77]:
#загрузка 310
df_310 = pd.read_excel('310.xlsx', sheet_name = 0)
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built,subway_time_2,floors_count_mean,bti_year_built_mean,floor_0.5,floor_1,pred_cat,pred_cat_2
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502,89,no,55.68166 37.39183,9,нет_пешей_доступности,2011,нет_пешей_доступности,9,2011,4,этажи_2-5,1.089047e+08,1.089047e+08
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14,нет_пешей_доступности,1999,нет_пешей_доступности,12,1992,7,этажи_6-9,8.999633e+06,8.999633e+06
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14,нет_пешей_доступности,1999,нет_пешей_доступности,12,1992,7,этажи_6-9,9.353351e+06,9.353351e+06
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617,69,no,55.68096 37.91318,27,от_17_до_30_минут,2015,от_17_до_30_минут,16,1998,13,этажи_10-15,1.012977e+07,1.012977e+07
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,10900,26.881163,от_19_до_30_км,266,0.272550,"улица Академика Волгина

In [78]:
df_310.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227 entries, 0 to 3226
Data columns (total 63 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    3227 non-null   int64         
 1   bank_code             3227 non-null   int64         
 2   obj_cd                3227 non-null   object        
 3   kadastr_no            3227 non-null   object        
 4   area_val              3227 non-null   float64       
 5   report_date           3227 non-null   object        
 6   market_tot_val        3227 non-null   float64       
 7   market_tot_dt         3227 non-null   datetime64[ns]
 8   Адрес                 2515 non-null   object        
 9   Адрес Yandex          3227 non-null   object        
 10  lat                   3227 non-null   float64       
 11  lon                   3227 non-null   float64       
 12  М_МО                  3227 non-null   object        
 13  poly_sputnik      

In [79]:
X = df_310[[
        #'bargainterms_price_LN', 
        'area_val',
        #'floor', 
        'floor_1', 
        'floors_count',
        'type', 
        'geo_region',
        'poly_r', 
        #'Районы_Kmeans',
        #'Районы_Kmeans_str', 
        'poly_circle', 
        'poly_okrug', 
        #'distance_to_nearest_metro',
        'subway_time_1', 
        #'distance_to_centre', 
        'distance_to_centre_1', 
        'rings_MKAD', 
        'repair_type',
        #'is_renovation',
        'bti_year_built',
        'Объем_рынка'
      ]]

In [80]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227 entries, 0 to 3226
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   area_val              3227 non-null   float64
 1   floor_1               3227 non-null   object 
 2   floors_count          3227 non-null   int64  
 3   type                  3227 non-null   object 
 4   geo_region            3227 non-null   object 
 5   poly_r                3227 non-null   object 
 6   poly_circle           3227 non-null   object 
 7   poly_okrug            3227 non-null   object 
 8   subway_time_1         3227 non-null   object 
 9   distance_to_centre_1  3227 non-null   object 
 10  rings_MKAD            3227 non-null   object 
 11  repair_type           3227 non-null   object 
 12  bti_year_built        3227 non-null   int64  
 13  Объем_рынка           3227 non-null   int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 353.1+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65527 entries, 0 to 65526
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   area_total            65527 non-null  float64
 1   floor_1               65527 non-null  object 
 2   floors_count          65527 non-null  int64  
 3   is_elit               65527 non-null  object 
 4   geo_region            65527 non-null  object 
 5   geo_city_district_1   65527 non-null  object 
 6   rings_MSK             65527 non-null  object 
 7   geo_city_okrug_1      65527 non-null  object 
 8   subway_time_1         65527 non-null  object 
 9   distance_to_centre_1  65527 non-null  object 
 10  rings_MKAD            65527 non-null  object 
 11  repair_type           65527 non-null  object 
 12  bti_year_built        65527 non-null  int64  
 13  Объем_рынка           65527 non-null  int64  

In [81]:
X = X.rename(columns={'area_val': 'area_total', 'type': 'is_elit', 'poly_r': 'geo_city_district_1', 'poly_circle': 'rings_MSK', 'poly_okrug': 'geo_city_okrug_1'})

In [82]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227 entries, 0 to 3226
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   area_total            3227 non-null   float64
 1   floor_1               3227 non-null   object 
 2   floors_count          3227 non-null   int64  
 3   is_elit               3227 non-null   object 
 4   geo_region            3227 non-null   object 
 5   geo_city_district_1   3227 non-null   object 
 6   rings_MSK             3227 non-null   object 
 7   geo_city_okrug_1      3227 non-null   object 
 8   subway_time_1         3227 non-null   object 
 9   distance_to_centre_1  3227 non-null   object 
 10  rings_MKAD            3227 non-null   object 
 11  repair_type           3227 non-null   object 
 12  bti_year_built        3227 non-null   int64  
 13  Объем_рынка           3227 non-null   int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 353.1+ KB


In [83]:
X['is_elit'].unique()

array(['нет', 'престижный', 'престижный+'], dtype=object)

In [84]:
import pickle
#model_cat = pickle.load(open('catmodel_mmo_second_best.pkl', 'rb'))
model_cat_out = pickle.load(open('catmodel_mmo_second_with_outliers_and_weights.pkl', 'rb'))

In [85]:
pred_cat = np.exp(model_cat_out.predict(X))

In [86]:
df_310['pred_cat_out'] = pred_cat
df_310

,ID,bank_code,obj_cd,kadastr_no,area_val,report_date,market_tot_val,market_tot_dt,Адрес,Адрес Yandex,lat,lon,М_МО,poly_sputnik,poly_okrug,poly_r,poly_circle,geo_region,rings_MKAD,repair_type,ОПлощадь_cut,Объем_рынка,distance_to_centre,distance_to_centre_1,nn_index,dist,OA_add,lat_OA,lon_OA,metrs,sim,Сцепка_к,is_elit,Сцепка_к_ОА,type,nn_index_data,dist_data,OA_add_data,lat_OA_data,lon_OA_data,metrs_data,sim_data,in_second,Сцепка_к_ОА_second,nn_index_data_1,dist_data_1,OA_add_data_1,lat_OA_data_1,lon_OA_data_1,metrs_data_1,sim_data_1,in_first,Сцепка_к_ОА_first,floors_count,subway_time_1,bti_year_built,subway_time_2,floors_count_mean,bti_year_built_mean,floor_0.5,floor_1,pred_cat,pred_cat_2,pred_cat_out
0,1,436,1287986428866,50:20:0020202:3956,226.9,2024-11-01,42500000.0,2022-11-02,NaN,"Россия, Московская область, Одинцовский городс...",55.677143,37.392005,МО,Заречье,МО_ЮЗ,Заречье,За_МКАД,Московская область,За_МКАД,cosmetic,более_105,6517,16.690113,от_14_до_19_км,243,0.107652,"улица Янковского, 1к3, Москва",55.695611,37.473510,5506,20,55.677143 37.392005,no,55.695611 37.47351,нет,18987,0.000957,"Московская область, Заречье, 12 Месяцев ЖК, 9",55.67708,37.39304,65,94,no,55.67708 37.39304,13389,0.008397,"Московская область, Одинцовский городской окру...",55.68166,37.39183,502,89,no,55.68166 37.39183,9,нет_пешей_доступности,2011,нет_пешей_доступности,9,2011,4,этажи_2-5,1.089047e+08,1.089047e+08,8.341101e+07
1,7,3300,3620000014599,50:64:0000000:16852,55.0,2024-11-01,8391000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 37","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14,нет_пешей_доступности,1999,нет_пешей_доступности,12,1992,7,этажи_6-9,8.999633e+06,8.999633e+06,8.667681e+06
2,8,3300,3620000014600,50:64:0000000:16853,58.0,2024-11-01,8752000.0,2022-07-05,"г Дзержинский, ул Угрешская, д. 26б, 38","Россия, Московская область, Дзержинский, Угреш...",55.631829,37.861357,МО,Дзержинский,МО_ЮВ,Дзержинский,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,20.209697,от_19_до_30_км,238,0.282835,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,13739,36,55.631829 37.861357,no,55.690522 37.668606,нет,675,0.000004,"Московская область, Дзержинский, Угрешская ули...",55.63183,37.86136,0,100,yes,55.63183 37.86136,15995,0.021481,"Московская область, Котельники, микрорайон Нов...",55.64037,37.85473,1037,54,no,55.64037 37.85473,14,нет_пешей_доступности,1999,нет_пешей_доступности,12,1992,7,этажи_6-9,9.353351e+06,9.353351e+06,9.072114e+06
3,38,1052,GUAR_OBJECT_2358_98_1,50:22:0010109:32905,60.0,2024-11-01,10081000.0,2023-08-15,NaN,"Россия, Московская область, Люберцы, микрорайо...",55.686508,37.913540,МО,Люберцы,МО_ЮВ,Люберцы,За_МКАД,Московская область,За_МКАД,cosmetic,от_42_до_65,25994,19.738682,от_19_до_30_км,238,0.226665,"проспект Андропова, 9/1к3, Москва",55.690522,37.668606,15359,41,55.686508 37.91354,no,55.690522 37.668606,нет,5362,0.000120,"Московская область, Люберцы, проспект Победы, ...",55.68646,37.91361,7,100,yes,55.68646 37.91361,5125,0.010443,"Московская область, Люберцы, Рождественская ул...",55.68096,37.91318,617,69,no,55.68096 37.91318,27,от_17_до_30_минут,2015,от_17_до_30_минут,16,1998,13,этажи_10-15,1.012977e+07,1.012977e+07,1.019827e+07
4,39,2721,3610000000543,50:20:0070227:4824,79.5,2024-11-01,12000000.0,2022-07-19,"Одинцовский р-н, п ВНИИССОК, ул Дениса Давыдов...","Россия, Московская область, Одинцовский городс...",55.660516,37.225395,МО,ВНИИССОК,ВНИИССОК,ВНИИССОК,За_МКАД,Московская область,За_МКАД,cosmetic,от_65_до_85,109

In [87]:
writer = pd.ExcelWriter('310_out.xlsx')
df_310.to_excel(writer, '310_обогащение')
writer.save()